## 定义函数信道等

In [ ]:
# coding:utf-8
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 使用第二块GPU（从0开始）
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import argparse
import time

import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

from math import *
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
from PIL import Image

C_dim = 64  # 语义编码的通道数
n_class   = 9

In [ ]:
def NMSE(x_hat,x):
    x = torch.reshape(x, (len(x), -1))
    x_hat = torch.reshape(x_hat, (len(x_hat), -1))
    
    power = torch.sum(abs(x) ** 2, axis=1)
    mse = torch.sum(abs(x - x_hat) ** 2, axis=1)
    nmse = torch.mean(mse / power)
    return nmse
def Hermitian(X):#torch矩阵共轭转置
    X = torch.real(X) - 1j*torch.imag(X)
    return X.transpose(-1,-2)
def kron(a,b):
    #a与b维度为[batch,N],输出应为[batch,N*N]
    batch = a.shape[0]
    a = a.reshape(batch,-1,1)
    b = b.reshape(batch,1,-1)
    c = a @ b
    return c.reshape(batch,-1) #输出的维度a在前，b在后
def kron_add(a,b):
    #a与b维度为[batch,N],输出应为[batch,N*N]
    batch = a.shape[0]
    a = a.reshape(batch,-1,1)
    b = b.reshape(batch,1,-1)
    c = a + b
    return c.reshape(batch,-1) #输出的维度a在前，b在后

def DFT_matrix(N):
    n = torch.arange(N).reshape(N,1).cuda()
    k = torch.arange(N).reshape(1,N).cuda()
    W = torch.exp(-1j*2*pi*n*k/N)/sqrt(N)
    return W


In [ ]:
fc  = 28e9
c = 3e8
# v_max_km = 120  #120km/h

def SV_Channel_set(batch,Nc,N,Nt,Nr,sigma_2_alpha,v_max_km):
    #Nc代表载波数 N代表路径数 sigma_2_alph每条路径能量 sigma_angle角度均匀分布的幅度 Nt=[Nx,Ny]是UPA天线数,t表示
    v_max = v_max_km*1000/3600
    fd_max = fc*v_max/c

    Tau = 10  #最大路径时延
    tau = torch.rand(batch,1,N).cuda()*Tau
    N_r = Nr[0]*Nr[1]
    N_t = Nt[0]*Nt[1]
    fait = torch.rand(batch,1,N).cuda()*2*pi
    fair = torch.rand(batch,1,N).cuda()*2*pi
    theatt = torch.rand(batch,1,N).cuda()*2*pi
    theatr = torch.rand(batch,1,N).cuda()*2*pi
    A_t = torch.zeros(batch,N_t,N).cuda() +0j
    A_r = torch.zeros(batch,N_r,N).cuda() +0j
    alpha = torch.zeros(batch,1,N).cuda() +0j
    n_t1 = torch.arange(Nt[0]).reshape(1,Nt[0],1).cuda()
    n_t2 = torch.arange(Nt[1]).reshape(1,Nt[1],1).cuda()
    n_r1 = torch.arange(Nr[0]).reshape(1,Nr[0],1).cuda()
    n_r2 = torch.arange(Nr[1]).reshape(1,Nr[1],1).cuda()
    for i in range(N):
        at1 = torch.exp(-2j*pi*0.5*n_t1*torch.cos(fait[:,:,i:(i+1)])*torch.sin(theatt[:,:,i:(i+1)]))
        at2 = torch.exp(-2j*pi*0.5*n_t2*torch.sin(fait[:,:,i:(i+1)]))
        A_t[:,:,i] = kron(at1,at2)

        ar1 = torch.exp(-2j*pi*0.5*n_r1*torch.cos(fair[:,:,i:(i+1)])*torch.sin(theatr[:,:,i:(i+1)]))
        ar2 = torch.exp(-2j*pi*0.5*n_r2*torch.sin(fair[:,:,i:(i+1)]))
        A_r[:,:,i] = kron(ar1,ar2)

        aa = (torch.randn(batch,1)+1j*torch.randn(batch,1))*sqrt(sigma_2_alpha/2)
        # if i == 0:
        #     aa = (torch.randn(batch,1)+1j*torch.randn(batch,1))*sqrt(sigma_2_alpha/2) 
        # else:
        #     aa = (torch.randn(batch,1)+1j*torch.randn(batch,1))*sqrt(sigma_2_alpha/2)*0.1
        alpha[:,:,i] = aa[:,:]

    fd = fd_max * torch.rand(batch,N).cuda()
    return alpha,A_r,A_t,fd,tau

def SV_Channel_final(batch,Nc,N,Nt,Nr,sigma_2_alpha,t,alpha,A_r,A_t,fd,tau):
    #Nc代表载波数 N代表路径数 sigma_2_alph每条路径能量 sigma_angle角度均匀分布的幅度 Nt=[Nx,Ny]是UPA天线数,t表示时间
    d=0.5
    #print(check)
    

    N_r = Nr[0]*Nr[1]
    N_t = Nt[0]*Nt[1]
    
    H_f = torch.zeros(batch,Nc,N_r,N_t).cuda() +0j
    alpha_dop = torch.zeros(batch,1,N).cuda() +0j
    for i in range(N):
        
        
        phi_d = torch.exp(-2j*pi*fd[:,i].reshape(batch,1)*t)
        aa = alpha[:,:,i]*phi_d
        # if i == 0:
        #     aa = (torch.randn(batch,1)+1j*torch.randn(batch,1))*sqrt(sigma_2_alpha/2) 
        # else:
        #     aa = (torch.randn(batch,1)+1j*torch.randn(batch,1))*sqrt(sigma_2_alpha/2)*0.1
        alpha_dop[:,:,i] = aa[:,:]


    for k in range(Nc):
        P = alpha_dop*torch.exp(-1j*2*pi*tau*k/Nc)
        P = torch.diag_embed(P).reshape(batch,N,N)
        H_f[:,k,:,:] = torch.matmul(torch.matmul(A_r,P),Hermitian(A_t))
        
    return H_f   #【batch,Nc,N_r,N_t】N_r是基站接收天线，N_t是用户发送天先

def calculate_average_channel_capacity(H, F, W, SNR_dB): # Pt代表总发射功率 snr代表总发射功率与每一根接收天线噪声功率之比
    #H [batch,Nc,N_r,N_t]     F[batch,Nc,N_t,Ns]  W[batch,Nc,N_r_RF,N_r]
    batch_size, Nc, N_r, N_t = H.shape
    Ns = F.shape[-1]
    N_r_RF = W.shape[2]

    N0 = 1
    Pt = 10**(SNR_dB/10)
    

    


    norm_factor = torch.sqrt(torch.sum(torch.abs(F)**2, dim=(-2, -1), keepdim=True))
    F = F / norm_factor * torch.sqrt(torch.tensor(Ns, dtype=H.dtype, device=H.device))

    norm_factor = torch.sqrt(torch.sum(torch.abs(W)**2, dim=(-2, -1), keepdim=True))
    W = W / norm_factor * torch.sqrt(torch.tensor(N_r_RF, dtype=H.dtype, device=H.device))
    
    # Calculate power per stream
    P_per_stream = Pt / Ns
    
    # Calculate effective channel
    H_eff = W @ H @ F  #H [batch,Nc,N_r_RF,Ns]
    
    # Calculate the identity matrix
    identity_matrix = torch.eye(N_r_RF, dtype=H.dtype, device=H.device).expand(batch_size, Nc, N_r_RF, N_r_RF)
    
    # Calculate the covariance matrix
    covariance_matrix = torch.matmul(H_eff, H_eff.conj().transpose(-1, -2))
    
    # Calculate the determinant part
    det_part = torch.det(identity_matrix + (P_per_stream / N0) * covariance_matrix)
    
    # Calculate the capacity
    capacity = torch.log2(det_part).real  #[b,Nc]
    
    # Calculate average capacity
    average_capacity = capacity.mean(1) # b
    
    return average_capacity # b

#这个函数计算T个时隙的平均谱效
def calculate_average_channel_capacity_T(H, F, W, SNR_dB): # Pt代表总发射功率 snr代表总发射功率与每一根接收天线噪声功率之比
    #H [batch,T, Nc,N_r,N_t]     F[batch,T, Nc,N_t,Ns]  W[batch,T, Nc,N_r_RF,N_r]
    batch_size, T, Nc, N_r, N_t = H.shape
    Ns = F.shape[-1]
    N_r_RF = W.shape[-2]

    N0 = 1
    Pt = 10**(SNR_dB/10)
    

    


    norm_factor = torch.sqrt(torch.sum(torch.abs(F)**2, dim=(-2, -1), keepdim=True))
    F = F / norm_factor * torch.sqrt(torch.tensor(Ns, dtype=H.dtype, device=H.device))

    norm_factor = torch.sqrt(torch.sum(torch.abs(W)**2, dim=(-2, -1), keepdim=True))
    W = W / norm_factor * torch.sqrt(torch.tensor(N_r_RF, dtype=H.dtype, device=H.device))
    
    # Calculate power per stream
    P_per_stream = Pt / Ns
    
    # Calculate effective channel
    H_eff = W @ H @ F  #H [batch,T, Nc,N_r_RF,Ns]
    
    # Calculate the identity matrix
    identity_matrix = torch.eye(N_r_RF, dtype=H.dtype, device=H.device).expand(batch_size, T, Nc, N_r_RF, N_r_RF)
    
    # Calculate the covariance matrix
    covariance_matrix = torch.matmul(H_eff, H_eff.conj().transpose(-1, -2))
    
    # Calculate the determinant part
    det_part = torch.det(identity_matrix + (P_per_stream / N0) * covariance_matrix)
    
    # Calculate the capacity
    capacity = torch.log2(det_part).real  #[b,Nc]
    
    # Calculate average capacity
    average_capacity = capacity.mean(1) # b
    
    return average_capacity # b

def calculate_average_channel_capacity_OMA(H_1, H_2, F_1, F_2, W, SNR_dB): # Pt代表总发射功率 snr代表总发射功率与每一根接收天线噪声功率之比
    #H [batch,Nc,N_r,N_t]     F[batch,Nc,N_t,1]  W[batch,Nc,N_r_RF,N_r]
    batch_size, Nc, N_r, N_t = H_1.shape
    Ns = F_1.shape[-1]
    N_r_RF = W.shape[2]

    N0 = 1
    Pt = 10**(SNR_dB/10)
    

    


    norm_factor = torch.sqrt(torch.sum(torch.abs(F_1)**2, dim=(-2, -1), keepdim=True))
    F_1 = F_1 / norm_factor * torch.sqrt(torch.tensor(Ns, dtype=H_1.dtype, device=H_1.device))
    norm_factor = torch.sqrt(torch.sum(torch.abs(F_2)**2, dim=(-2, -1), keepdim=True))
    F_2 = F_2 / norm_factor * torch.sqrt(torch.tensor(Ns, dtype=H_1.dtype, device=H_1.device))



    norm_factor = torch.sqrt(torch.sum(torch.abs(W)**2, dim=(-2, -1), keepdim=True))
    W = W / norm_factor * torch.sqrt(torch.tensor(N_r_RF, dtype=H_1.dtype, device=H_1.device))
    
    # Calculate power per stream
    P_per_stream = Pt / Ns
    
    # Calculate effective channel
    W_1 = W[:,:,0:1,:] # [batch,Nc,1,N_r]
    W_2 = W[:,:,1:2,:] # [batch,Nc,1,N_r]
    H_eff_11 = W_1 @ H_1 @ F_1  #H [batch,Nc,1,1]
    H_eff_12 = W_2 @ H_1 @ F_1  #H [batch,Nc,1,1]
    H_eff_21 = W_1 @ H_2 @ F_2  #H [batch,Nc,1,1]
    H_eff_22 = W_2 @ H_2 @ F_2  #H [batch,Nc,1,1]


    signal_1 = (P_per_stream ) * H_eff_11 @ H_eff_11.mH
    inter_1 = N0 + (P_per_stream ) * H_eff_21 @ H_eff_21.mH
    capacity_1 = torch.log2(1 + signal_1/inter_1).real.mean(1)  #[b]

    signal_2 = (P_per_stream ) * H_eff_22 @ H_eff_22.mH
    inter_2 = N0 + (P_per_stream ) * H_eff_12 @ H_eff_12.mH
    capacity_2 = torch.log2(1 + signal_2/inter_2).real.mean(1)  #[b]
    
    
    
    return torch.min(capacity_1,capacity_2).mean() 


def calculate_average_channel_capacity_OMA_woInt(H_1, H_2, F_1, F_2, W, SNR_dB): # Pt代表总发射功率 snr代表总发射功率与每一根接收天线噪声功率之比
    #H [batch,Nc,N_r,N_t]     F[batch,Nc,N_t,1]  W[batch,Nc,N_r_RF,N_r]
    batch_size, Nc, N_r, N_t = H_1.shape
    Ns = F_1.shape[-1]
    N_r_RF = W.shape[2]

    N0 = 1
    Pt = 10**(SNR_dB/10)
    

    


    norm_factor = torch.sqrt(torch.sum(torch.abs(F_1)**2, dim=(-2, -1), keepdim=True))
    F_1 = F_1 / norm_factor * torch.sqrt(torch.tensor(Ns, dtype=H_1.dtype, device=H_1.device))
    norm_factor = torch.sqrt(torch.sum(torch.abs(F_2)**2, dim=(-2, -1), keepdim=True))
    F_2 = F_2 / norm_factor * torch.sqrt(torch.tensor(Ns, dtype=H_1.dtype, device=H_1.device))



    norm_factor = torch.sqrt(torch.sum(torch.abs(W)**2, dim=(-2, -1), keepdim=True))
    W = W / norm_factor * torch.sqrt(torch.tensor(N_r_RF, dtype=H_1.dtype, device=H_1.device))
    
    # Calculate power per stream
    P_per_stream = Pt / Ns
    
    # Calculate effective channel
    W_1 = W[:,:,0:1,:] # [batch,Nc,1,N_r]
    W_2 = W[:,:,1:2,:] # [batch,Nc,1,N_r]
    H_eff_11 = W_1 @ H_1 @ F_1  #H [batch,Nc,1,1]
    # H_eff_12 = W_2 @ H_1 @ F_1  #H [batch,Nc,1,1]
    # H_eff_21 = W_1 @ H_2 @ F_2  #H [batch,Nc,1,1]
    H_eff_22 = W_2 @ H_2 @ F_2  #H [batch,Nc,1,1]


    signal_1 = (P_per_stream ) * H_eff_11 @ H_eff_11.mH
    inter_1 = N0
    capacity_1 = torch.log2(1 + signal_1/inter_1).real.mean(1)  #[b]

    signal_2 = (P_per_stream ) * H_eff_22 @ H_eff_22.mH
    inter_2 = N0
    capacity_2 = torch.log2(1 + signal_2/inter_2).real.mean(1)  #[b]
    
    
    
    return torch.min(capacity_1,capacity_2).mean() 

def save_images(tensor, fpath_out):
    """
    保存4通道张量为RGB图像和灰度图像
    
    参数:
        tensor: PyTorch张量, shape [b, 4, H, W]
        fpath_out: 包含b个输出路径的列表, 每个以'.png'结尾
    """
    # 确保输入张量在CPU上
    tensor = tensor.cpu()
    
    # 检查输入是否有效
    assert tensor.size(1) == 4, "输入张量必须有4个通道"
    assert len(fpath_out) == tensor.size(0), "路径数量必须与批次大小匹配"
    
    for i in range(tensor.size(0)):
        # 获取当前图像的路径
        original_path = fpath_out[i]
        
        # 检查路径是否以.png结尾
        assert original_path.endswith('.png'), "路径必须以'.png'结尾"
        
        # 创建RGB路径(在.png前插入'_rgb')
        base, ext = os.path.splitext(original_path)
        rgb_path = f"{base}_rgb{ext}"
        
        # 创建灰度路径(在.png前插入'_inf')
        inf_path = f"{base}_inf{ext}"
        
        # 获取当前图像的4个通道 [4, H, W]
        img_data = tensor[i]
        
        # 保存RGB图像(前3个通道)
        rgb_img = img_data[:3]  # [3, H, W]
        rgb_img = (rgb_img.permute(1, 2, 0).numpy() * 255).astype('uint8')  # 转换为HWC numpy数组
        rgb_pil = Image.fromarray(rgb_img)
        rgb_pil.save(rgb_path)
        
        # 保存灰度图像(第4个通道)
        inf_img = img_data[3]  # [H, W]
        inf_img = (inf_img.numpy() * 255).astype('uint8')  # 转换为numpy数组
        inf_pil = Image.fromarray(inf_img, mode='L')  # 'L'表示灰度模式
        inf_pil.save(inf_path)

In [ ]:
#==================================================================================#
# quant
#==================================================================================#
#=======================================================================================================================
#=======================================================================================================================
# Number to Bit Function Defining
def Num2Bit(Num, B):
    Num_ = Num.type(torch.uint8)
    def integer2bit(integer, num_bits=B):
        dtype = integer.type()
        device = integer.device;
        exponent_bits = -torch.arange(1-B, 1).type(dtype).to(device);
        exponent_bits = exponent_bits.repeat(integer.shape + (1,))
        out = integer.unsqueeze(-1) // 2 ** exponent_bits
        return out% 2
    bit = integer2bit(Num_)
    bit = bit.reshape(-1, Num_.shape[1] * B)
    return bit.type(torch.float32)
def Bit2Num(Bit, B):
    device=Bit.device;
    Bit_ = Bit.type(torch.float32)
    Bit_ = torch.reshape(Bit_, [-1, int(Bit_.shape[1] / B), B])
    num = torch.zeros(Bit_[:, :, 0].shape).to(device)
    for i in range(B):
        num = num + Bit_[:, :, i] * 2 ** (B - 1 - i)
    return num
#=======================================================================================================================
#=======================================================================================================================
# Quantization and Dequantization Layers Defining
class Quantization(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, B):
        ctx.constant = B
        step = 2**B-1e-6;
        out = torch.round(x*step-0.5);
        out = Num2Bit(out, B)
        return out.type(x.type())
    @staticmethod
    def backward(ctx, grad_output):
        # return as many input gradients as there were arguments.
        # Gradients of constant arguments to forward must be None.
        # Gradient of a number is the sum of its B bits.
        b, _ = grad_output.shape
        grad_num = torch.sum(grad_output.reshape(b, -1, ctx.constant), dim=2) / ctx.constant
        return grad_num, None
class Dequantization(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, B):
        ctx.constant = B
        step = 2**B;
        out = Bit2Num(x, B)
        out = (out+0.5)/step;
        return out.type(x.type())
    @staticmethod
    def backward(ctx, grad_output):
        # return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        # repeat the gradient of a Num for B time.
        b, c = grad_output.shape
        grad_output = grad_output.unsqueeze(2) / ctx.constant
        grad_bit = grad_output.expand(b, c, ctx.constant)
        return torch.reshape(grad_bit, (-1, c * ctx.constant)), None
class QuantizationLayer(nn.Module):
    def __init__(self, B):
        super(QuantizationLayer, self).__init__()
        self.B = B
    def forward(self, x):
        out = Quantization.apply(x, self.B)
        return out
class DequantizationLayer(nn.Module):
    def __init__(self, B):
        super(DequantizationLayer, self).__init__()
        self.B = B
    def forward(self, x):
        out = Dequantization.apply(x, self.B)
        return out


#==================================================================================#
# utils
#==================================================================================#
class TransposeLayer(nn.Module):
    def __init__(
            self, 
            i, 
            j, 
        ):
        super().__init__();
        self.i=i;
        self.j=j;

    def forward(
            self, 
            x, 
        ):
        return x.transpose(self.i, self.j);



class MergeLayer(nn.Module):
    def __init__(
            self, 
            i, 
        ):
        super().__init__();
        self.i=i;

    def forward(
            self, 
            x, 
        ):
        assert self.i!=-1;
        assert self.i!=x.shape[-1];
        if self.i!=-2:
            return x.reshape(*x.shape[:self.i], -1, *x.shape[self.i+2:]);
        else:
            return x.reshape(*x.shape[:self.i], -1);



class SplitLayer(nn.Module):
    def __init__(
            self, 
            i, 
            n1=None, 
            n2=None, 
        ):
        super().__init__();
        assert (n1 is not None and n2 is None) or (n1 is None and n2 is not None);
        self.i=i;
        self.n1=n1;
        self.n2=n2;

    def forward(
            self, 
            x, 
        ):
        if self.i!=-1:
            if self.n2 is None:
                return x.reshape(*x.shape[:self.i], self.n1, -1, *x.shape[self.i+1:]);
            else:
                return x.reshape(*x.shape[:self.i], -1, self.n2, *x.shape[self.i+1:]);
        else:
            if self.n2 is None:
                return x.reshape(*x.shape[:self.i], self.n1, -1);
            else:
                return x.reshape(*x.shape[:self.i], -1, self.n2);



class PermuteLayer(nn.Module):
    def __init__(
            self, 
            permute_order, 
        ):
        super().__init__();
        self.permute_order=permute_order;
    
    def forward(
            self, 
            x, 
        ):
        return x.permute(self.permute_order);



class ReshapeLayer(nn.Module):
    def __init__(
            self, 
            shape, 
        ):
        super().__init__();
        self.shape=shape;
    
    def forward(
            self, 
            x, 
        ):
        return x.reshape(self.shape);



class NormLayer(nn.Module):
    def __init__(
            self, 
            d, 
            c_index=-1, 
        ):
        super().__init__();
        self.norm=nn.Sequential(
            TransposeLayer(c_index, -1), 
            nn.LayerNorm(d), 
            TransposeLayer(c_index, -1), 
        );
    
    def forward(
            self, 
            x, 
        ):
        return self.norm(x);



class Residual(nn.Module):
    def __init__(
            self, 
            func, 
        ):
        super().__init__();
        self.func=func;
    
    def forward(
            self, 
            x, 
        ):
        return self.func(x)+x;


#==================================================================================#
# Gated Mlp Block
#==================================================================================#
class SpatialGatingUnit(nn.Module):
    def __init__(
            self, 
            n, 
            d, 
            head, 
        ):
        super().__init__();
        dim_out=d//2;
        self.head=head;

        self.norm=nn.LayerNorm(dim_out);
        #self.proj=nn.Conv1d(n, n, 1);
        self.proj_w=nn.Parameter(torch.Tensor(head, n, n));
        self.proj_b=nn.Parameter(torch.Tensor(head, n, 1));
        
        self.reset_parameters();
    
    def reset_parameters(self):
        nn.init.normal_(self.proj_w, mean=0, std=1e-2);
        nn.init.ones_(self.proj_b);
    
    def forward(
            self, 
            x, 
        ):
        b, n, _=x.shape;

        res, gate=x.chunk(2, dim=-1);
        gate=self.norm(gate);
        res=res.reshape(b, n, self.head, -1).transpose(1, 2);
        gate=gate.reshape(b, n, self.head, -1).transpose(1, 2);
        gate=self.proj_w@gate+self.proj_b;
        x=(gate*res).transpose(1, 2).reshape(b, n, -1);
        return x;



class Channel_Attention(nn.Module):
    def __init__(self, n_embd, dim_ffn):
        super().__init__()
        self.n_embd = n_embd
        self.dim_ffn = dim_ffn

        self.key = nn.Linear(self.n_embd, self.dim_ffn)
        self.receptance = nn.Linear(self.n_embd, self.n_embd)
        self.value = nn.Linear(self.dim_ffn, self.n_embd)

        self.norm = nn.LayerNorm(n_embd)

    def forward(self, x):

        
        x_ini = x+0
        x = self.norm(x)
        k = self.key(x)
        k = torch.relu(k) ** 2
        kv = self.value(k)
        return torch.sigmoid(self.receptance(x)) * kv + x_ini



class GMlpBlock(nn.Module):
    def __init__(
            self, 
            n, 
            d, 
            d_hidden, 
            dropout=0., 
        ):
        super().__init__();
        self.proj_in=nn.Sequential(
            nn.LayerNorm(d), 
            nn.Linear(d, d_hidden), 
            nn.GELU(), 
        );
        self.sgu=SpatialGatingUnit(n, d_hidden, head=1);
        self.proj_out=nn.Sequential(
            nn.Linear(d_hidden//2, d), 
            nn.Dropout(dropout), 
        );

    def forward(
            self, 
            x, 
        ):
        shortcut=x;
        x=self.proj_in(x);
        x=self.sgu(x);
        x=self.proj_out(x);
        x=x+shortcut;
        return x;



class MAXIMBlock(nn.Module):
    def __init__(
            self, 
            n1, 
            n2, 
            d, 
            d_hidden, 
            dropout=0., 
        ):
        super().__init__();
        self.proj_in=nn.Sequential(
            nn.LayerNorm(d), 
            nn.Linear(d, 2*d), 
            nn.GELU(), 
        );
        self.gmlp1=GMlpBlock(
            n=n2, 
            d=d, 
            d_hidden=d_hidden, 
            dropout=dropout, 
        );
        self.gmlp2=GMlpBlock(
            n=n1, 
            d=d, 
            d_hidden=d_hidden, 
            dropout=dropout, 
        );
        self.proj_out=nn.Sequential(
            nn.Linear(2*d, d), 
            nn.Dropout(dropout), 
        );
        self.CA=Channel_Attention(
            n_embd=d, 
            dim_ffn=d_hidden, 
        );

    def forward(
            self, 
            x, 
        ):
        b, n1, n2, d=x.shape;

        shortcut=x;

        x=self.proj_in(x);  # b, n1, n2, 2*d
        u, v=x.chunk(2, dim=-1);  # b, n1, n2, d

        u=u.reshape(b*n1, n2, d);
        u=self.gmlp1(u);
        u=u.reshape(b, n1, n2, d);

        v=v.transpose(1, 2).reshape(b*n2, n1, d);
        v=self.gmlp2(v);
        v=v.reshape(b, n2, n1, d).transpose(1, 2);
        
        x=torch.cat((u, v), dim=-1);  # b, n1, n2, 2*d
        x=self.proj_out(x);

        x=x+shortcut;
        
        x=self.CA(x);

        return x;


## 提出的E2E非正交

### 阶段0：预训练语义提取与融合

#### 预训练1 

维度为C_dim乘30乘40，其中30作为子载波数，C_dim之后经过语义融合输出维度可以自定义到C_out，那么Q的数量为C_out/2/N_rf*40，我们可以考虑多射频传输多个流，所以要除以射频数再除以实虚部2

In [ ]:
class ConvBnLeakyRelu2d(nn.Module):
    # convolution
    # batch normalization
    # leaky relu
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1, dilation=1, groups=1):
        super(ConvBnLeakyRelu2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, dilation=dilation, groups=groups)
        self.bn   = nn.BatchNorm2d(out_channels)
    def forward(self, x):
        return F.leaky_relu(self.bn(self.conv(x)), negative_slope=0.2)


class MiniInception(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MiniInception, self).__init__()
        self.conv1_left  = ConvBnLeakyRelu2d(in_channels,   out_channels//2)
        self.conv1_right = ConvBnLeakyRelu2d(in_channels,   out_channels//2, padding=2, dilation=2)
        self.conv2_left  = ConvBnLeakyRelu2d(out_channels,  out_channels//2)
        self.conv2_right = ConvBnLeakyRelu2d(out_channels,  out_channels//2, padding=2, dilation=2)
        self.conv3_left  = ConvBnLeakyRelu2d(out_channels,  out_channels//2)
        self.conv3_right = ConvBnLeakyRelu2d(out_channels,  out_channels//2, padding=2, dilation=2)
    def forward(self,x):
        x = torch.cat((self.conv1_left(x), self.conv1_right(x)), dim=1)
        x = torch.cat((self.conv2_left(x), self.conv2_right(x)), dim=1)
        x = torch.cat((self.conv3_left(x), self.conv3_right(x)), dim=1)
        return x


class MFNet_Enc_rgb(nn.Module):

    def __init__(self, ):
        super(MFNet_Enc_rgb, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        # 压缩后的每个样本维度为[2，30，40]我们正好可以设置为单流2个实虚部组成的2、30个子载波和40个OFDM符号
        compress_ch = [192,96,96,C_dim]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.conv1_rgb   = ConvBnLeakyRelu2d(3, rgb_ch[0])
        self.conv2_1_rgb = ConvBnLeakyRelu2d(rgb_ch[0], rgb_ch[1])
        self.conv2_2_rgb = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[1])
        self.conv3_1_rgb = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[2])
        self.conv3_2_rgb = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[2])
        self.conv4_rgb   = MiniInception(rgb_ch[2], rgb_ch[3])
        self.conv5_rgb   = MiniInception(rgb_ch[3], rgb_ch[4])
        self.compress_1_rgb = ConvBnLeakyRelu2d(compress_ch[0], compress_ch[1])
        self.compress_2_rgb = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[2])
        self.compress_3_rgb = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[3])
    def forward(self, x):
        # split data into RGB and INF
        x_rgb = x[:,:3]

        # encode
        x_rgb    = self.conv1_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb, kernel_size=2, stride=2) # pool1
        x_rgb    = self.conv2_1_rgb(x_rgb)
        x_rgb_p2 = self.conv2_2_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p2, kernel_size=2, stride=2) # pool2
        x_rgb    = self.conv3_1_rgb(x_rgb)
        x_rgb_p3 = self.conv3_2_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p3, kernel_size=2, stride=2) # pool3
        x_rgb_p4 = self.conv4_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p4, kernel_size=2, stride=2) # pool4 #[batch,2,30,40]
        x_rgb    = self.conv5_rgb(x_rgb)  #[batch,96,30,40]
        x_rgb    = F.max_pool2d(x_rgb, kernel_size=(3,4), stride=(3,4)) # pool5  

        x_rgb    = self.compress_1_rgb(x_rgb)  
        x_rgb    = self.compress_2_rgb(x_rgb)
        x_rgb    = self.compress_3_rgb(x_rgb) #[batch,C_dim,10,10]
        return x_rgb


class MFNet_Enc_inf(nn.Module):

    def __init__(self, ):
        super(MFNet_Enc_inf, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        # 压缩后的每个样本维度为[2，30，40]我们正好可以设置为单流2个实虚部组成的2、30个子载波和40个OFDM符号
        compress_ch = [192,96,96,C_dim]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.conv1_inf   = ConvBnLeakyRelu2d(1, rgb_ch[0])
        self.conv2_1_inf = ConvBnLeakyRelu2d(rgb_ch[0], rgb_ch[1])
        self.conv2_2_inf = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[1])
        self.conv3_1_inf = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[2])
        self.conv3_2_inf = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[2])
        self.conv4_inf   = MiniInception(rgb_ch[2], rgb_ch[3])
        self.conv5_inf   = MiniInception(rgb_ch[3], rgb_ch[4])
        self.compress_1_inf = ConvBnLeakyRelu2d(compress_ch[0], compress_ch[1])
        self.compress_2_inf = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[2])
        self.compress_3_inf = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[3])
    def forward(self, x):
        # split data into RGB and INF
        x_inf = x[:,3:]

        # encode
        x_inf    = self.conv1_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf, kernel_size=2, stride=2) # pool1
        x_inf    = self.conv2_1_inf(x_inf)
        x_inf_p2 = self.conv2_2_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p2, kernel_size=2, stride=2) # pool2
        x_inf    = self.conv3_1_inf(x_inf)
        x_inf_p3 = self.conv3_2_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p3, kernel_size=2, stride=2) # pool3
        x_inf_p4 = self.conv4_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p4, kernel_size=2, stride=2) # pool4 #[batch,96,30,40]
        x_inf    = self.conv5_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf, kernel_size=(3,4), stride=(3,4)) # pool5  

        x_inf    = self.compress_1_inf(x_inf)  
        x_inf    = self.compress_2_inf(x_inf)  
        x_inf    = self.compress_3_inf(x_inf)    #[batch,C_dim,10,10]
        return x_inf

class MFNet_Dec(nn.Module):

    def __init__(self, n_class):
        super(MFNet_Dec, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        # 压缩后的每个样本维度为 #[C_dim,10,10]我们keyi
        compress_ch = [192,96,96,C_dim]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.decompress_3 = ConvBnLeakyRelu2d(compress_ch[3], compress_ch[2])
        self.decompress_2 = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[1])
        self.decompress_1 = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[0])


        self.decode5     = ConvBnLeakyRelu2d(rgb_ch[4], rgb_ch[3])
        self.decode4     = ConvBnLeakyRelu2d(rgb_ch[3], rgb_ch[2])
        self.decode3     = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[1])
        self.decode2     = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[0])
        self.decode1     = ConvBnLeakyRelu2d(rgb_ch[0], n_class)
    def forward(self, x):
        #[batch,8,30,40]
        # x = self.decompress_4(x)
        # x = x.reshape(-1,8,self.pho,15,20).permute(0,2,1,3,4)
        # x = x.reshape(-1,self.pho*8,15,20)
        # x = self.decompress_4(x.permute(0,1,3,2)).permute(0,1,3,2)
        x = self.decompress_3(x)
        x = self.decompress_2(x)
        x = self.decompress_1(x) #[batch,96,30,40]

        # decode
        x = F.interpolate(x, scale_factor=(3,4), mode='nearest')# unpool5
        x = self.decode5(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool4
        x = self.decode4(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool3
        x = self.decode3(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool2
        x = self.decode2(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool1
        x = self.decode1(x)

        return x

class MFNet_NOMA(nn.Module):

    def __init__(self, n_class):
        super(MFNet_NOMA, self).__init__()
        self.enc_rgb = MFNet_Enc_rgb()
        self.enc_inf = MFNet_Enc_inf()
        self.dec = MFNet_Dec(n_class)
        
        

    def forward(self, x):
        x_rgb = self.enc_rgb(x)
        x_inf = self.enc_inf(x)
        x = x_rgb + x_inf 
        x = self.dec(x)
        return x
        # x_rgb = self.enc_rgb(x) 
        # x_rgb = x_rgb[:,0] + 1j*x_rgb[:,1]
        # x_rgb = x_rgb * (torch.randn(x.shape[0],1,40) + 1j*torch.randn(x.shape[0],1,40)).cuda()
        # x_inf = self.enc_inf(x)
        # x_inf = x_inf[:,0] + 1j*x_inf[:,1]
        # x_inf = x_inf * (torch.randn(x.shape[0],1,40) + 1j*torch.randn(x.shape[0],1,40)).cuda()
        # x = x_rgb + x_inf 
        # x = x.reshape(-1,1,30,40)
        # x = torch.cat([x.real,x.imag],dim=1)
        # x = self.dec(x)
        # return x

In [ ]:
# coding:utf-8
import os
import argparse
import time

import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

from util.MF_dataset import MF_dataset
from util.util import calculate_accuracy, calculate_result
from util.augmentation import RandomFlip, RandomCrop, RandomCropOut, RandomBrightness, RandomNoise
from model import MFNet, SegNet

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
# config
n_class   = 9
data_dir  = './ir_seg_dataset/'
model_dir = 'weights/'
augmentation_methods = [
    RandomFlip(prob=0.5),
    RandomCrop(crop_rate=0.1, prob=1.0), 
    # RandomCropOut(crop_rate=0.2, prob=1.0),
    # RandomBrightness(bright_range=0.15, prob=0.9),
    # RandomNoise(noise_range=5, prob=0.9),
]
lr_start  = 0.01
lr_decay  = 0.95


def train(epo, model, train_loader, optimizer):

    lr_this_epo = lr_start * lr_decay**(epo-1)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr_this_epo

    loss_avg = 0.
    acc_avg  = 0.
    start_t = t = time.time()
    model.train()

    for it, (images, labels, names) in enumerate(train_loader):
        images = Variable(images).cuda(args.gpu) 
        labels = Variable(labels).cuda(args.gpu)
        # if args.gpu >= 0:
        #     images = images.cuda(args.gpu)
        #     labels = labels.cuda(args.gpu)

        optimizer.zero_grad()
        logits = model(images)
        # print(labels.min(), labels.max())

        loss = F.cross_entropy(logits, labels)
        loss.backward()
        optimizer.step()

        acc = calculate_accuracy(logits, labels)
        loss_avg += float(loss.item())
        acc_avg  += float(acc)

        # cur_t = time.time()
        # if cur_t-t > 5:
        #     print('|- epo %s/%s. train iter %s/%s. %.2f img/sec loss: %.4f, acc: %.4f' \
        #         % (epo, args.epoch_max, it+1, train_loader.n_iter, (it+1)*args.batch_size/(cur_t-start_t), float(loss), float(acc)))
        #     t += 5

    content = '| epo:%s/%s lr:%.4f train_loss_avg:%.4f train_acc_avg:%.4f ' \
            % (epo, args.epoch_max, lr_this_epo, loss_avg/train_loader.n_iter, acc_avg/train_loader.n_iter)
    print(content)
    with open(log_file, 'a') as appender:
        appender.write(content)


def validation(epo, model, val_loader):

    loss_avg = 0.
    acc_avg  = 0.
    start_t = time.time()
    model.eval()

    cf = np.zeros((n_class, n_class))

    with torch.no_grad():
        for it, (images, labels, names) in enumerate(val_loader):
            images = Variable(images)
            labels = Variable(labels)
            if args.gpu >= 0:
                images = images.cuda(args.gpu)
                labels = labels.cuda(args.gpu)

            logits = model(images)
            loss = F.cross_entropy(logits, labels)
            acc = calculate_accuracy(logits, labels)
            loss_avg += float(loss)
            acc_avg  += float(acc)

            # cur_t = time.time()
            # print('|- epo %s/%s. val iter %s/%s. %.2f img/sec loss: %.4f, acc: %.4f' \
            #         % (epo, args.epoch_max, it+1, val_loader.n_iter, (it+1)*args.batch_size/(cur_t-start_t), float(loss), float(acc)))

            predictions = logits.argmax(1)
            for gtcid in range(n_class): 
                for pcid in range(n_class):
                    gt_mask      = labels == gtcid 
                    pred_mask    = predictions == pcid
                    intersection = gt_mask * pred_mask
                    cf[gtcid, pcid] += int(intersection.sum())
        overall_acc, acc, IoU = calculate_result(cf)

    content = '| val_loss_avg:%.4f val_acc_avg:%.4f' \
            % (loss_avg/val_loader.n_iter, acc_avg/val_loader.n_iter)
    print(content)
    content = '| class accuracy avg:%.4f class IoU avg:%.4f\n' \
                % (acc.mean(), IoU.mean())
    print(content)
    
    # with open(log_file, 'a') as appender:
    #     appender.write(content)


import os
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

from ipdb import set_trace as st
class MF_dataset(Dataset):

    def __init__(self, data_dir, split, have_label, input_h=480, input_w=640 ,transform=[]):
        super(MF_dataset, self).__init__()

        # assert split in ['train', 'val', 'test'], 'split must be "train"|"val"|"test"'

        with open(os.path.join(data_dir, split+'.txt'), 'r') as f:
            self.names = [name.strip() for name in f.readlines()]

        self.data_dir  = data_dir
        self.split     = split
        self.input_h   = input_h
        self.input_w   = input_w
        self.transform = transform
        self.is_train  = have_label
        self.n_data    = len(self.names)


    def read_image(self, name, folder):
        file_path = os.path.join(self.data_dir, '%s/%s.png' % (folder, name))
        image     = np.asarray(Image.open(file_path)) # (w,h,c)
        image = np.require(image, dtype='f4', requirements=['O', 'W'])
        image.flags.writeable = True
        return image

    def get_train_item(self, index):
        name  = self.names[index]
        image = self.read_image(name, 'images')
        label = self.read_image(name, 'labels')

        for func in self.transform:
            image, label = func(image, label)

        image = np.asarray(Image.fromarray(np.uint8(image)).resize((self.input_w, self.input_h)), dtype=np.float32).transpose((2,0,1))/255
        # label = np.asarray(Image.fromarray(label).resize((self.input_w, self.input_h)), dtype=np.int64)
        label = np.asarray(Image.fromarray(label).resize((self.input_w, self.input_h), Image.NEAREST), dtype=np.int64)

        return torch.tensor(image), torch.tensor(label), name

    def get_test_item(self, index):
        name  = self.names[index]
        image = self.read_image(name, 'images')
        image = np.asarray(Image.fromarray(image).resize((self.input_w, self.input_h)), dtype=np.float32).transpose((2,0,1))/255

        return torch.tensor(image), name


    def __getitem__(self, index):

        if self.is_train is True:
            return self.get_train_item(index)
        else: 
            return self.get_test_item (index)

    def __len__(self):
        return self.n_data

##### 训练

In [ ]:
import sys
sys.argv = ['run.py']
parser = argparse.ArgumentParser(description='Train MFNet with pytorch')
parser.add_argument('--model_name',  '-M',  type=str, default='MFNet_NOMA')
parser.add_argument('--batch_size',  '-B',  type=int, default=8)

parser.add_argument('--epoch_max' ,  '-E',  type=int, default=100)
# parser.add_argument('--epoch_max' ,  '-E',  type=int, default=2)

parser.add_argument('--epoch_from',  '-EF', type=int, default=1)
parser.add_argument('--gpu',         '-G',  type=int, default=0)
parser.add_argument('--num_workers', '-j',  type=int, default=0)
args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)
os.makedirs(model_dir, exist_ok=True)
checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1.pth')
checkpoint_optim_file = os.path.join(model_dir, 'tmp_pretrain1.optim')
final_model_file      = os.path.join(model_dir, 'final_pretrain1_C4.pth')
log_file              = os.path.join(model_dir, 'log_pretrain1.txt')

print('| training %s on GPU #%d with pytorch' % (args.model_name, args.gpu))
print('| from epoch %d / %s' % (args.epoch_from, args.epoch_max))
print('| model will be saved in: %s' % model_dir)


In [ ]:
model = eval(args.model_name)(n_class=n_class)
if args.gpu >= 0: model.cuda(args.gpu)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_start, momentum=0.9, weight_decay=0.0005) 
# optimizer = torch.optim.Adam(model.parameters(), lr=lr_start)

if args.epoch_from > 1:
    print('| loading checkpoint file %s... ' % checkpoint_model_file, end='')
    model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
    optimizer.load_state_dict(torch.load(checkpoint_optim_file))
    print('done!')

train_dataset = MF_dataset(data_dir, 'train', have_label=True, transform=augmentation_methods)
val_dataset  = MF_dataset(data_dir, 'val', have_label=True)

train_loader  = DataLoader(
    dataset     = train_dataset,
    batch_size  = args.batch_size,
    shuffle     = True,
    num_workers = args.num_workers,
    pin_memory  = True,
    drop_last   = True
)
val_loader  = DataLoader(
    dataset     = val_dataset,
    batch_size  = args.batch_size,
    shuffle     = False,
    num_workers = args.num_workers,
    pin_memory  = True,
    drop_last   = False
)
train_loader.n_iter = len(train_loader)
val_loader.n_iter   = len(val_loader)

for epo in (range(args.epoch_from, args.epoch_max+1)):
    print('\n| epo #%s begin...' % epo)

    train(epo, model, train_loader, optimizer)
    validation(epo, model, val_loader)

    # save check point model
    # print('| saving check point model file... ', end='')
    torch.save(model.state_dict(), checkpoint_model_file)
    torch.save(optimizer.state_dict(), checkpoint_optim_file)
    # print('done!')
if os.path.exists(final_model_file):
    os.remove(final_model_file) # 删除已存在的文件
os.rename(checkpoint_model_file, final_model_file)

##### 测试

###### demo

In [ ]:
from util.util import visualize
import sys

sys.argv = ['run.py']


parser = argparse.ArgumentParser(description='Run MFNet demo with pytorch')
parser.add_argument('--model_name', '-M',  type=str, default='MFNet_NOMA')
parser.add_argument('--gpu',        '-G',  type=int, default=0)
args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)

checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1.pth')
final_model_file      = os.path.join(model_dir, 'final_pretrain1_C4.pth')
print('| running %s demo on GPU #%d with pytorch' % (args.model_name, args.gpu))




model = eval(args.model_name)(n_class=n_class)
if args.gpu >= 0: model.cuda(args.gpu)
if os.path.exists(final_model_file):
    model.load_state_dict(torch.load(final_model_file, map_location='cuda:0'))
elif os.path.exists(checkpoint_model_file):
    model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
else:
    raise Exception('| model file do not exists in %s' % model_dir)
print('| model loaded!')


files = os.listdir('image')
images = []
fpath  = []
fpath_out = []
fpath_original = []
for file in files:
    if file[-3:] != 'png': continue
    fpath.append('image/'+file)
    fpath_out.append('seg_MFNet/'+file)
    fpath_original.append('image_original/'+file)
    images.append( np.asarray(Image.open('image/'+file)) )
images = np.asarray(images, dtype=np.float32).transpose((0,3,1,2))/255.
images = Variable(torch.tensor(images))
if args.gpu >= 0: images = images.cuda(args.gpu)

model.eval()
with torch.no_grad():
    logits = model(images)
    predictions = logits.argmax(1)
    save_images(images, fpath_original)  #保存原始图像
    visualize(fpath_out, predictions)

###### 性能指标

In [ ]:
from util.util import calculate_accuracy, calculate_result
import sys

sys.argv = ['run.py']


cf = np.zeros((n_class, n_class))


parser = argparse.ArgumentParser(description='Test MFNet with pytorch')
parser.add_argument('--model_name', '-M',  type=str, default='MFNet_NOMA')
parser.add_argument('--batch_size',  '-B',  type=int, default=16)
parser.add_argument('--gpu',        '-G',  type=int, default=0)
parser.add_argument('--num_workers', '-j',  type=int, default=0)



args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)

checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1.pth')
final_model_file      = os.path.join(model_dir, 'final_pretrain1_C4.pth')
print('| testing %s on GPU #%d with pytorch' % (args.model_name, args.gpu))

model = eval(args.model_name)(n_class=n_class)
if args.gpu >= 0: model.cuda(args.gpu)
if os.path.exists(final_model_file):
    model.load_state_dict(torch.load(final_model_file, map_location='cuda:0'))
elif os.path.exists(checkpoint_model_file):
    model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
else:
    raise Exception('| model file do not exists in %s' % model_dir)
print('| model loaded!')


test_dataset  = MF_dataset(data_dir, 'test', have_label=True)
test_loader  = DataLoader(
    dataset     = test_dataset,
    batch_size  = args.batch_size,
    shuffle     = False,
    num_workers = args.num_workers,
    pin_memory  = True,
    drop_last   = False
)
test_loader.n_iter = len(test_loader)

loss_avg = 0.
acc_avg  = 0.
model.eval()
with torch.no_grad():
    for it, (images, labels, names) in enumerate(test_loader):
        images = Variable(images)
        labels = Variable(labels)
        if args.gpu >= 0:
            images = images.cuda(args.gpu)
            labels = labels.cuda(args.gpu)

        logits = model(images)
        loss = F.cross_entropy(logits, labels)
        acc = calculate_accuracy(logits, labels)
        loss_avg += float(loss)
        acc_avg  += float(acc)

        print('|- test iter %s/%s. loss: %.4f, acc: %.4f' \
                % (it+1, test_loader.n_iter, float(loss), float(acc)))

        predictions = logits.argmax(1)
        for gtcid in range(n_class): 
            for pcid in range(n_class):
                gt_mask      = labels == gtcid 
                pred_mask    = predictions == pcid
                intersection = gt_mask * pred_mask
                cf[gtcid, pcid] += int(intersection.sum())

overall_acc, acc, IoU = calculate_result(cf)

print('| overall accuracy:', overall_acc)
print('| accuracy of each class:', acc)
print('| class accuracy avg:', acc.mean())
print('| IoU:', IoU)
print('| class IoU avg:', IoU.mean())



### 阶段1：预训练上行导频发送、模拟合并器和上行预编码

#### 网络定义

In [ ]:
from tqdm import tqdm
from tqdm import trange
from time import sleep
from my_log import logger, rmLogger
# 三个可学习的导频，分别是基站端两个发送导频 用户端模拟合并
# Transformer1：用户端输入接收导频输出输出混合预编码矩阵 （两个用户用不同的参数）
# Transformer2: 用户输入接收导频输出反馈比特 （两个用户用不同的参数）
# Transformer3：基站输入反馈比特，输出模拟合并矩阵
# 拆分成4个网络，三个Transformer分成3个类，最后一个总的里面也包含三个导频
# 考虑信道时变，那么损失函数用导频之后的第L+1信道的频谱效率


d_sem = 192 #语义特征维度

# 可训练的三个导频
class Pilot_trans(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
        ):
        super().__init__();

        self.N_r = N_r
        self.L = L
        self.N_t = N_t
        self.N_t_RF = N_t_RF

        # self.p_UE_1 = nn.Parameter(torch.randn(1, Nc, L, N_t, 1) + 1j*torch.randn(1, Nc, L, N_t, 1))
        # self.p_UE_2 = nn.Parameter(torch.randn(1, Nc, L, N_t, 1) + 1j*torch.randn(1, Nc, L, N_t, 1))

        # self.p_UE_1 = nn.Parameter(torch.randn(1, 1, L, N_t, 1) + 1j*torch.randn(1, 1, L, N_t, 1))
        # self.p_UE_2 = nn.Parameter(torch.randn(1, 1, L, N_t, 1) + 1j*torch.randn(1, 1, L, N_t, 1))

        self.p_BS_RF = nn.Parameter(torch.randn(1, 1, L, N_r, N_r_RF) + 1j*torch.randn(1, 1, L, N_r, N_r_RF))
        self.p_BS_BB = nn.Parameter(torch.randn(1, Nc, L, N_r_RF, 1) + 1j*torch.randn(1, Nc, L, N_r_RF, 1))

        self.p_UE = nn.Parameter(torch.randn(1, 1, L, N_t_RF, N_t) + 1j*torch.randn(1, 1, L, N_t_RF, N_t))

    def forward(
            self, 
            H_batch_1,  # b,T, Nc, N_r, N_t
            H_batch_2,  # b,T, Nc, N_r, N_t
        ):


        b,T, Nc, N_r, N_t = H_batch_1.shape
        N_t_RF = self.N_t_RF
        L = self.L
        H_batch_1 = H_batch_1[:,0:L].permute(0,2,1,4,3) # b, Nc, L, N_t, N_r
        H_batch_2 = H_batch_2[:,0:L].permute(0,2,1,4,3) # b, Nc, L, N_t, N_r


        p_UE = self.p_UE/torch.abs(self.p_UE)/sqrt(self.N_t)
        p_BS = self.p_BS_RF @ self.p_BS_BB





        received_pilot_1 = p_UE @ (H_batch_1 @ p_BS) # b, Nc, L, N_t_RF, 1
        received_pilot_2 = p_UE @ (H_batch_2 @ p_BS) # b, Nc, L, N_t_RF, 1


        norm_factor = torch.sqrt(torch.sum(torch.abs(received_pilot_1)**2, dim=(-4,-3,-2, -1), keepdim=True))
        received_pilot_1 = received_pilot_1 / norm_factor * torch.sqrt(torch.tensor(Nc*L*N_t_RF, dtype=received_pilot_1.dtype, device=received_pilot_1.device))

        norm_factor = torch.sqrt(torch.sum(torch.abs(received_pilot_2)**2, dim=(-4,-3,-2, -1), keepdim=True))
        received_pilot_2 = received_pilot_2 / norm_factor * torch.sqrt(torch.tensor(Nc*L*N_t_RF, dtype=received_pilot_2.dtype, device=received_pilot_2.device))


        return received_pilot_1,received_pilot_2   


class pilot2sem(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
        ):
        super().__init__();

        self.d_model = 192
        self.B = 4  #量化精度

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)

        self.embed= nn.Linear(2*N_t_RF*L, self.d_model) 
        self.positional_embedding=nn.Parameter(torch.zeros(1, Nc, self.d_model));

        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(2)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model

        self.head=nn.Sequential(  
            nn.Linear(self.d_model, d_sem),  # b, Nc, d_sem
        );

    



    
    def forward(
            self, 
            received_pilot,  # b, Nc, L, N_t_RF, 1
        ):
        # b = H_DL_batch.shape[0]
        # H_DL_batch_1 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,0,:].conj() #这里加了conj是因为上下行信道之间有个共轭转置，所以共轭这部分需要conj回去消除掉
        # H_DL_batch_2 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,1,:].conj()

        # H_DL_batch_beam_1 = (H_DL_batch_1*np.exp(1j*pi/2 * 0) + H_DL_batch_2*np.exp(1j*pi/2 * 0)).reshape(b,1,48,4,256)

        b, Nc, L, N_t_RF, _ = received_pilot.shape


        y_cat = received_pilot.reshape(b,Nc,L*N_t_RF) 
        y_cat=torch.cat([y_cat.real, y_cat.imag], dim=-1);  # b,Nc,2*L*N_t_RF

        x = self.embed(y_cat) + self.positional_embedding  # b,Nc,d_model

        x = self.backbone(x)   # b,Nc,d_model
        CSI_sem_UE = self.head(x)   #b, Nc, d_sem
        return CSI_sem_UE     #b, Nc, d_sem

class sem2precoding(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
        ):
        super().__init__();

        self.d_model = 192
        self.N_t = N_t
        self.N_t_RF = N_t_RF
        self.Nc = Nc
        self.N_r = N_r
        self.N_r_RF = N_r_RF


        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)

        self.embed=nn.Sequential(  
            nn.Linear(d_sem, self.d_model),  
            NormLayer(self.d_model), 
        );
        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(2)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model

        self.head_RF=nn.Sequential(  
            ReshapeLayer([-1, Nc*self.d_model]),  
            nn.Linear(Nc*self.d_model, 2*N_t_RF*N_t),  
        );
        # self.head_BB=nn.Sequential(  
        #     nn.Linear(self.d_model, 2*N_t_RF*N_t_RF),  
        # );
    



    
    def forward(
            self, 
            CSI_sem_UE,  # b, Nc, d_sem
        ):

        N_t = self.N_t
        N_t_RF = self.N_t_RF
        Nc = self.Nc
        N_r = self.N_r
        N_r_RF = self.N_r_RF
        b = CSI_sem_UE.shape[0]



        x = self.embed(CSI_sem_UE)

        x = self.backbone(x)   # b,Nc,d_model

        F = self.head_RF(x)
        F = F.reshape(b,1,N_t,N_t_RF,2) # b,1,N_t,N_t_RF,2
        F = F[:,:,:,:,0] + 1j*F[:,:,:,:,1]
        F = F/torch.abs(F)/sqrt(N_t) #[batch, 1,N_t,N_t_RF]




        return F     #[batch, 1,N_t,N_t_RF]

class sem2bits(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
            feedback_bits,  #反馈比特数
        ):
        super().__init__();

        self.d_model = 192
        self.B = 4  #量化精度

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)

        self.N_t = N_t
        self.N_t_RF = N_t_RF
        self.Nc = Nc
        self.N_r = N_r
        self.N_r_RF = N_r_RF

        self.embed=nn.Sequential(  
            nn.Linear(d_sem, self.d_model),  
            NormLayer(self.d_model), 
        );
        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(2)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model


        self.quan=nn.Sequential(  
            ReshapeLayer([-1, Nc*self.d_model]),
            nn.Linear(Nc*self.d_model, feedback_bits//self.B),  # b, feedback_bits//self.B
            nn.BatchNorm1d(feedback_bits//self.B), 
            nn.Sigmoid(), 
            QuantizationLayer(self.B),  # b, feedback_bits
        );

    



    
    def forward(
            self, 
            CSI_sem_UE,  # b, Nc, d_sem
        ):

        N_t = self.N_t
        N_t_RF = self.N_t_RF
        Nc = self.Nc
        N_r = self.N_r
        N_r_RF = self.N_r_RF
        b = CSI_sem_UE.shape[0]

        x = self.embed(CSI_sem_UE)
        x = self.backbone(x)
        out = self.quan(x)   #[batch, Bits]
        return out     #[batch, Bits]


class bits2sem(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
            feedback_bits,  #反馈比特数
        ):
        super().__init__();

        self.d_model = 192
        self.B = 4  #量化精度
        self.feedback_bits = feedback_bits

        self.Nc = Nc
        self.N_r = N_r
        self.N_r_RF = N_r_RF

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)
        self.Dequan=nn.Sequential(  
            DequantizationLayer(self.B),  
        );

        self.embed=nn.Sequential(  
            nn.Linear(2*feedback_bits//self.B, Nc*self.d_model),  
            ReshapeLayer([-1, Nc,self.d_model]),  
            NormLayer(self.d_model), 
        );

        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(2)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model

        # self.head=nn.Sequential(  
        #     nn.Linear(self.d_model, N_t*2),  
        # );
        self.head=nn.Sequential(  
            nn.Linear(self.d_model, d_sem),  
        );
    
    

    
    
    def forward(
            self, 
            x1,  # b, feedback_bits   #反馈比特
            x2,  # b, feedback_bits   #反馈比特
        ):
        # b = H_DL_batch.shape[0]
        # H_DL_batch_1 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,0,:].conj() #这里加了conj是因为上下行信道之间有个共轭转置，所以共轭这部分需要conj回去消除掉
        # H_DL_batch_2 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,1,:].conj()

        # H_DL_batch_beam_1 = (H_DL_batch_1*np.exp(1j*pi/2 * 0) + H_DL_batch_2*np.exp(1j*pi/2 * 0)).reshape(b,1,48,4,256)

        b = x1.shape[0]

        x1 = self.Dequan(x1) # b,feedback_bits//self.B
        x2 = self.Dequan(x2) # b,feedback_bits//self.B
        x = torch.cat([x1,x2],dim=-1) # b,2*feedback_bits//self.B

        out = self.embed(x) # b,Nc,d_model
        out = self.backbone(out)# b,Nc,d_model
        CSI_sem_BS = self.head(out)   # b,Nc, d_sem



        return CSI_sem_BS     #b,Nc, d_sem

class sem2combiner(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
        ):
        super().__init__();

        self.d_model = 192
        self.B = 4  #量化精度

        self.Nc = Nc
        self.N_r = N_r
        self.N_r_RF = N_r_RF

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)


        self.embed=nn.Sequential(  
            nn.Linear(d_sem, self.d_model),  
            NormLayer(self.d_model), 
        );

        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(2)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model

        # self.head=nn.Sequential(  
        #     nn.Linear(self.d_model, N_t*2),  
        # );
        self.head=nn.Sequential(  
            ReshapeLayer([-1, Nc*self.d_model]),  
            nn.Linear(Nc*self.d_model, 2*N_r*N_r_RF),  
        );

    
    
    def forward(
            self, 
            CSI_sem_BS
        ):
        # b = H_DL_batch.shape[0]
        # H_DL_batch_1 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,0,:].conj() #这里加了conj是因为上下行信道之间有个共轭转置，所以共轭这部分需要conj回去消除掉
        # H_DL_batch_2 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,1,:].conj()

        # H_DL_batch_beam_1 = (H_DL_batch_1*np.exp(1j*pi/2 * 0) + H_DL_batch_2*np.exp(1j*pi/2 * 0)).reshape(b,1,48,4,256)

        b = CSI_sem_BS.shape[0]

        

        out = self.embed(CSI_sem_BS) # b,Nc,d_model
        out = self.backbone(out)# b,Nc,d_model
        out = self.head(out)   # b,N_t*2
        # F = out.reshape(b,1,self.N_t,N_t_RF,2) # b,1,N_t,N_t_RF,2
        # F = F[:,:,:,:,0] + 1j*F[:,:,:,:,1]
        # F = F/torch.abs(F)/sqrt(self.N_t) #[batch, 1,N_t,N_t_RF]

        W = out.reshape(b,1,self.N_r_RF,self.N_r,2)
        W = W[:,:,:,:,0] + 1j*W[:,:,:,:,1]
        W = W/torch.abs(W)/sqrt(self.N_r)  #b,1,self.N_r_RF,self.N_r


        return W     #[b,1,N_r_RF,N_r]

class physical_E2E(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
            feedback_bits,  #反馈比特数
        ):
        super().__init__();

        self.pilot2sem_1 = pilot2sem(Nc,N_r,N_r_RF,N_t,N_t_RF,L)
        self.sem2precoding_1 = sem2precoding(Nc,N_r,N_r_RF,N_t,N_t_RF,L)
        self.sem2bits_1 = sem2bits(Nc,N_r,N_r_RF,N_t,N_t_RF,L,feedback_bits)

        self.pilot2sem_2 = pilot2sem(Nc,N_r,N_r_RF,N_t,N_t_RF,L)
        self.sem2precoding_2 = sem2precoding(Nc,N_r,N_r_RF,N_t,N_t_RF,L)
        self.sem2bits_2 = sem2bits(Nc,N_r,N_r_RF,N_t,N_t_RF,L,feedback_bits)


        self.bits2sem   = bits2sem(Nc,N_r,N_r_RF,N_t,N_t_RF,L,feedback_bits)
        self.sem2combiner   = sem2combiner(Nc,N_r,N_r_RF,N_t,N_t_RF,L)
        self.Pilot_trans = Pilot_trans(Nc,N_r,N_r_RF,N_t,N_t_RF,L)

       



    
    def forward(
            self, 
            H_batch_1,  # b,T, Nc, N_r, N_t 用户1
            H_batch_2,  # b,T, Nc, N_r, N_t 用户2
        ):
        received_pilot_1,received_pilot_2 = self.Pilot_trans(H_batch_1,H_batch_2)
        CSI_sem_UE_1 = self.pilot2sem_1(received_pilot_1)
        CSI_sem_UE_2 = self.pilot2sem_2(received_pilot_2)

        F_1 = self.sem2precoding_1(CSI_sem_UE_1)
        F_2 = self.sem2precoding_2(CSI_sem_UE_2)


        bits_1 = self.sem2bits_1(CSI_sem_UE_1)
        bits_2 = self.sem2bits_2(CSI_sem_UE_2)

        CSI_sem_BS = self.bits2sem(bits_1,bits_2)


        W = self.sem2combiner(CSI_sem_BS)
        return W,F_1,F_2,CSI_sem_UE_1,CSI_sem_UE_2,CSI_sem_BS
    
    


#### 训练函数

In [ ]:
def train_physical_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre = None,v_max_km_pre=None,ITERS=20_000,b=32,acc_num=1):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;

    model_name = 'physical_E2E' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)

    dec=physical_E2E(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L,feedback_bits=feedback_bits).cuda()
    
    opt_dec=torch.optim.AdamW(
        dec.parameters(), 
        lr=lr, 
    )
    T = L+1
    load_path=save_path+"physical_E2E"+"/";
    if feedback_bits_pre == None:
        print('从头训练')
    else:
        dec_dict = dec.state_dict()
        model_name_pre = 'physical_E2E' + str(feedback_bits_pre)+'_L' + str(L)+'_v' + str(v_max_km_pre)
        dec_load_path=load_path+model_name_pre+"_model.pth.tar";
        ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
        pretrained_dict=ckpt['state_dict'];
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict and ".quan" not in k and ".Dequan" not in k}
        dec_dict.update(pretrained_dict);
        dec.load_state_dict(dec_dict);
        print('全部预训练参数读取成功')

    def step(step_id):
        dec.train();
        opt_dec.zero_grad();
        loss_acc=0;

        


        

        for _ in range(acc_num):
            alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
            H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

            for i in range(T):
                t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                H_batch[:,i] = H
            H_batch_1 = H_batch[0:b]
            H_batch_2 = H_batch[b:2*b]

            

            
            W,F_1,F_2,CSI_sem_UE_1,CSI_sem_UE_2,CSI_sem_BS=dec(
                    H_batch_1 = H_batch_1,  # b,T, Nc, N_r, N_t 用户1
                    H_batch_2 = H_batch_2,  # b,T, Nc, N_r, N_t 用户2
                );



            # loss_SE = -calculate_average_channel_capacity(H_DL_batch_beam[:,0:1], F, Pt,N0)/acc_num

            loss = -torch.min(calculate_average_channel_capacity(H_batch_1[:,L], F_1, W, SNR_dB),calculate_average_channel_capacity(H_batch_2[:,L], F_2, W, SNR_dB))/acc_num
            loss = loss.mean()
            loss_acc += loss.item()

            loss.backward();
        

        opt_dec.step();

        return loss_acc






    def evaluate(step_id):
        global loader_test;
        dec.eval();
        loss_acc=0;
        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]


                
                W,F_1,F_2,CSI_sem_UE_1,CSI_sem_UE_2,CSI_sem_BS=dec(
                        H_batch_1 = H_batch_1,  # b,T, Nc, N_r, N_t 用户1
                        H_batch_2 = H_batch_2,  # b,T, Nc, N_r, N_t 用户2
                    );
                loss = -torch.min(calculate_average_channel_capacity(H_batch_1[:,L], F_1, W, SNR_dB),calculate_average_channel_capacity(H_batch_2[:,L], F_2, W, SNR_dB))/val_num
                loss = loss.mean()
                loss_acc += loss.item()

        return loss_acc

    def save_ckpt(folder_name):
        folder_path=save_path+folder_name+"/";
        if not os.path.exists(folder_path):
            os.makedirs(folder_path);

        torch.save({'state_dict': dec.state_dict()}, folder_path+model_name+"_model.pth.tar");
        torch.save({'best_loss': best_loss, 
                    'opt_dec_dict': opt_dec.state_dict(), 
                    }, folder_path+model_name+"_opt.pth.tar");

    loss_SE_log=0;
    t1=time.time();




    with trange(iter_begin, iter_begin+ITERS) as t:
        for i in t:            
            loss_SE = step(i);
            loss_SE_log+=loss_SE/log_interval;

            if (i+1)%log_interval==0:
                loss_SE = evaluate(i);
                
                logger("");
                if loss_SE<best_loss:
                    best_loss=loss_SE;
                    save_ckpt("physical_E2E");
                    logger("\nbest - SE = {}".format(-best_loss));
                
                logger("log - iter[{}] - train - SE = {}".format(i, -loss_SE_log));
                logger("log - iter[{}] - test - SE = {}".format(i, -loss_SE));

                

                logger("time: {} s".format(time.time()-t1));
                t1=time.time();

                if (i+1)%save_interval==0:
                    logger("");
                    logger("save - iter[{}] - test - SE = {}".format(i, -loss_SE));
                    logger("save - iter[{}] - best - SE = {}".format(i, -best_loss));

                    

                t.set_postfix(train_SE=-loss_SE_log,test_SE=-loss_SE,best_SE=-best_loss)

                loss_SE_log=0;
    rmLogger();

In [ ]:
def test_physical_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;

    model_name = 'physical_E2E' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)

    dec=physical_E2E(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L,feedback_bits=feedback_bits).cuda()
    
   
    T = L+1
    load_path=save_path+"physical_E2E"+"/";
   
    dec_dict = dec.state_dict()
    dec_load_path=load_path+model_name+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec.load_state_dict(dec_dict);

    def evaluate(step_id):
        global loader_test;
        dec.eval();
        loss_acc=0;
        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]


                
                W,F_1,F_2,CSI_sem_UE_1,CSI_sem_UE_2,CSI_sem_BS=dec(
                        H_batch_1 = H_batch_1,  # b,T, Nc, N_r, N_t 用户1
                        H_batch_2 = H_batch_2,  # b,T, Nc, N_r, N_t 用户2
                    );
                loss = -torch.min(calculate_average_channel_capacity(H_batch_1[:,L], F_1, W, SNR_dB),calculate_average_channel_capacity(H_batch_2[:,L], F_2, W, SNR_dB))/val_num
                loss = loss.mean()
                loss_acc += loss.item()

        return loss_acc

    




    loss_SE = evaluate(0);
    print("SE：",-loss_SE)
    return -loss_SE

#### 开始训练

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for L in [4,1,2,8]:
# for L in [16]:
    train_physical_E2E(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre,v_max_km_pre,ITERS=100_000,b=128,acc_num=1)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
# for feedback_bits in [4,16,64,256]:
for feedback_bits in [64,256]:
    train_physical_E2E(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre,v_max_km_pre,ITERS=100_000,b=128,acc_num=1)

#### 开始测试

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for L in [1,2,4,8]:
    test_physical_E2E(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for feedback_bits in [4,16,64,256,1024]:
    test_physical_E2E(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km)

### 阶段2：信源信道语义融合联合训练

#### 网络定义

In [ ]:
class Sem_fusion(nn.Module):

    def __init__(
            self,
            C_out,     #输出通道数
        ):
        super(Sem_fusion, self).__init__()
    
        self.B = 4  #量化精度
        self.C_out = C_out
        self.CSI_dim = 16
        self.d_model = 128



        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)

        self.embed_CSI=nn.Sequential(  
            ReshapeLayer([-1, Nc*d_sem]),  
            nn.Linear(Nc*d_sem, self.CSI_dim),  
        );


        
    
        self.layers = nn.Sequential(
            nn.Linear(in_features=C_dim+self.CSI_dim, out_features=C_dim),
            nn.LeakyReLU(),
            nn.Linear(in_features=C_dim, out_features=C_dim),
            nn.Sigmoid(),
        )

        self.head=nn.Sequential(  
            nn.Linear(C_dim, C_out),

        );
    
    

        
        

    def forward(
            self, 
            x_sem,   # b， C_dim, 10, 10  #语义编码结果
            CSI_sem,  # b, Nc, d_sem   #反馈比特
        ):
        x_CSI = self.embed_CSI(CSI_sem) # b, 16
        x_ini = torch.mean(x_sem, dim=(2, 3)) # b, C_dim
        context_input = torch.cat([x_CSI,x_ini],dim=1) # b, C_dim + 16

        mask = self.layers(context_input).view(-1, C_dim, 1, 1)

        x = mask * x_sem  # b， C_dim, 10, 10  #语义编码结果

        x = x.permute(0,2,3,1) # b, 10, 10, C_dim
        x = self.head(x)     # b, 10, 10, C_out
        # x = x.permute(0,3,1,2) # b, C_out, 30, 40
        

        return x # b, 10, 10, C_out  #语义融合结果

# Q的数量为C_out/2/N_rf*40，根据要的Q以及N_rf反推出所需的C_out

class Sem_E2E(nn.Module):

    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
            feedback_bits,  #反馈比特数
            n_class,#类别数
            Q,      #传输数据符号数
        ):
        super(Sem_E2E, self).__init__()
        self.N_r_RF = N_r_RF
        self.N_t_RF = N_t_RF
        self.Nc = Nc

        self.Q = Q
        self.C_out = Q*2*N_t_RF
        self.MFNet = MFNet_NOMA(n_class)
        self.physical = physical_E2E(Nc,N_r, N_r_RF,N_t,N_t_RF,L,feedback_bits)
        self.fusion_1 = Sem_fusion(self.C_out)
        self.fusion_2 = Sem_fusion(self.C_out)

        self.Dembed=nn.Linear(Q*2*N_r_RF, C_dim)
        self.fusion_3 = Sem_fusion(C_dim)
        

    def forward(
            self, 
            H_batch_1,  # b,T, Nc, N_r, N_t  T就是L+Q
            H_batch_2,  # b,T, Nc, N_r, N_t
            SNR_dB_Sem, 
            x, # 
        ):
        Q = self.Q
        Nc = self.Nc
        C_out = self.C_out
        N_t_RF = self.N_t_RF
        # split data into RGB and INF
        x_rgb = self.MFNet.enc_rgb(x) # [batch,C_dim,10,10]
        x_inf = self.MFNet.enc_inf(x) # [batch,C_dim,10,10]

        

        W,F_1,F_2,CSI_sem_UE_1,CSI_sem_UE_2,CSI_sem_BS = self.physical(H_batch_1,H_batch_2)


        # Calculate effective channel
        b,_,N_r_RF,N_r = W.shape
        N_t = F_1.shape[2]
        H_eff_1 = W.reshape(b,1,1,N_r_RF,N_r) @ H_batch_1[:,L:(L+Q)] @ F_1.reshape(b,1,1,N_t,N_t_RF) #H [batch,Q,Nc,N_r_RF,N_t_RF]
        H_eff_2 = W.reshape(b,1,1,N_r_RF,N_r) @ H_batch_2[:,L:(L+Q)] @ F_2.reshape(b,1,1,N_t,N_t_RF) #H [batch,Q,Nc,N_r_RF,N_t_RF]

        #把bits和x融合
        x_1 = self.fusion_1(x_rgb,CSI_sem_UE_1)  # [batch,10,10,C_out]
        x_2 = self.fusion_2(x_inf,CSI_sem_UE_2)  # [batch,10,10,C_out]



        x1 = x_1.reshape(b,Nc,Q,N_t_RF,1,2) #b,Nc,Q,N_t_RF,1,2
        x1 = x1.permute(0,2,1,3,4,5)  #b,Q,Nc,N_t_RF,1,2
        x1 = x1[:,:,:,:,:,0] + 1j*x1[:,:,:,:,:,0] #b,Q,Nc,N_t_RF,1
        norm_factor = torch.sqrt(torch.sum(torch.abs(x1)**2, dim=(-4,-3,-2, -1), keepdim=True))
        x1 = x1 / norm_factor * torch.sqrt(torch.tensor(Q*Nc*N_t_RF, dtype=H_batch_1.dtype, device=H_batch_1.device))
        x1 = H_eff_1 @ x1           #b,Q,Nc,N_r_RF,1

        x2 = x_2.reshape(b,Nc,Q,N_t_RF,1,2) #b,Nc,Q,N_t_RF,1,2
        x2 = x2.permute(0,2,1,3,4,5)  #b,Q,Nc,N_t_RF,1,2
        x2 = x2[:,:,:,:,:,0] + 1j*x2[:,:,:,:,:,0] #b,Q,Nc,N_t_RF,1
        norm_factor = torch.sqrt(torch.sum(torch.abs(x2)**2, dim=(-4,-3,-2, -1), keepdim=True))
        x2 = x2 / norm_factor * torch.sqrt(torch.tensor(Q*Nc*N_t_RF, dtype=H_batch_1.dtype, device=H_batch_1.device))
        x2 = H_eff_2 @ x2           #b,Q,Nc,N_r_RF,1

        sigma = 10**(-SNR_dB_Sem/10)

        n = (torch.randn(b,Q,Nc,N_r_RF,1).cuda() + 1j*torch.randn(b,Q,Nc,N_r_RF,1).cuda())/sqrt(2)*sqrt(sigma)
        x = x1 + x2 + n # b,Q,Nc,N_r_RF,1

        norm_factor = torch.sqrt(torch.sum(torch.abs(x)**2, dim=(-4,-3,-2, -1), keepdim=True))
        x = x / norm_factor * torch.sqrt(torch.tensor(Q*Nc*N_r_RF, dtype=H_batch_1.dtype, device=H_batch_1.device)) # b,Q,Nc,N_r_RF,1

        
        x = torch.cat([x.real,x.imag],dim=-1) # b,Q,Nc,N_r_RF,2
        x = x.permute(0,2,1,3,4) # b,Nc,Q,N_r_RF,2
        x = x.reshape(b,10,10,Q*2*N_r_RF)
        x = self.Dembed(x) #b, 10,10,C_dim
        x = x.permute(0,3,1,2) #b,C_dim,10,10
        ####接收端融合####
        x = self.fusion_3(x,CSI_sem_BS)  #b,10,10,C_dim
        x = x.permute(0,3,1,2) #b,C_dim,10,10
        #################
        x = self.MFNet.dec(x)

        power_signal = torch.sqrt(torch.sum(torch.abs(x1 + x2)**2))/b

        return x,power_signal

#### 训练函数

In [ ]:
def train_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre = True,Phy_pre = True,feedback_bits_pre = None,SNR_dB_pre = None,v_max_km_pre=None,Q_pre=None,ITERS=20_000,b=8):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;

    model_name_physical = 'physical_E2E'+ str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    model_name = 'Sem_E2E'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)

    dec=Sem_E2E(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L,feedback_bits=feedback_bits,n_class=n_class,Q=Q).cuda()
    
    opt_dec=torch.optim.AdamW(
        dec.parameters(), 
        lr=lr, 
    )
    T = L+Q
    load_path=save_path+"Sem_E2E"+"/";
    load_path_physical = save_path+"physical_E2E"+"/";
    if feedback_bits_pre == None:
        if Phy_pre == True:
            dec_load_path=load_path_physical+model_name_physical+"_model.pth.tar";
            ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
            pretrained_dict=ckpt['state_dict'];
            model_dict=dec.physical.state_dict();
            pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict.keys()};
            model_dict.update(pretrained_dict);
            dec.physical.load_state_dict(model_dict);
            print('物理层预训练网络读取成功')
        if NOMA_pre == True:
            model_dir_pre = 'weights/'
            model_dir_pre = os.path.join(model_dir_pre, 'MFNet_NOMA')
            # os.makedirs(model_dir_pre, exist_ok=True)
            final_model_file_pre      = os.path.join(model_dir_pre, 'final_pretrain1_C4.pth')
            dec.MFNet.load_state_dict(torch.load(final_model_file_pre, map_location='cuda:0'))
            print('语义编码预训练网络读取成功')
    else:
        dec_dict = dec.state_dict()
        model_name_pre = 'Sem_E2E'+str(SNR_dB_pre) + '_' + str(feedback_bits_pre)+'_L' + str(L)+'_v' + str(v_max_km_pre)+'_Q' + str(Q_pre)
        dec_load_path=load_path+model_name_pre+"_model.pth.tar";
        ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
        pretrained_dict=ckpt['state_dict'];
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict and ".quan" not in k and ".Dequan" not in k and ".fusion_1.head" not in k and ".fusion_2.head" not in k  and ".Dembed" not in k}
        dec_dict.update(pretrained_dict);
        dec.load_state_dict(dec_dict);
        print('全部预训练参数读取成功')

###################################################################
    def train(epo, model, train_loader, optimizer):


        loss_avg = 0.
        acc_avg  = 0.
        start_t = t = time.time()
        model.train()

        for it, (images, labels, names) in enumerate(tqdm(train_loader, desc="Training", leave=True)):
            images = Variable(images).cuda(args.gpu) 
            labels = Variable(labels).cuda(args.gpu)

            b = images.shape[0]
            alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
            H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j
            for i in range(T):
                t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                H_batch[:,i] = H
            H_batch_1 = H_batch[0:b]
            H_batch_2 = H_batch[b:2*b]



            optimizer.zero_grad()
            logits,power_signal = model(H_batch_1,H_batch_2, SNR_dB, images)
            # print(labels.min(), labels.max())

            loss = F.cross_entropy(logits, labels)
            loss.backward()
            optimizer.step()

            acc = calculate_accuracy(logits, labels)
            loss_avg += float(loss.item())
            acc_avg  += float(acc)

            # cur_t = time.time()
            # if cur_t-t > 5:
            #     print('|- epo %s/%s. train iter %s/%s. %.2f img/sec loss: %.4f, acc: %.4f' \
            #         % (epo, args.epoch_max, it+1, train_loader.n_iter, (it+1)*args.batch_size/(cur_t-start_t), float(loss), float(acc)))
            #     t += 5

        content = '| epo:%s/%s train_loss_avg:%.4f train_acc_avg:%.4f ' \
                % (epo, args.epoch_max, loss_avg/train_loader.n_iter, acc_avg/train_loader.n_iter)
        print(content)
        with open(log_file, 'a') as appender:
            appender.write(content)


    def validation(epo, model, val_loader):

        loss_avg = 0.
        acc_avg  = 0.
        power_avg = 0.
        start_t = time.time()
        model.eval()

        cf = np.zeros((n_class, n_class))

        with torch.no_grad():
            for it, (images, labels, names) in enumerate(val_loader):
                images = Variable(images)
                labels = Variable(labels)
                if args.gpu >= 0:
                    images = images.cuda(args.gpu)
                    labels = labels.cuda(args.gpu)
                
                b = images.shape[0]
                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j
                for i in range(T):
                    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]

                logits,power_signal = model(H_batch_1,H_batch_2, SNR_dB, images)
                loss = F.cross_entropy(logits, labels)
                acc = calculate_accuracy(logits, labels)
                loss_avg += float(loss)
                acc_avg  += float(acc)
                power_avg += float(power_signal.item())

                predictions = logits.argmax(1)
                for gtcid in range(n_class): 
                    for pcid in range(n_class):
                        gt_mask      = labels == gtcid 
                        pred_mask    = predictions == pcid
                        intersection = gt_mask * pred_mask
                        cf[gtcid, pcid] += int(intersection.sum())
            overall_acc, acc, IoU = calculate_result(cf)

                # cur_t = time.time()
                # print('|- epo %s/%s. val iter %s/%s. %.2f img/sec loss: %.4f, acc: %.4f' \
                #         % (epo, args.epoch_max, it+1, val_loader.n_iter, (it+1)*args.batch_size/(cur_t-start_t), float(loss), float(acc)))

        content = '| val_loss_avg:%.4f val_acc_avg:%.4f  val_power_avg:%.2f' \
                % (loss_avg/val_loader.n_iter, acc_avg/val_loader.n_iter, power_avg/val_loader.n_iter)
        print(content)

        content = '| class accuracy avg:%.4f class IoU avg:%.4f' \
                % (acc.mean(), IoU.mean())
        print(content)
        # with open(log_file, 'a') as appender:
        #     appender.write(content)
        return IoU.mean()
    
    def save_ckpt(folder_name):
        folder_path=save_path+folder_name+"/";
        if not os.path.exists(folder_path):
            os.makedirs(folder_path);

        torch.save({'state_dict': dec.state_dict()}, folder_path+model_name+"_model.pth.tar");
        # torch.save({'best_loss': best_loss, 
        #             'opt_dec_dict': opt_dec.state_dict(), 
        #             }, folder_path+model_name+"_opt.pth.tar");


#################################################################################################
    import sys
    sys.argv = ['run.py']
    parser = argparse.ArgumentParser(description='Train MFNet with pytorch')
    parser.add_argument('--model_name',  '-M',  type=str, default='Sem_E2E')
    parser.add_argument('--batch_size',  '-B',  type=int, default=b)

    parser.add_argument('--epoch_max' ,  '-E',  type=int, default=100)
    # parser.add_argument('--epoch_max' ,  '-E',  type=int, default=2)

    parser.add_argument('--epoch_from',  '-EF', type=int, default=1)
    parser.add_argument('--gpu',         '-G',  type=int, default=0)
    parser.add_argument('--num_workers', '-j',  type=int, default=0)
    args = parser.parse_args()
    model_dir = 'weights/'
    model_dir = os.path.join(model_dir, args.model_name)
    os.makedirs(model_dir, exist_ok=True)
    log_file              = os.path.join(model_dir, 'log_Sem_E2E.txt')

    print('| training %s on GPU #%d with pytorch' % (args.model_name, args.gpu))
    print('| from epoch %d / %s' % (args.epoch_from, args.epoch_max))
    print('| model will be saved in: %s' % model_dir)



    best_mIOU = -1

    train_dataset = MF_dataset(data_dir, 'train', have_label=True, transform=augmentation_methods)
    val_dataset  = MF_dataset(data_dir, 'val', have_label=True)

    train_loader  = DataLoader(
        dataset     = train_dataset,
        batch_size  = args.batch_size,
        shuffle     = True,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = True
    )
    val_loader  = DataLoader(
        dataset     = val_dataset,
        batch_size  = args.batch_size,
        shuffle     = False,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = False
    )
    train_loader.n_iter = len(train_loader)
    val_loader.n_iter   = len(val_loader)

    for epo in (range(args.epoch_from, args.epoch_max+1)):
        print('\n| epo #%s begin...' % epo)

        train(epo, dec, train_loader, opt_dec)
        mIOU = validation(epo, dec, val_loader)

        if mIOU>best_mIOU:
            best_mIOU=mIOU;
            save_ckpt("Sem_E2E");
            print("| save best")
        
        






#### 开始训练

In [ ]:
# t = 0
# Nc = 100
# N = 2
# Nt = [1,8]
# Nr = [1,64]
# N_r_RF = 2
# N_t_RF = 2

# L = 8

# feedback_bits_pre = None
# SNR_dB_pre = None
# v_max_km_pre = None
# Q_pre = None

# NOMA_pre=True
# Phy_pre=True

# feedback_bits = 1024
# SNR_dB = -25
# v_max_km = 120
# Q = 4
# for L in [16]:
#     train_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 1
for L in [2]:
    train_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
# t = 0
# Nc = 100
# N = 2
# Nt = [1,8]
# Nr = [1,64]
# N_r_RF = 2
# N_t_RF = 2

# L = 8

# feedback_bits_pre = None
# SNR_dB_pre = None
# v_max_km_pre = None
# Q_pre = None

# NOMA_pre=True
# Phy_pre=True

# feedback_bits = 1024
# SNR_dB = -25
# v_max_km = 120
# Q = 4
# for feedback_bits in [16,64,256]:
#     train_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 1
for feedback_bits in [4,16,64,256]:
    train_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for Q in [2,3,5]:
    train_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
# t = 0
# Nc = 100
# N = 2
# Nt = [1,8]
# Nr = [1,64]
# N_r_RF = 2
# N_t_RF = 2

# L = 8

# feedback_bits_pre = None
# SNR_dB_pre = None
# v_max_km_pre = None
# Q_pre = None

# NOMA_pre=True
# Phy_pre=True

# feedback_bits = 1024
# SNR_dB = -25
# v_max_km = 120
# Q = 4
# for SNR_dB in [-5,0]:
#     train_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

#### 测试

##### demo

In [ ]:
from util.util import visualize
import sys


t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2
N_t = Nt[0]*Nt[1]
N_r = Nr[0]*Nr[1]
L = 8
feedback_bits = 1024
SNR_dB = -25
v_max_km = 120

Q = 4
T = L+Q
U = 2

sys.argv = ['run.py']


parser = argparse.ArgumentParser(description='Run MFNet demo with pytorch')
parser.add_argument('--model_name', '-M',  type=str, default='Sem_E2E')
parser.add_argument('--gpu',        '-G',  type=int, default=0)
args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)

model_name = 'Sem_E2E'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)

final_model_file      = os.path.join(model_dir, model_name+"_model.pth.tar")
print('| running %s demo on GPU #%d with pytorch' % (args.model_name, args.gpu))




model = eval(args.model_name)(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L,feedback_bits=feedback_bits,n_class=n_class,Q=Q)
if args.gpu >= 0: model.cuda(args.gpu)

device = 0
save_path="./weights/";
load_path=save_path+"Sem_E2E"+"/";
dec_dict = model.state_dict()
model_name_pre = 'Sem_E2E'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)
dec_load_path=load_path+model_name_pre+"_model.pth.tar";
ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
pretrained_dict=ckpt['state_dict'];
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
dec_dict.update(pretrained_dict);
model.load_state_dict(dec_dict);
print('全部参数读取成功')

files = os.listdir('image')
images = []
fpath  = []
fpath_out = []
fpath_original = []
for file in files:
    if file[-3:] != 'png': continue
    fpath.append('image/'+file)
    fpath_out.append('seg_E2E_NOMA/'+file)
    fpath_original.append('image_original/'+file)
    images.append( np.asarray(Image.open('image/'+file)) )
images = np.asarray(images, dtype=np.float32).transpose((0,3,1,2))/255.
images = Variable(torch.tensor(images))
if args.gpu >= 0: images = images.cuda(args.gpu)

model.eval()
with torch.no_grad():
    b = images.shape[0]
    alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
    H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j
    for i in range(T):
        t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
        H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
        H_batch[:,i] = H
    H_batch_1 = H_batch[0:b]
    H_batch_2 = H_batch[b:2*b]
    logits,power_signal = model(H_batch_1,H_batch_2, SNR_dB, images)
    predictions = logits.argmax(1)
    save_images(images, fpath_original)  #保存原始图像
    visualize(fpath_out, predictions)

##### 性能指标 

In [ ]:
from util.util import calculate_accuracy, calculate_result
import sys

def test_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q):
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    
    T = L+Q
    U = 2





    from util.util import calculate_accuracy, calculate_result
    import sys
    sys.argv = ['run.py']



    cf = np.zeros((n_class, n_class))


    parser = argparse.ArgumentParser(description='Test MFNet with pytorch')
    parser.add_argument('--model_name', '-M',  type=str, default='Sem_E2E')
    parser.add_argument('--batch_size',  '-B',  type=int, default=16)
    parser.add_argument('--gpu',        '-G',  type=int, default=0)
    parser.add_argument('--num_workers', '-j',  type=int, default=0)
    args = parser.parse_args()
    model_dir = 'weights/'
    model_dir = os.path.join(model_dir, args.model_name)

    model_name = 'Sem_E2E'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)

    final_model_file      = os.path.join(model_dir, model_name+"_model.pth.tar")
    # print('| running %s demo on GPU #%d with pytorch' % (args.model_name, args.gpu))




    model = eval(args.model_name)(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L,feedback_bits=feedback_bits,n_class=n_class,Q=Q)
    if args.gpu >= 0: model.cuda(args.gpu)

    device = 0
    save_path="./weights/";
    load_path=save_path+"Sem_E2E"+"/";
    dec_dict = model.state_dict()
    model_name_pre = 'Sem_E2E'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)
    dec_load_path=load_path+model_name_pre+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    model.load_state_dict(dec_dict);
    # print('全部参数读取成功')

    test_dataset  = MF_dataset(data_dir, 'test', have_label=True)
    test_loader  = DataLoader(
        dataset     = test_dataset,
        batch_size  = args.batch_size,
        shuffle     = False,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = False
    )
    test_loader.n_iter = len(test_loader)

    loss_avg = 0.
    acc_avg  = 0.
    model.eval()
    results_IOU = 0
    val_num = 5
    with torch.no_grad():
        for aa in range(val_num):
            cf = np.zeros((n_class, n_class))
            for it, (images, labels, names) in enumerate(test_loader):
                images = Variable(images)
                labels = Variable(labels)
                if args.gpu >= 0:
                    images = images.cuda(args.gpu)
                    labels = labels.cuda(args.gpu)

                b = images.shape[0]
                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j
                for i in range(T):
                    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]
                logits,power_signal = model(H_batch_1,H_batch_2, SNR_dB, images)
                loss = F.cross_entropy(logits, labels)
                acc = calculate_accuracy(logits, labels)
                loss_avg += float(loss)
                acc_avg  += float(acc)

                # print('|- test iter %s/%s. loss: %.4f, acc: %.4f' \
                #         % (it+1, test_loader.n_iter, float(loss), float(acc)))

                predictions = logits.argmax(1)
                for gtcid in range(n_class): 
                    for pcid in range(n_class):
                        gt_mask      = labels == gtcid 
                        pred_mask    = predictions == pcid
                        intersection = gt_mask * pred_mask
                        cf[gtcid, pcid] += int(intersection.sum())

            overall_acc, acc, IoU = calculate_result(cf)
            results_IOU += IoU.mean()/val_num

    # print('| overall accuracy:', overall_acc)
    # print('| accuracy of each class:', acc)
    # print('| class accuracy avg:', acc.mean())
    # print('| IoU:', IoU)
    print('| class IoU avg:', results_IOU)


In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for L in [1,2,4,8]:   #L=1的时候MIOU为0，所以模型没有保存，这里直接放弃掉，根据Q=4推算大概在0.13左右
    test_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 1
for L in [1,2,4,8]:   #L=1的时候MIOU为0，所以模型没有保存，这里直接放弃掉，根据Q=4推算大概在0.13左右
    test_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for feedback_bits in [4,16,64,256,1024]:
    test_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 1
for feedback_bits in [4,16,64,256,1024]:
    test_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 1
for Q in [1,2,3,4,5]:
    test_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for SNR_dB in [-40,-35,-30,-25,-20,-15,-10]:
    test_Sem_E2E(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

## 基线（E2E正交设计）

### 预训练OMA的语义提取

In [ ]:
class ConvBnLeakyRelu2d(nn.Module):
    # convolution
    # batch normalization
    # leaky relu
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1, dilation=1, groups=1):
        super(ConvBnLeakyRelu2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, dilation=dilation, groups=groups)
        self.bn   = nn.BatchNorm2d(out_channels)
    def forward(self, x):
        return F.leaky_relu(self.bn(self.conv(x)), negative_slope=0.2)


class MiniInception(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MiniInception, self).__init__()
        self.conv1_left  = ConvBnLeakyRelu2d(in_channels,   out_channels//2)
        self.conv1_right = ConvBnLeakyRelu2d(in_channels,   out_channels//2, padding=2, dilation=2)
        self.conv2_left  = ConvBnLeakyRelu2d(out_channels,  out_channels//2)
        self.conv2_right = ConvBnLeakyRelu2d(out_channels,  out_channels//2, padding=2, dilation=2)
        self.conv3_left  = ConvBnLeakyRelu2d(out_channels,  out_channels//2)
        self.conv3_right = ConvBnLeakyRelu2d(out_channels,  out_channels//2, padding=2, dilation=2)
    def forward(self,x):
        x = torch.cat((self.conv1_left(x), self.conv1_right(x)), dim=1)
        x = torch.cat((self.conv2_left(x), self.conv2_right(x)), dim=1)
        x = torch.cat((self.conv3_left(x), self.conv3_right(x)), dim=1)
        return x


class MFNet_Enc_rgb_OMA(nn.Module):

    def __init__(self, ):
        super(MFNet_Enc_rgb_OMA, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        compress_ch = [192,96,96,C_dim]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.conv1_rgb   = ConvBnLeakyRelu2d(3, rgb_ch[0])
        self.conv2_1_rgb = ConvBnLeakyRelu2d(rgb_ch[0], rgb_ch[1])
        self.conv2_2_rgb = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[1])
        self.conv3_1_rgb = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[2])
        self.conv3_2_rgb = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[2])
        self.conv4_rgb   = MiniInception(rgb_ch[2], rgb_ch[3])
        self.conv5_rgb   = MiniInception(rgb_ch[3], rgb_ch[4])
        self.compress_1_rgb = ConvBnLeakyRelu2d(compress_ch[0], compress_ch[1])
        self.compress_2_rgb = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[2])
        self.compress_3_rgb = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[3])
    def forward(self, x):
        # split data into RGB and INF
        x_rgb = x[:,:3]

        # encode
        x_rgb    = self.conv1_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb, kernel_size=2, stride=2) # pool1
        x_rgb    = self.conv2_1_rgb(x_rgb)
        x_rgb_p2 = self.conv2_2_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p2, kernel_size=2, stride=2) # pool2
        x_rgb    = self.conv3_1_rgb(x_rgb)
        x_rgb_p3 = self.conv3_2_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p3, kernel_size=2, stride=2) # pool3
        x_rgb_p4 = self.conv4_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p4, kernel_size=2, stride=2) # pool4 #[batch,2,30,40]
        x_rgb    = self.conv5_rgb(x_rgb)  #[batch,96,30,40]
        x_rgb    = F.max_pool2d(x_rgb, kernel_size=(6,4), stride=(6,4)) # pool5  

        x_rgb    = self.compress_1_rgb(x_rgb)  
        x_rgb    = self.compress_2_rgb(x_rgb)
        x_rgb    = self.compress_3_rgb(x_rgb) #[batch,C_dim,5,10]
        return x_rgb


class MFNet_Enc_inf_OMA(nn.Module):

    def __init__(self, ):
        super(MFNet_Enc_inf_OMA, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        # 压缩后的每个样本维度为[2，30，40]我们正好可以设置为单流2个实虚部组成的2、30个子载波和40个OFDM符号
        compress_ch = [192,96,96,C_dim]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.conv1_inf   = ConvBnLeakyRelu2d(1, rgb_ch[0])
        self.conv2_1_inf = ConvBnLeakyRelu2d(rgb_ch[0], rgb_ch[1])
        self.conv2_2_inf = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[1])
        self.conv3_1_inf = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[2])
        self.conv3_2_inf = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[2])
        self.conv4_inf   = MiniInception(rgb_ch[2], rgb_ch[3])
        self.conv5_inf   = MiniInception(rgb_ch[3], rgb_ch[4])
        self.compress_1_inf = ConvBnLeakyRelu2d(compress_ch[0], compress_ch[1])
        self.compress_2_inf = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[2])
        self.compress_3_inf = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[3])
    def forward(self, x):
        # split data into RGB and INF
        x_inf = x[:,3:]

        # encode
        x_inf    = self.conv1_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf, kernel_size=2, stride=2) # pool1
        x_inf    = self.conv2_1_inf(x_inf)
        x_inf_p2 = self.conv2_2_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p2, kernel_size=2, stride=2) # pool2
        x_inf    = self.conv3_1_inf(x_inf)
        x_inf_p3 = self.conv3_2_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p3, kernel_size=2, stride=2) # pool3
        x_inf_p4 = self.conv4_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p4, kernel_size=2, stride=2) # pool4 #[batch,96,30,40]
        x_inf    = self.conv5_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf, kernel_size=(6,4), stride=(6,4)) # pool5  

        x_inf    = self.compress_1_inf(x_inf)  
        x_inf    = self.compress_2_inf(x_inf)  
        x_inf    = self.compress_3_inf(x_inf)    #[batch,C_dim,5,10]
        return x_inf

class MFNet_Dec_OMA(nn.Module):

    def __init__(self, n_class):
        super(MFNet_Dec_OMA, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        # 压缩后的每个样本维度为 #[C_dim,10,10]我们keyi
        compress_ch = [192,96,96,C_dim]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.decompress_3 = ConvBnLeakyRelu2d(compress_ch[3], compress_ch[2])
        self.decompress_2 = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[1])
        self.decompress_1 = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[0])


        self.decode5     = ConvBnLeakyRelu2d(rgb_ch[4], rgb_ch[3])
        self.decode4     = ConvBnLeakyRelu2d(rgb_ch[3], rgb_ch[2])
        self.decode3     = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[1])
        self.decode2     = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[0])
        self.decode1     = ConvBnLeakyRelu2d(rgb_ch[0], n_class)
    def forward(self, x):
        #[batch,8,30,40]
        # x = self.decompress_4(x)
        # x = x.reshape(-1,8,self.pho,15,20).permute(0,2,1,3,4)
        # x = x.reshape(-1,self.pho*8,15,20)
        # x = self.decompress_4(x.permute(0,1,3,2)).permute(0,1,3,2)
        x = self.decompress_3(x)
        x = self.decompress_2(x)
        x = self.decompress_1(x) #[batch,96,5,10]

        # decode
        x = F.interpolate(x, scale_factor=(6,4), mode='nearest')# unpool5
        x = self.decode5(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool4
        x = self.decode4(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool3
        x = self.decode3(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool2
        x = self.decode2(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool1
        x = self.decode1(x)

        return x

class MFNet_OMA(nn.Module):

    def __init__(self, n_class):
        super(MFNet_OMA, self).__init__()
        self.enc_rgb = MFNet_Enc_rgb_OMA()
        self.enc_inf = MFNet_Enc_inf_OMA()
        self.dec = MFNet_Dec_OMA(n_class)
        
        

    def forward(self, x):
        x_rgb = self.enc_rgb(x)
        x_inf = self.enc_inf(x)
        x = x_rgb + x_inf 
        x = self.dec(x)
        return x
        # x_rgb = self.enc_rgb(x) 
        # x_rgb = x_rgb[:,0] + 1j*x_rgb[:,1]
        # x_rgb = x_rgb * (torch.randn(x.shape[0],1,40) + 1j*torch.randn(x.shape[0],1,40)).cuda()
        # x_inf = self.enc_inf(x)
        # x_inf = x_inf[:,0] + 1j*x_inf[:,1]
        # x_inf = x_inf * (torch.randn(x.shape[0],1,40) + 1j*torch.randn(x.shape[0],1,40)).cuda()
        # x = x_rgb + x_inf 
        # x = x.reshape(-1,1,30,40)
        # x = torch.cat([x.real,x.imag],dim=1)
        # x = self.dec(x)
        # return x

#### 训练

In [ ]:
import sys
sys.argv = ['run.py']
parser = argparse.ArgumentParser(description='Train MFNet with pytorch')
parser.add_argument('--model_name',  '-M',  type=str, default='MFNet_OMA')
parser.add_argument('--batch_size',  '-B',  type=int, default=8)

parser.add_argument('--epoch_max' ,  '-E',  type=int, default=100)
# parser.add_argument('--epoch_max' ,  '-E',  type=int, default=2)

parser.add_argument('--epoch_from',  '-EF', type=int, default=1)
parser.add_argument('--gpu',         '-G',  type=int, default=0)
parser.add_argument('--num_workers', '-j',  type=int, default=0)
args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)
os.makedirs(model_dir, exist_ok=True)
checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1.pth')
checkpoint_optim_file = os.path.join(model_dir, 'tmp_pretrain1.optim')
final_model_file      = os.path.join(model_dir, 'final_pretrain1_C4.pth')
log_file              = os.path.join(model_dir, 'log_pretrain1.txt')

print('| training %s on GPU #%d with pytorch' % (args.model_name, args.gpu))
print('| from epoch %d / %s' % (args.epoch_from, args.epoch_max))
print('| model will be saved in: %s' % model_dir)


In [ ]:
model = eval(args.model_name)(n_class=n_class)
if args.gpu >= 0: model.cuda(args.gpu)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_start, momentum=0.9, weight_decay=0.0005) 
# optimizer = torch.optim.Adam(model.parameters(), lr=lr_start)

if args.epoch_from > 1:
    print('| loading checkpoint file %s... ' % checkpoint_model_file, end='')
    model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
    optimizer.load_state_dict(torch.load(checkpoint_optim_file))
    print('done!')

train_dataset = MF_dataset(data_dir, 'train', have_label=True, transform=augmentation_methods)
val_dataset  = MF_dataset(data_dir, 'val', have_label=True)

train_loader  = DataLoader(
    dataset     = train_dataset,
    batch_size  = args.batch_size,
    shuffle     = True,
    num_workers = args.num_workers,
    pin_memory  = True,
    drop_last   = True
)
val_loader  = DataLoader(
    dataset     = val_dataset,
    batch_size  = args.batch_size,
    shuffle     = False,
    num_workers = args.num_workers,
    pin_memory  = True,
    drop_last   = False
)
train_loader.n_iter = len(train_loader)
val_loader.n_iter   = len(val_loader)

for epo in (range(args.epoch_from, args.epoch_max+1)):
    print('\n| epo #%s begin...' % epo)

    train(epo, model, train_loader, optimizer)
    validation(epo, model, val_loader)

    # save check point model
    # print('| saving check point model file... ', end='')
    torch.save(model.state_dict(), checkpoint_model_file)
    torch.save(optimizer.state_dict(), checkpoint_optim_file)
    # print('done!')
if os.path.exists(final_model_file):
    os.remove(final_model_file) # 删除已存在的文件
os.rename(checkpoint_model_file, final_model_file)

#### 测试

##### demo

In [ ]:
from util.util import visualize
import sys

sys.argv = ['run.py']


parser = argparse.ArgumentParser(description='Run MFNet demo with pytorch')
parser.add_argument('--model_name', '-M',  type=str, default='MFNet_OMA')
parser.add_argument('--gpu',        '-G',  type=int, default=0)
args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)

checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1.pth')
final_model_file      = os.path.join(model_dir, 'final_pretrain1_C4.pth')
print('| running %s demo on GPU #%d with pytorch' % (args.model_name, args.gpu))




model = eval(args.model_name)(n_class=n_class)
if args.gpu >= 0: model.cuda(args.gpu)
if os.path.exists(final_model_file):
    model.load_state_dict(torch.load(final_model_file, map_location='cuda:0'))
elif os.path.exists(checkpoint_model_file):
    model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
else:
    raise Exception('| model file do not exists in %s' % model_dir)
print('| model loaded!')

files = os.listdir('image')
images = []
fpath  = []
fpath_out = []
fpath_original = []
for file in files:
    if file[-3:] != 'png': continue
    fpath.append('image/'+file)
    fpath_out.append('seg_MFNet_OMA/'+file)
    fpath_original.append('image_original/'+file)
    images.append( np.asarray(Image.open('image/'+file)) )
images = np.asarray(images, dtype=np.float32).transpose((0,3,1,2))/255.
images = Variable(torch.tensor(images))
if args.gpu >= 0: images = images.cuda(args.gpu)

model.eval()
with torch.no_grad():
    logits = model(images)
    predictions = logits.argmax(1)
    save_images(images, fpath_original)  #保存原始图像
    visualize(fpath_out, predictions)


##### 性能指标

In [ ]:
from util.util import calculate_accuracy, calculate_result
import sys

sys.argv = ['run.py']


cf = np.zeros((n_class, n_class))


parser = argparse.ArgumentParser(description='Test MFNet with pytorch')
parser.add_argument('--model_name', '-M',  type=str, default='MFNet_OMA')
parser.add_argument('--batch_size',  '-B',  type=int, default=16)
parser.add_argument('--gpu',        '-G',  type=int, default=0)
parser.add_argument('--num_workers', '-j',  type=int, default=0)



args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)

checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1.pth')
final_model_file      = os.path.join(model_dir, 'final_pretrain1_C4.pth')
print('| testing %s on GPU #%d with pytorch' % (args.model_name, args.gpu))

model = eval(args.model_name)(n_class=n_class)
if args.gpu >= 0: model.cuda(args.gpu)
if os.path.exists(final_model_file):
    model.load_state_dict(torch.load(final_model_file, map_location='cuda:0'))
elif os.path.exists(checkpoint_model_file):
    model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
else:
    raise Exception('| model file do not exists in %s' % model_dir)
print('| model loaded!')


test_dataset  = MF_dataset(data_dir, 'test', have_label=True)
test_loader  = DataLoader(
    dataset     = test_dataset,
    batch_size  = args.batch_size,
    shuffle     = False,
    num_workers = args.num_workers,
    pin_memory  = True,
    drop_last   = False
)
test_loader.n_iter = len(test_loader)

loss_avg = 0.
acc_avg  = 0.
model.eval()
with torch.no_grad():
    for it, (images, labels, names) in enumerate(test_loader):
        images = Variable(images)
        labels = Variable(labels)
        if args.gpu >= 0:
            images = images.cuda(args.gpu)
            labels = labels.cuda(args.gpu)

        logits = model(images)
        loss = F.cross_entropy(logits, labels)
        acc = calculate_accuracy(logits, labels)
        loss_avg += float(loss)
        acc_avg  += float(acc)

        print('|- test iter %s/%s. loss: %.4f, acc: %.4f' \
                % (it+1, test_loader.n_iter, float(loss), float(acc)))

        predictions = logits.argmax(1)
        for gtcid in range(n_class): 
            for pcid in range(n_class):
                gt_mask      = labels == gtcid 
                pred_mask    = predictions == pcid
                intersection = gt_mask * pred_mask
                cf[gtcid, pcid] += int(intersection.sum())

overall_acc, acc, IoU = calculate_result(cf)

print('| overall accuracy:', overall_acc)
print('| accuracy of each class:', acc)
print('| class accuracy avg:', acc.mean())
print('| IoU:', IoU)
print('| class IoU avg:', IoU.mean())



### 信源信道语义融合联合训练（E2E正交）

#### 网络和训练函数

In [ ]:
class Sem_fusion_OMA(nn.Module):

    def __init__(
            self,
            C_out,     #输出通道数
        ):
        super(Sem_fusion_OMA, self).__init__()
    
        self.B = 4  #量化精度
        self.C_out = C_out
        self.CSI_dim = 16
        self.d_model = 128



        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)

        self.embed_CSI=nn.Sequential(  
            ReshapeLayer([-1, Nc*d_sem]),  
            nn.Linear(Nc*d_sem, self.CSI_dim),  
        );


        
    
        self.layers = nn.Sequential(
            nn.Linear(in_features=C_dim+self.CSI_dim, out_features=C_dim),
            nn.LeakyReLU(),
            nn.Linear(in_features=C_dim, out_features=C_dim),
            nn.Sigmoid(),
        )

        self.head=nn.Sequential(  
            nn.Linear(C_dim, C_out),

        );
    
    

        
        

    def forward(
            self, 
            x_sem,   # b， C_dim, 5, 10  #语义编码结果
            CSI_sem,  # b, Nc, d_sem   #反馈比特
        ):
        x_CSI = self.embed_CSI(CSI_sem) # b, 16
        x_ini = torch.mean(x_sem, dim=(2, 3)) # b, C_dim
        context_input = torch.cat([x_CSI,x_ini],dim=1) # b, C_dim + 16

        mask = self.layers(context_input).view(-1, C_dim, 1, 1)

        x = mask * x_sem  # b， C_dim, 5, 10  #语义编码结果

        x = x.permute(0,2,3,1) # b, 5, 10, C_dim
        x = self.head(x)     # b, 5, 10, C_out
        

        return x # b, 5, 10, C_out  #语义融合结果

# Q的数量为C_out/2/N_rf*40，根据要的Q以及N_rf反推出所需的C_out

class Sem_E2E_OMA(nn.Module):

    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
            feedback_bits,  #反馈比特数
            n_class,#类别数
            Q,      #传输数据符号数
        ):
        super(Sem_E2E_OMA, self).__init__()
        self.N_r_RF = N_r_RF
        self.N_t_RF = N_t_RF
        self.Nc = Nc

        self.Q = Q
        self.C_out = Q*2*N_t_RF
        self.MFNet = MFNet_OMA(n_class)
        self.physical = physical_E2E(Nc,N_r, N_r_RF,N_t,N_t_RF,L,feedback_bits)
        self.fusion_1 = Sem_fusion_OMA(self.C_out)
        self.fusion_2 = Sem_fusion_OMA(self.C_out)

        # self.embed_1 = nn.Linear(C_dim, self.C_out)
        # self.embed_2 = nn.Linear(C_dim, self.C_out)

        self.Dembed=nn.Linear(Q*2*N_r_RF, C_dim)
        self.fusion_3 = Sem_fusion_OMA(C_dim)
        

    def forward(
            self, 
            H_batch_1,  # b,T, Nc, N_r, N_t  T就是L+Q
            H_batch_2,  # b,T, Nc, N_r, N_t
            SNR_dB_Sem, 
            x, # 
        ):
        Q = self.Q
        Nc = self.Nc
        C_out = self.C_out
        N_t_RF = self.N_t_RF
        # split data into RGB and INF
        x_rgb = self.MFNet.enc_rgb(x) # [batch,C_dim,5,10]
        x_inf = self.MFNet.enc_inf(x) # [batch,C_dim,5,10]

        

        W,F_1,F_2,CSI_sem_UE_1,CSI_sem_UE_2,CSI_sem_BS = self.physical(H_batch_1,H_batch_2)


        # Calculate effective channel
        b,_,N_r_RF,N_r = W.shape
        N_t = F_1.shape[2]
        H_eff_1 = W.reshape(b,1,1,N_r_RF,N_r) @ H_batch_1[:,L:(L+Q)] @ F_1.reshape(b,1,1,N_t,N_t_RF) #H [batch,Q,Nc,N_r_RF,N_t_RF]
        H_eff_2 = W.reshape(b,1,1,N_r_RF,N_r) @ H_batch_2[:,L:(L+Q)] @ F_2.reshape(b,1,1,N_t,N_t_RF) #H [batch,Q,Nc,N_r_RF,N_t_RF]

        #把bits和x融合
        x_1 = self.fusion_1(x_rgb,CSI_sem_BS)  # [batch,5,10,C_out]
        x_2 = self.fusion_2(x_inf,CSI_sem_BS)  # [batch,5,10,C_out]

        # x_1 = self.embed_1(x_rgb.permute(0,2,3,1))  # [batch,5,10,C_out]
        # x_2 = self.embed_2(x_inf.permute(0,2,3,1))  # [batch,5,10,C_out]

        sigma = 10**(-SNR_dB_Sem/10)

        x1 = x_1.reshape(b,Nc//2,Q,N_t_RF,1,2) #b,Nc//2,Q,N_t_RF,1,2
        x1 = x1.permute(0,2,1,3,4,5)  #b,Q,Nc//2,N_t_RF,1,2
        x1 = x1[:,:,:,:,:,0] + 1j*x1[:,:,:,:,:,0] #b,Q,Nc//2,N_t_RF,1
        norm_factor = torch.sqrt(torch.sum(torch.abs(x1)**2, dim=(-4,-3,-2, -1), keepdim=True))
        x1 = x1 / norm_factor * torch.sqrt(torch.tensor(Q*Nc//2*N_t_RF, dtype=H_batch_1.dtype, device=H_batch_1.device))
        x1 = H_eff_1[:,:,0::2] @ x1           #b,Q,Nc//2,N_r_RF,1
        n = (torch.randn(b,Q,Nc//2,N_r_RF,1).cuda() + 1j*torch.randn(b,Q,Nc//2,N_r_RF,1).cuda())/sqrt(2)*sqrt(sigma)
        x1 = x1 + n # b,Q,Nc//2,N_r_RF,1

        x2 = x_2.reshape(b,Nc//2,Q,N_t_RF,1,2) #b,Nc//2,Q,N_t_RF,1,2
        x2 = x2.permute(0,2,1,3,4,5)  #b,Q,Nc//2,N_t_RF,1,2
        x2 = x2[:,:,:,:,:,0] + 1j*x2[:,:,:,:,:,0] #b,Q,Nc//2,N_t_RF,1
        norm_factor = torch.sqrt(torch.sum(torch.abs(x2)**2, dim=(-4,-3,-2, -1), keepdim=True))
        x2 = x2 / norm_factor * torch.sqrt(torch.tensor(Q*Nc//2*N_t_RF, dtype=H_batch_1.dtype, device=H_batch_1.device))
        x2 = H_eff_2[:,:,1::2] @ x2           #b,Q,Nc//2,N_r_RF,1
        n = (torch.randn(b,Q,Nc//2,N_r_RF,1).cuda() + 1j*torch.randn(b,Q,Nc//2,N_r_RF,1).cuda())/sqrt(2)*sqrt(sigma)
        x2 = x2 + n # b,Q,Nc//2,N_r_RF,1

        x = x1 + x2 # b,Q,Nc//2,N_r_RF,1

        
        norm_factor = torch.sqrt(torch.sum(torch.abs(x)**2, dim=(-4,-3,-2, -1), keepdim=True))
        x = x / norm_factor * torch.sqrt(torch.tensor(Q*Nc*N_r_RF//2, dtype=H_batch_1.dtype, device=H_batch_1.device)) # b,Q,Nc,1,1

        
        x = torch.cat([x.real,x.imag],dim=-1) # b,Q,Nc//2,N_r_RF,2
        x = x.permute(0,2,1,3,4) # b,Nc//2,Q,N_r_RF,2
        x = x.reshape(b,5,10,Q*2*N_r_RF)
        x = self.Dembed(x) #b, 5,10,C_dim
        x = x.permute(0,3,1,2) #b,C_dim,5,10
        ####接收端融合####
        x = self.fusion_3(x,CSI_sem_BS)  #b,5,10,C_dim
        x = x.permute(0,3,1,2) #b,C_dim,5,10
        #################
        x = self.MFNet.dec(x)

        power_signal = torch.sqrt(torch.sum(torch.abs(x1 + x2)**2))/b

        return x,power_signal

In [ ]:
def train_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre = True,Phy_pre = True,feedback_bits_pre = None,SNR_dB_pre = None,v_max_km_pre=None,Q_pre=None,ITERS=20_000,b=8):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;

    model_name_physical = 'physical_E2E'+ str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    model_name = 'Sem_E2E_OMA'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)


    dec=Sem_E2E_OMA(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L,feedback_bits=feedback_bits,n_class=n_class,Q=Q).cuda()
    
    opt_dec=torch.optim.AdamW(
        dec.parameters(), 
        lr=lr, 
    )
    T = L+Q
    load_path=save_path+"Sem_E2E_OMA"+"/";
    load_path_physical = save_path+"physical_E2E"+"/";
    if feedback_bits_pre == None:
        if Phy_pre == True:
            dec_load_path=load_path_physical+model_name_physical+"_model.pth.tar";
            ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
            pretrained_dict=ckpt['state_dict'];
            model_dict=dec.physical.state_dict();
            pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict.keys()};
            model_dict.update(pretrained_dict);
            dec.physical.load_state_dict(model_dict);
            print('物理层预训练网络读取成功')
        if NOMA_pre == True:
            model_dir_pre = 'weights/'
            model_dir_pre = os.path.join(model_dir_pre, 'MFNet_OMA')
            # os.makedirs(model_dir_pre, exist_ok=True)
            final_model_file_pre      = os.path.join(model_dir_pre, 'final_pretrain1_C4.pth')
            dec.MFNet.load_state_dict(torch.load(final_model_file_pre, map_location='cuda:0'))
            print('语义编码预训练网络读取成功')
    else:
        dec_dict = dec.state_dict()
        model_name_pre = 'Sem_E2E_OMA'+str(SNR_dB_pre) + '_' + str(feedback_bits_pre)+'_L' + str(L)+'_v' + str(v_max_km_pre)+'_Q' + str(Q_pre)
        dec_load_path=load_path+model_name_pre+"_model.pth.tar";
        ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
        pretrained_dict=ckpt['state_dict'];
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict and ".quan" not in k and ".Dequan" not in k and ".fusion_1.head" not in k and ".fusion_2.head" not in k  and ".Dembed" not in k}
        dec_dict.update(pretrained_dict);
        dec.load_state_dict(dec_dict);
        print('全部预训练参数读取成功')

###################################################################
    def train(epo, model, train_loader, optimizer):


        loss_avg = 0.
        acc_avg  = 0.
        start_t = t = time.time()
        model.train()

        for it, (images, labels, names) in enumerate(tqdm(train_loader, desc="Training", leave=True)):
            images = Variable(images).cuda(args.gpu) 
            labels = Variable(labels).cuda(args.gpu)

            b = images.shape[0]
            alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
            H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j
            for i in range(T):
                t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                H_batch[:,i] = H
            H_batch_1 = H_batch[0:b]
            H_batch_2 = H_batch[b:2*b]



            optimizer.zero_grad()
            logits,power_signal = model(H_batch_1,H_batch_2, SNR_dB, images)
            # print(labels.min(), labels.max())

            loss = F.cross_entropy(logits, labels)
            loss.backward()
            optimizer.step()

            acc = calculate_accuracy(logits, labels)
            loss_avg += float(loss.item())
            acc_avg  += float(acc)

            # cur_t = time.time()
            # if cur_t-t > 5:
            #     print('|- epo %s/%s. train iter %s/%s. %.2f img/sec loss: %.4f, acc: %.4f' \
            #         % (epo, args.epoch_max, it+1, train_loader.n_iter, (it+1)*args.batch_size/(cur_t-start_t), float(loss), float(acc)))
            #     t += 5

        content = '| epo:%s/%s train_loss_avg:%.4f train_acc_avg:%.4f ' \
                % (epo, args.epoch_max, loss_avg/train_loader.n_iter, acc_avg/train_loader.n_iter)
        print(content)
        with open(log_file, 'a') as appender:
            appender.write(content)


    def validation(epo, model, val_loader):

        loss_avg = 0.
        acc_avg  = 0.
        power_avg = 0.
        start_t = time.time()
        model.eval()

        cf = np.zeros((n_class, n_class))

        with torch.no_grad():
            for it, (images, labels, names) in enumerate(val_loader):
                images = Variable(images)
                labels = Variable(labels)
                if args.gpu >= 0:
                    images = images.cuda(args.gpu)
                    labels = labels.cuda(args.gpu)
                
                b = images.shape[0]
                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j
                for i in range(T):
                    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]

                logits,power_signal = model(H_batch_1,H_batch_2, SNR_dB, images)
                loss = F.cross_entropy(logits, labels)
                acc = calculate_accuracy(logits, labels)
                loss_avg += float(loss)
                acc_avg  += float(acc)
                power_avg += float(power_signal.item())

                predictions = logits.argmax(1)
                for gtcid in range(n_class): 
                    for pcid in range(n_class):
                        gt_mask      = labels == gtcid 
                        pred_mask    = predictions == pcid
                        intersection = gt_mask * pred_mask
                        cf[gtcid, pcid] += int(intersection.sum())
            overall_acc, acc, IoU = calculate_result(cf)

                # cur_t = time.time()
                # print('|- epo %s/%s. val iter %s/%s. %.2f img/sec loss: %.4f, acc: %.4f' \
                #         % (epo, args.epoch_max, it+1, val_loader.n_iter, (it+1)*args.batch_size/(cur_t-start_t), float(loss), float(acc)))

        content = '| val_loss_avg:%.4f val_acc_avg:%.4f  val_power_avg:%.2f' \
                % (loss_avg/val_loader.n_iter, acc_avg/val_loader.n_iter, power_avg/val_loader.n_iter)
        print(content)

        content = '| class accuracy avg:%.4f class IoU avg:%.4f' \
                % (acc.mean(), IoU.mean())
        print(content)
        # with open(log_file, 'a') as appender:
        #     appender.write(content)
        return IoU.mean()
    
    def save_ckpt(folder_name):
        folder_path=save_path+folder_name+"/";
        if not os.path.exists(folder_path):
            os.makedirs(folder_path);

        torch.save({'state_dict': dec.state_dict()}, folder_path+model_name+"_model.pth.tar");
        # torch.save({'best_loss': best_loss, 
        #             'opt_dec_dict': opt_dec.state_dict(), 
        #             }, folder_path+model_name+"_opt.pth.tar");


#################################################################################################
    import sys
    sys.argv = ['run.py']
    parser = argparse.ArgumentParser(description='Train MFNet with pytorch')
    parser.add_argument('--model_name',  '-M',  type=str, default='Sem_E2E_OMA')
    parser.add_argument('--batch_size',  '-B',  type=int, default=b)

    parser.add_argument('--epoch_max' ,  '-E',  type=int, default=100)
    # parser.add_argument('--epoch_max' ,  '-E',  type=int, default=2)

    parser.add_argument('--epoch_from',  '-EF', type=int, default=1)
    parser.add_argument('--gpu',         '-G',  type=int, default=0)
    parser.add_argument('--num_workers', '-j',  type=int, default=0)
    args = parser.parse_args()
    model_dir = 'weights/'
    model_dir = os.path.join(model_dir, args.model_name)
    os.makedirs(model_dir, exist_ok=True)
    log_file              = os.path.join(model_dir, 'log_Sem_E2E.txt')

    print('| training %s on GPU #%d with pytorch' % (args.model_name, args.gpu))
    print('| from epoch %d / %s' % (args.epoch_from, args.epoch_max))
    print('| model will be saved in: %s' % model_dir)



    best_mIOU = -1

    train_dataset = MF_dataset(data_dir, 'train', have_label=True, transform=augmentation_methods)
    val_dataset  = MF_dataset(data_dir, 'val', have_label=True)

    train_loader  = DataLoader(
        dataset     = train_dataset,
        batch_size  = args.batch_size,
        shuffle     = True,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = True
    )
    val_loader  = DataLoader(
        dataset     = val_dataset,
        batch_size  = args.batch_size,
        shuffle     = False,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = False
    )
    train_loader.n_iter = len(train_loader)
    val_loader.n_iter   = len(val_loader)

    for epo in (range(args.epoch_from, args.epoch_max+1)):
        print('\n| epo #%s begin...' % epo)

        train(epo, dec, train_loader, opt_dec)
        mIOU = validation(epo, dec, val_loader)

        if mIOU>best_mIOU:
            best_mIOU=mIOU;
            save_ckpt("Sem_E2E_OMA");
            print("| save best")
        
        






#### 开始训练

In [ ]:
# t = 0
# Nc = 100
# N = 2
# Nt = [1,8]
# Nr = [1,64]
# N_r_RF = 2
# N_t_RF = 2

# L = 8

# feedback_bits_pre = None
# SNR_dB_pre = None
# v_max_km_pre = None
# Q_pre = None

# NOMA_pre=True
# Phy_pre=True

# feedback_bits = 1024
# SNR_dB = -25
# v_max_km = 120
# Q = 4
# for L in [2,4,8]:
#     train_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 1
for L in [2]:
    train_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
# t = 0
# Nc = 100
# N = 2
# Nt = [1,8]
# Nr = [1,64]
# N_r_RF = 2
# N_t_RF = 2

# L = 8

# feedback_bits_pre = None
# SNR_dB_pre = None
# v_max_km_pre = None
# Q_pre = None

# NOMA_pre=True
# Phy_pre=True

# feedback_bits = 1024
# SNR_dB = -25
# v_max_km = 120
# Q = 4
# for feedback_bits in [4,16,64,256]:
#     train_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
# t = 0
# Nc = 100
# N = 2
# Nt = [1,8]
# Nr = [1,64]
# N_r_RF = 2
# N_t_RF = 2

# L = 8

# feedback_bits_pre = None
# SNR_dB_pre = None
# v_max_km_pre = None
# Q_pre = None

# NOMA_pre=True
# Phy_pre=True

# feedback_bits = 1024
# SNR_dB = -25
# v_max_km = 120
# Q = 1
# for feedback_bits in [64,256]:
#     train_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

In [ ]:
# t = 0
# Nc = 100
# N = 2
# Nt = [1,8]
# Nr = [1,64]
# N_r_RF = 2
# N_t_RF = 2

# L = 8

# feedback_bits_pre = None
# SNR_dB_pre = None
# v_max_km_pre = None
# Q_pre = None

# NOMA_pre=True
# Phy_pre=True

# feedback_bits = 1024
# SNR_dB = -25
# v_max_km = 120
# Q = 4
# for Q in [2,3,5]:
#     train_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8) 

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for SNR_dB in [-15,-10,-5,0]:
    train_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q,NOMA_pre, Phy_pre,feedback_bits_pre,SNR_dB_pre,v_max_km_pre,Q_pre,ITERS=600_000,b=8)

#### 测试

##### demo

In [ ]:
from util.util import visualize
import sys


t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2
N_t = Nt[0]*Nt[1]
N_r = Nr[0]*Nr[1]
L = 8
feedback_bits = 1024
SNR_dB = -25
v_max_km = 120

Q = 4
T = L+Q
U = 2

sys.argv = ['run.py']


parser = argparse.ArgumentParser(description='Run MFNet demo with pytorch')
parser.add_argument('--model_name', '-M',  type=str, default='Sem_E2E_OMA')
parser.add_argument('--gpu',        '-G',  type=int, default=0)
args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)

model_name = 'Sem_E2E_OMA'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)

final_model_file      = os.path.join(model_dir, model_name+"_model.pth.tar")
print('| running %s demo on GPU #%d with pytorch' % (args.model_name, args.gpu))




model = eval(args.model_name)(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L,feedback_bits=feedback_bits,n_class=n_class,Q=Q)
if args.gpu >= 0: model.cuda(args.gpu)

device = 0
save_path="./weights/";
load_path=save_path+"Sem_E2E_OMA"+"/";
dec_dict = model.state_dict()
model_name_pre = 'Sem_E2E_OMA'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)
dec_load_path=load_path+model_name_pre+"_model.pth.tar";
ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
pretrained_dict=ckpt['state_dict'];
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
dec_dict.update(pretrained_dict);
model.load_state_dict(dec_dict);
print('全部参数读取成功')

files = os.listdir('image')
images = []
fpath  = []
fpath_out = []
fpath_original = []
for file in files:
    if file[-3:] != 'png': continue
    fpath.append('image/'+file)
    fpath_out.append('seg_E2E_OMA/'+file)
    fpath_original.append('image_original/'+file)
    images.append( np.asarray(Image.open('image/'+file)) )
images = np.asarray(images, dtype=np.float32).transpose((0,3,1,2))/255.
images = Variable(torch.tensor(images))
if args.gpu >= 0: images = images.cuda(args.gpu)

model.eval()
with torch.no_grad():
    b = images.shape[0]
    alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
    H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j
    for i in range(T):
        t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
        H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
        H_batch[:,i] = H
    H_batch_1 = H_batch[0:b]
    H_batch_2 = H_batch[b:2*b]
    logits,power_signal = model(H_batch_1,H_batch_2, SNR_dB, images)
    predictions = logits.argmax(1)
    save_images(images, fpath_original)  #保存原始图像
    visualize(fpath_out, predictions)


##### 性能指标

In [ ]:
from util.util import calculate_accuracy, calculate_result
import sys
def test_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q):

    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]

    T = L+Q
    U = 2
    from util.util import calculate_accuracy, calculate_result
    import sys
    sys.argv = ['run.py']








    cf = np.zeros((n_class, n_class))


    parser = argparse.ArgumentParser(description='Test MFNet with pytorch')
    parser.add_argument('--model_name', '-M',  type=str, default='Sem_E2E_OMA')
    parser.add_argument('--batch_size',  '-B',  type=int, default=16)
    parser.add_argument('--gpu',        '-G',  type=int, default=0)
    parser.add_argument('--num_workers', '-j',  type=int, default=0)
    args = parser.parse_args()
    model_dir = 'weights/'
    model_dir = os.path.join(model_dir, args.model_name)

    model_name = 'Sem_E2E_OMA'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)

    final_model_file      = os.path.join(model_dir, model_name+"_model.pth.tar")
    # print('| running %s demo on GPU #%d with pytorch' % (args.model_name, args.gpu))




    model = Sem_E2E_OMA(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L,feedback_bits=feedback_bits,n_class=n_class,Q=Q).cuda()
    if args.gpu >= 0: model.cuda(args.gpu)

    device = 0
    save_path="./weights/";
    load_path=save_path+"Sem_E2E_OMA"+"/";
    dec_dict = model.state_dict()
    model_name_pre = 'Sem_E2E_OMA'+str(SNR_dB) + '_' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)+'_Q' + str(Q)
    dec_load_path=load_path+model_name_pre+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    model.load_state_dict(dec_dict);
    # print('全部参数读取成功')

    test_dataset  = MF_dataset(data_dir, 'test', have_label=True)
    test_loader  = DataLoader(
        dataset     = test_dataset,
        batch_size  = args.batch_size,
        shuffle     = False,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = False
    )
    test_loader.n_iter = len(test_loader)

    loss_avg = 0.
    acc_avg  = 0.
    model.eval()
    results_IOU = 0
    val_num = 5
    with torch.no_grad():
        for aa in range(val_num):
            cf = np.zeros((n_class, n_class))
            for it, (images, labels, names) in enumerate(test_loader):
                images = Variable(images)
                labels = Variable(labels)
                if args.gpu >= 0:
                    images = images.cuda(args.gpu)
                    labels = labels.cuda(args.gpu)

                b = images.shape[0]
                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j
                for i in range(T):
                    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]
                logits,power_signal = model(H_batch_1,H_batch_2, SNR_dB, images)
                loss = F.cross_entropy(logits, labels)
                acc = calculate_accuracy(logits, labels)
                loss_avg += float(loss)
                acc_avg  += float(acc)

                # print('|- test iter %s/%s. loss: %.4f, acc: %.4f' \
                #         % (it+1, test_loader.n_iter, float(loss), float(acc)))

                predictions = logits.argmax(1)
                for gtcid in range(n_class): 
                    for pcid in range(n_class):
                        gt_mask      = labels == gtcid 
                        pred_mask    = predictions == pcid
                        intersection = gt_mask * pred_mask
                        cf[gtcid, pcid] += int(intersection.sum())

            overall_acc, acc, IoU = calculate_result(cf)
            results_IOU += IoU.mean()/val_num

    # print('| overall accuracy:', overall_acc)
    # print('| accuracy of each class:', acc)
    # print('| class accuracy avg:', acc.mean())
    # print('| IoU:', IoU)
    print('| class IoU avg:', results_IOU)



In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for L in [1,2,4,8]:
    test_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 1

for L in [1,2,4,8]:
    test_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for feedback_bits in [4,16,64,256,1024]:
    test_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 1
for feedback_bits in [4,16,64,256,1024]:
    test_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for Q in [1,2,3,4,5]:
    test_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
SNR_dB_pre = None
v_max_km_pre = None
Q_pre = None

NOMA_pre=True
Phy_pre=True

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q = 4
for SNR_dB in [-40,-35,-30,-25,-20,-10]:
    test_Sem_E2E_OMA(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q)

## 基线（分离设计）

### 插值(仅验证)

In [ ]:
### 时域和频域插值外推
def interpolate_upsample(x, scale_factors, mode='bilinear', align_corners=False):
    """
    使用双线性插值对张量 x 在时间和频域维度上进行上采样。
    
    参数:
        x (torch.Tensor): 输入张量，形状为 [b, T, Nc, N_r, N_t]。
        scale_factors (tuple): 时间和频域的上采样因子 (scale_time, scale_freq)。
        mode (str): 插值模式，默认为 'bilinear'。
        align_corners (bool): 插值参数，默认为 False。
    
    返回:
        x_up (torch.Tensor): 上采样后的张量，形状为 [b, T*scale_time, Nc*scale_freq, N_r, N_t]。
    """
    scale_time, scale_freq = scale_factors
    
    # Reshape to [b * N_r * N_t, 1, T, Nc] for 2D interpolation
    b, T, Nc, N_r, N_t = x.shape
    x_reshaped = x.permute(0, 3, 4, 1, 2).contiguous().view(b * N_r * N_t, 1, T, Nc)
    
    # Perform 2D interpolation on time and frequency dimensions
    x_up = F.interpolate(x_reshaped, scale_factor=(scale_time, scale_freq), mode=mode, align_corners=align_corners)
    
    # Reshape back to [b, T*scale_time, Nc*scale_freq, N_r, N_t]
    b_new, _, T_up, Nc_up = x_up.shape
    x_up = x_up.view(b, N_r, N_t, T_up, Nc_up).permute(0, 3, 4, 1, 2).contiguous()
    
    return x_up
def Subcar_Interp(H1, H2, rate1, rate2):
    """
    对下采样后的信道 H1 和 H2 进行时间域和频域的插值上采样，恢复到完整的维度。
    
    参数:
        H1 (torch.Tensor): 下采样后的信道矩阵，形状为 [b, T_down, Nc_down, N_r, N_t]，复数类型。
        H2 (torch.Tensor): 下采样后的信道矩阵，形状为 [b, T_down, Nc_down, N_r, N_t]，复数类型。
        rate1 (int): 时间域上采样因子（多普勒域）。
        rate2 (int): 频域上采样因子（延时域）。
    
    返回:
        H1_up (torch.Tensor): 上采样后的 H1，形状为 [b, T*rate1, Nc*rate2, N_r, N_t]。
        H2_up_freq_shifted (torch.Tensor): 上采样并频率偏移后的 H2，形状为 [b, T*rate1, Nc*rate2, N_r, N_t]。
    """
    # 分离实部和虚部进行插值
    H1_real = H1.real
    H1_imag = H1.imag
    H2_real = H2.real
    H2_imag = H2.imag
    
    # Perform interpolation on real and imaginary parts
    H1_up_real = interpolate_upsample(H1_real, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H1_up_imag = interpolate_upsample(H1_imag, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H2_up_real = interpolate_upsample(H2_real, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H2_up_imag = interpolate_upsample(H2_imag, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    
    # Reconstruct complex tensors
    H1_up = torch.complex(H1_up_real, H1_up_imag)
    H2_up = torch.complex(H2_up_real, H2_up_imag)
    
    # Frequency shift for H2_up
    H1_up = torch.roll(H1_up, shifts=-rate2//2, dims=2)  # Shift along frequency dimension
    
    return H1_up, H2_up

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2
SNR_dB = -25
v_max_km = 120
T = 12
b=16
N_t = Nt[0]*Nt[1]
N_r = Nr[0]*Nr[1]
alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

for i in range(T):
    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
    H_batch[:,i] = H
H_batch_1 = H_batch[0:b]
H_batch_2 = H_batch[b:2*b]


rate1 = 4
rate2 = 2
H_1 = H_batch_1[:, ::rate1, ::rate2, :, :]
H_2 = H_batch_2[:, ::rate1, rate2//2::rate2, :, :]
H1_up, H2_up = Subcar_Interp(H_1, H_2, rate1, rate2)
print(NMSE(H1_up,H_batch_1))
print(NMSE(H2_up,H_batch_2))


### 预编码和合并(仅验证)

In [ ]:
import torch

def Pre_PCA(H1, H2):
    """
    MIMO预编码函数 Pre_PCA

    参数:
        H1: 张量，形状为 [b, T, Nc, N_r, N_t]，用户1到基站的信道矩阵
        H2: 张量，形状为 [b, T, Nc, N_r, N_t]，用户2到基站的信道矩阵

    返回:
        {
            'P_analog1': 用户1模拟预编码器，形状 [b, T, 1, N_t, 2],
            'P_analog2': 用户2模拟预编码器，形状 [b, T, 1, N_t, 2],
            'C_analog': 基站端模拟合并器，形状 [b, T, 1, 2, N_r],
            'P_digital1': 用户1数字预编码器，形状 [b, T, Nc, 2, 1],
            'P_digital2': 用户2数字预编码器, 形状 [b, T, Nc, 2, 1],
            'C_digital': 基站端数字合并器，形状 [b, T, Nc, 2, 2]
        }
    """
    # 获取输入信道矩阵的尺寸信息
    b, T, Nc, N_r, N_t = H1.shape
    N_RF = 2  # 射频链路数

    def compute_user_analog_precoder(H):
        """
        计算用户端的模拟预编码器

        输入:
            H: 张量，形状为 [b, T, Nc, N_r, N_t]

        输出:
            P_analog: 张量，形状为 [b, T, 1, N_t, 2]
        """
        # 将子载波和接收天线维度展平
        H_reshaped = H.view(b, T, Nc * N_r, N_t)  # [b, T, Nc*N_r, N_t]

        # 对展平后的信道矩阵进行奇异值分解（SVD）
        U, S, Vh = torch.linalg.svd(H_reshaped, full_matrices=False)  # Vh: [b, T, Nc*N_r, N_t]

        # 取右奇异矩阵的共轭转置，得到 V 矩阵
        V = Vh.conj().transpose(-2, -1)  # [b, T, N_t, Nc*N_r]

        # 取前两个右奇异向量作为模拟预编码器
        P_analog = V[:, :, :, :N_RF]  # [b, T, N_t, 2]

        # 恒模约束：仅保留相位信息
        P_analog = P_analog / (P_analog.abs() + 1e-8)  # 避免除零

        # 调整维度以匹配输出形状 [b, T, 1, N_t, 2]
        P_analog = P_analog.unsqueeze(2)  # [b, T, 1, N_t, 2]

        return P_analog

    # 计算两个用户的模拟预编码器
    P_analog1 = compute_user_analog_precoder(H1)  # 用户1的模拟预编码器 [b, T, 1, N_t, 2]
    P_analog2 = compute_user_analog_precoder(H2)  # 用户2的模拟预编码器 [b, T, 1, N_t, 2]

    def compute_base_analog_combiner(H):
        """
        计算基站端的模拟合并器

        输入:
            H: 张量，形状为 [b, T, Nc, N_r, N_t]

        输出:
            C_analog_i: 张量，形状为 [b, T, 1, 1, N_r]
        """
        # 置换维度，确保子载波和发送天线维度相邻
        H_permuted = H.permute(0, 1, 2, 4, 3).contiguous()  # [b, T, Nc, N_t, N_r]

        # 将子载波和发送天线维度展平
        H_reshaped = H_permuted.view(b, T, Nc * N_t, N_r)  # [b, T, Nc*N_t, N_r]

        # 对展平后的信道矩阵进行奇异值分解（SVD）
        U, S, Vh = torch.linalg.svd(H_reshaped, full_matrices=False)  # Vh: [b, T, Nc*N_t, N_r]

        # 取右奇异矩阵的共轭转置，得到 V 矩阵
        V = Vh.conj().transpose(-2, -1)  # [b, T, N_r, Nc*N_t]

        # 取前一个右奇异向量作为模拟合并器
        C_analog_i = V[:, :, :, :1]  # [b, T, N_r, 1]

        # 恒模约束：仅保留相位信息
        C_analog_i = C_analog_i / (C_analog_i.abs() + 1e-8)  # 避免除零

        # 调整维度以匹配输出形状 [b, T, 1, 1, N_r]
        C_analog_i = C_analog_i.unsqueeze(2).permute(0, 1, 2, 4, 3)  # [b, T, 1, 1, N_r]

        return C_analog_i

    # 计算两个用户的模拟合并器
    C_analog1 = compute_base_analog_combiner(H1)  # 用户1的模拟合并器 [b, T, 1, 1, N_r]
    C_analog2 = compute_base_analog_combiner(H2)  # 用户2的模拟合并器 [b, T, 1, 1, N_r]

    # 将两个用户的模拟合并器拼接到基站端的2个射频链路
    C_analog = torch.cat([C_analog1, C_analog2], dim=3)  # [b, T, 1, 2, N_r]

    def compute_digital_precoder(H, P_analog):
        """
        计算用户端的数字预编码器

        输入:
            H: 张量，形状为 [b, T, Nc, N_r, N_t]
            P_analog: 张量，形状为 [b, T, 1, N_t, 2]

        输出:
            P_digital: 张量，形状为 [b, T, Nc, 2, 1]
        """
        # 计算 H * P_analog -> [b, T, Nc, N_r, 2]
        H_P = torch.matmul(H, P_analog)  # [b, T, Nc, N_r, 2]

        # 将 H_P 重塑为 [b*T*Nc, N_r, 2] 以进行批量 SVD
        H_P_reshaped = H_P.view(-1, H_P.shape[-2], H_P.shape[-1])  # [b*T*Nc, N_r, 2]

        # 对每个子载波的等效信道进行奇异值分解（SVD）
        U, S, Vh = torch.linalg.svd(H_P_reshaped, full_matrices=False)  # Vh: [b*T*Nc, 2, 2]

        # 取前一个右奇异向量作为数字预编码器
        P_digital = Vh[:, :1, :].conj().transpose(-2, -1)  # [b*T*Nc, 2, 1]

        # 恢复到原始形状 [b, T, Nc, 2, 1]
        P_digital = P_digital.view(b, T, Nc, 2, 1)  # [b, T, Nc, 2, 1]

        return P_digital  # [b, T, Nc, 2, 1]

    # 计算两个用户的数字预编码器
    P_digital1 = compute_digital_precoder(H1, P_analog1)  # 用户1数字预编码器 [b, T, Nc, 2, 1]
    P_digital2 = compute_digital_precoder(H2, P_analog2)  # 用户2数字预编码器 [b, T, Nc, 2, 1]

    def compute_digital_combiner(C_analog, H, P_analog, P_digital):
        """
        计算数字合并器的等效信道

        输入:
            C_analog: 张量，形状为 [b, T, 1, 2, N_r]
            H: 张量，形状为 [b, T, Nc, N_r, N_t]
            P_analog: 张量，形状为 [b, T, 1, N_t, 2]
            P_digital: 张量，形状为 [b, T, Nc, 2, 1]

        输出:
            H_eff: 张量，形状为 [b, T, Nc, 2, 1]
        """
        # 计算 H * P_analog -> [b, T, Nc, N_r, 2]
        H_eff = C_analog @ H @ P_analog @ P_digital  # [b, T, Nc, 2, 1]


        return H_eff  # [b, T, Nc, 2, 1]

    # 计算两个用户的等效信道
    H_eff1 = compute_digital_combiner(C_analog, H1, P_analog1, P_digital1)  # 用户1等效信道 [b, T, Nc, 2, 1]
    H_eff2 = compute_digital_combiner(C_analog, H2, P_analog2, P_digital2)  # 用户2等效信道 [b, T, Nc, 2, 1]

    # 拼接两个用户的等效信道，形成 [b, T, Nc, 2, 2]
    H_eff_stack = torch.cat([H_eff1, H_eff2], dim=-1)  # [b, T, Nc, 2, 2]

    def compute_zf_digital_combiner(H_eff_stack):
        """
        计算基站端的数字合并器，使用零迫（ZF）方法

        输入:
            H_eff_stack: 张量，形状为 [b, T, Nc, 2, 2]

        输出:
            C_digital: 张量，形状为 [b, T, Nc, 2, 2]
        """
        # 计算数字合并器 W = (H_eff^H * H_eff)^{-1} * H_eff^H
        # 首先计算 H_eff^H
        H_eff_H = H_eff_stack.conj().transpose(-2, -1)  # [b, T, Nc, 2, 2]

        # 计算 H_eff^H * H_eff
        H_eff_H_H_eff = torch.matmul(H_eff_H, H_eff_stack)  # [b, T, Nc, 2, 2]

        # 计算 (H_eff^H * H_eff)^{-1}
        H_eff_H_H_eff_inv = torch.linalg.inv(H_eff_H_H_eff)  # [b, T, Nc, 2, 2]

        # 计算 W = (H_eff^H * H_eff)^{-1} * H_eff^H
        C_digital = torch.matmul(H_eff_H_H_eff_inv, H_eff_H)  # [b, T, Nc, 2, 2]

        return C_digital  # [b, T, Nc, 2, 2]

    # 计算基站端数字合并器
    C_digital = compute_zf_digital_combiner(H_eff_stack)  # [b, T, Nc, 2, 2]

    # 最终输出
    return P_analog1,P_analog2,C_analog,P_digital1,P_digital2,C_digital




In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2
SNR_dB = -25
v_max_km = 120
T = 1
b=16
N_t = Nt[0]*Nt[1]
N_r = Nr[0]*Nr[1]
alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

for i in range(T):
    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
    H_batch[:,i] = H
H_batch_1 = H_batch[0:b]
H_batch_2 = H_batch[b:2*b]

P_analog1,P_analog2,C_analog,P_digital1,P_digital2,C_digital = Pre_PCA(H_batch_1, H_batch_2)

# 打印输出形状以验证
print("用户1端模拟预编码器 P_analog1 的形状:", P_analog1.shape)    # [b, T, 1, N_t, 2]
print("用户2端模拟预编码器 P_analog2 的形状:", P_analog2.shape)    # [b, T, 1, N_t, 2]
print("基站端模拟合并器 C_analog 的形状:", C_analog.shape)           # [b, T, 1, 2, N_r]
print("用户1端数字预编码器 P_digital1 的形状:", P_digital1.shape)  # [b, T, Nc, 2, 1]
print("用户2端数字预编码器 P_digital2 的形状:", P_digital2.shape)  # [b, T, Nc, 2, 1]
print("基站端数字合并器 C_digital 的形状:", C_digital.shape)         # [b, T, Nc, 2, 2]

SNR_dB = 10
L = 0
F_1 = P_analog1 @ P_digital1
F_2 = P_analog2 @ P_digital2
W = C_digital @ C_analog
# W = torch.randn(b, T, Nc, 2, N_r).cuda() + 1j*torch.randn(b, T, Nc, 2, N_r).cuda()
R1 = calculate_average_channel_capacity(H_batch_1[:,L], F_1[:,L], W[:,L], SNR_dB)
R2 = calculate_average_channel_capacity(H_batch_2[:,L], F_2[:,L], W[:,L], SNR_dB)
print("R1:", R1.mean(),";  R2:", R2.mean())


### 物理层预编码合并插值外推函数完整版

In [ ]:
import torch
### 时域和频域插值外推
def interpolate_upsample(x, scale_factors, mode='bilinear', align_corners=False):
    """
    使用双线性插值对张量 x 在时间和频域维度上进行上采样。
    
    参数:
        x (torch.Tensor): 输入张量，形状为 [b, T, Nc, N_r, N_t]。
        scale_factors (tuple): 时间和频域的上采样因子 (scale_time, scale_freq)。
        mode (str): 插值模式，默认为 'bilinear'。
        align_corners (bool): 插值参数，默认为 False。
    
    返回:
        x_up (torch.Tensor): 上采样后的张量，形状为 [b, T*scale_time, Nc*scale_freq, N_r, N_t]。
    """
    scale_time, scale_freq = scale_factors
    
    # Reshape to [b * N_r * N_t, 1, T, Nc] for 2D interpolation
    b, T, Nc, N_r, N_t = x.shape
    x_reshaped = x.permute(0, 3, 4, 1, 2).contiguous().view(b * N_r * N_t, 1, T, Nc)
    
    # Perform 2D interpolation on time and frequency dimensions
    x_up = F.interpolate(x_reshaped, scale_factor=(scale_time, scale_freq), mode=mode, align_corners=align_corners)
    
    # Reshape back to [b, T*scale_time, Nc*scale_freq, N_r, N_t]
    b_new, _, T_up, Nc_up = x_up.shape
    x_up = x_up.view(b, N_r, N_t, T_up, Nc_up).permute(0, 3, 4, 1, 2).contiguous()
    
    return x_up
def Subcar_Interp(H1, H2, rate1, rate2):
    """
    对下采样后的信道 H1 和 H2 进行时间域和频域的插值上采样，恢复到完整的维度。
    
    参数:
        H1 (torch.Tensor): 下采样后的信道矩阵，形状为 [b, T_down, Nc_down, N_r, N_t]，复数类型。
        H2 (torch.Tensor): 下采样后的信道矩阵，形状为 [b, T_down, Nc_down, N_r, N_t]，复数类型。
        rate1 (int): 时间域上采样因子（多普勒域）。
        rate2 (int): 频域上采样因子（延时域）。
    
    返回:
        H1_up (torch.Tensor): 上采样后的 H1，形状为 [b, T*rate1, Nc*rate2, N_r, N_t]。
        H2_up_freq_shifted (torch.Tensor): 上采样并频率偏移后的 H2，形状为 [b, T*rate1, Nc*rate2, N_r, N_t]。
    """
    # 分离实部和虚部进行插值
    H1_real = H1.real
    H1_imag = H1.imag
    H2_real = H2.real
    H2_imag = H2.imag
    
    # Perform interpolation on real and imaginary parts
    H1_up_real = interpolate_upsample(H1_real, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H1_up_imag = interpolate_upsample(H1_imag, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H2_up_real = interpolate_upsample(H2_real, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H2_up_imag = interpolate_upsample(H2_imag, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    
    # Reconstruct complex tensors
    H1_up = torch.complex(H1_up_real, H1_up_imag)
    H2_up = torch.complex(H2_up_real, H2_up_imag)
    
    # Frequency shift for H2_up
    H1_up = torch.roll(H1_up, shifts=-rate2//2, dims=2)  # Shift along frequency dimension
    
    return H1_up, H2_up
def Pre_PCA_total(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2):
    """
    MIMO预编码函数 Pre_PCA

    参数:
        H1: 张量，形状为 [b, Q, Nc, N_r, N_t]，用户1到基站的信道矩阵
        H2: 张量，形状为 [b, Q, Nc, N_r, N_t]，用户2到基站的信道矩阵
        H1_UE: 张量，形状为 [b, 1, Nc, N_r, N_t]，用户端信道估计的信道 用户1到基站的信道矩阵
        H2_UE: 张量，形状为 [b, 1, Nc, N_r, N_t]，用户端信道估计的信道 用户2到基站的信道矩阵
        H1_base: 张量，形状为 [b, 1, Nc, N_r, N_t]，基站端CSI反馈重建的信道 用户1到基站的信道矩阵
        H2_base: 张量，形状为 [b, 1, Nc, N_r, N_t]，基站端CSI反馈重建的信道 用户2到基站的信道矩阵
    返回:
        {
            'P_analog1': 用户1模拟预编码器，形状 [b, T, 1, N_t, 2],
            'P_analog2': 用户2模拟预编码器，形状 [b, T, 1, N_t, 2],
            'C_analog': 基站端模拟合并器，形状 [b, T, 1, 2, N_r],
            'P_digital1': 用户1数字预编码器，形状 [b, T, Nc, 2, 1],
            'P_digital2': 用户2数字预编码器, 形状 [b, T, Nc, 2, 1],
            'C_digital': 基站端数字合并器，形状 [b, T, Nc, 2, 2]
        }
    """
    # 获取输入信道矩阵的尺寸信息
    b, Q, Nc, N_r, N_t = H1.shape
    N_RF = 2  # 射频链路数

    def compute_user_analog_precoder(H):
        T = 1
        """
        计算用户端的模拟预编码器

        输入:
            H: 张量，形状为 [b, T, Nc, N_r, N_t]

        输出:
            P_analog: 张量，形状为 [b, T, 1, N_t, 2]
        """
        # 将子载波和接收天线维度展平
        H_reshaped = H.view(b, T, Nc * N_r, N_t)  # [b, T, Nc*N_r, N_t]

        # 对展平后的信道矩阵进行奇异值分解（SVD）
        U, S, Vh = torch.linalg.svd(H_reshaped, full_matrices=False)  # Vh: [b, T, Nc*N_r, N_t]

        # 取右奇异矩阵的共轭转置，得到 V 矩阵
        V = Vh.conj().transpose(-2, -1)  # [b, T, N_t, Nc*N_r]

        # 取前两个右奇异向量作为模拟预编码器
        P_analog = V[:, :, :, :N_RF]  # [b, T, N_t, 2]

        # 恒模约束：仅保留相位信息
        P_analog = P_analog / (P_analog.abs() + 1e-8)  # 避免除零

        # 调整维度以匹配输出形状 [b, T, 1, N_t, 2]
        P_analog = P_analog.unsqueeze(2)  # [b, T, 1, N_t, 2]

        return P_analog

    # 计算两个用户的模拟预编码器
    P_analog1 = compute_user_analog_precoder(H1_UE)  # 用户1的模拟预编码器 [b, 1, 1, N_t, 2]
    P_analog2 = compute_user_analog_precoder(H2_UE)  # 用户2的模拟预编码器 [b, 1, 1, N_t, 2]

    def compute_base_analog_combiner(H):
        T = 1
        """
        计算基站端的模拟合并器

        输入:
            H: 张量，形状为 [b, T, Nc, N_r, N_t]

        输出:
            C_analog_i: 张量，形状为 [b, T, 1, 1, N_r]
        """
        # 置换维度，确保子载波和发送天线维度相邻
        H_permuted = H.permute(0, 1, 2, 4, 3).contiguous()  # [b, T, Nc, N_t, N_r]

        # 将子载波和发送天线维度展平
        H_reshaped = H_permuted.view(b, T, Nc * N_t, N_r)  # [b, T, Nc*N_t, N_r]

        # 对展平后的信道矩阵进行奇异值分解（SVD）
        U, S, Vh = torch.linalg.svd(H_reshaped, full_matrices=False)  # Vh: [b, T, Nc*N_t, N_r]

        # 取右奇异矩阵的共轭转置，得到 V 矩阵
        V = Vh.conj().transpose(-2, -1)  # [b, T, N_r, Nc*N_t]

        # 取前一个右奇异向量作为模拟合并器
        C_analog_i = V[:, :, :, :1]  # [b, T, N_r, 1]

        # 恒模约束：仅保留相位信息
        C_analog_i = C_analog_i / (C_analog_i.abs() + 1e-8)  # 避免除零

        # 调整维度以匹配输出形状 [b, T, 1, 1, N_r]
        C_analog_i = C_analog_i.unsqueeze(2).permute(0, 1, 2, 4, 3)  # [b, T, 1, 1, N_r]

        return C_analog_i

    # 计算两个用户的模拟合并器
    C_analog1 = compute_base_analog_combiner(H1_base)  # 用户1的模拟合并器 [b, 1, 1, 1, N_r]
    C_analog2 = compute_base_analog_combiner(H2_base)  # 用户2的模拟合并器 [b, 1, 1, 1, N_r]

    # 将两个用户的模拟合并器拼接到基站端的2个射频链路
    C_analog = torch.cat([C_analog1, C_analog2], dim=3)  # [b, 1, 1, 2, N_r]

    def compute_digital_precoder(H, P_analog):
        T = 1
        """
        计算用户端的数字预编码器

        输入:
            H: 张量，形状为 [b, T, Nc, N_r, N_t]
            P_analog: 张量，形状为 [b, T, 1, N_t, 2]

        输出:
            P_digital: 张量，形状为 [b, T, Nc, 2, 1]
        """
        # 计算 H * P_analog -> [b, T, Nc, N_r, 2]
        H_P = torch.matmul(H, P_analog)  # [b, T, Nc, N_r, 2]

        # 将 H_P 重塑为 [b*T*Nc, N_r, 2] 以进行批量 SVD
        H_P_reshaped = H_P.view(-1, H_P.shape[-2], H_P.shape[-1])  # [b*T*Nc, N_r, 2]

        # 对每个子载波的等效信道进行奇异值分解（SVD）
        U, S, Vh = torch.linalg.svd(H_P_reshaped, full_matrices=False)  # Vh: [b*T*Nc, 2, 2]

        # 取前一个右奇异向量作为数字预编码器
        P_digital = Vh[:, :1, :].conj().transpose(-2, -1)  # [b*T*Nc, 2, 1]

        # 恢复到原始形状 [b, T, Nc, 2, 1]
        P_digital = P_digital.view(b, T, Nc, 2, 1)  # [b, T, Nc, 2, 1]

        return P_digital  # [b, T, Nc, 2, 1]

    # 计算两个用户的数字预编码器
    P_digital1 = compute_digital_precoder(H1_UE, P_analog1)  # 用户1数字预编码器 [b, 1, Nc, 2, 1]
    P_digital2 = compute_digital_precoder(H2_UE, P_analog2)  # 用户2数字预编码器 [b, 1, Nc, 2, 1]

    def compute_digital_combiner(C_analog, H, F):
        """
        计算数字合并器的等效信道,实际在抽取之后就是接收DMRS导频（已加噪声）

        输入:
            C_analog: 张量，形状为 [b, 1, 1, 2, N_r]
            H: 张量，形状为 [b, Q, Nc, N_r, N_t]
            P_analog: 张量，形状为 [b, 1, 1, N_t, 2]
            P_digital: 张量，形状为 [b, 1, Nc, 2, 1]

        输出:
            H_eff: 张量，形状为 [b, T, Nc, 2, 1]
        """
       

        
        norm_factor = torch.sqrt(torch.sum(torch.abs(F)**2, dim=(-2, -1), keepdim=True))
        F = F / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))  #[b, 1, 1, N_t, 1]

        norm_factor = torch.sqrt(torch.sum(torch.abs(C_analog)**2, dim=(-2, -1), keepdim=True))
        W = C_analog / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))

        # Calculate effective channel
        sigma = 10**(-SNR_dB/10)
        n = (torch.randn(b,Q,Nc,N_RF,1).cuda() + 1j*torch.randn(b,Q,Nc,N_RF,1).cuda())/sqrt(2)*sqrt(sigma)
        H_eff = W @ H @ F + n  # [b, Q, Nc, 2, 1]
        # H_eff = W @ H @ F   # [b, Q, Nc, 2, 1]


        return H_eff  # [b, Q, Nc, 2, 1]

    # 计算两个用户的等效信道
    F_1 = P_analog1 @ P_digital1
    F_2 = P_analog2 @ P_digital2

    norm_factor = torch.sqrt(torch.sum(torch.abs(F_1)**2, dim=(-2, -1), keepdim=True))
    F_1 = F_1 / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))
    norm_factor = torch.sqrt(torch.sum(torch.abs(F_2)**2, dim=(-2, -1), keepdim=True))
    F_2 = F_2 / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))
    norm_factor = torch.sqrt(torch.sum(torch.abs(C_analog)**2, dim=(-2, -1), keepdim=True))
    C_analog = C_analog / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))

    H_eff1 = compute_digital_combiner(C_analog, H1, F_1)  # 用户1等效信道 [b, Q, Nc, 2, 1]
    H_eff2 = compute_digital_combiner(C_analog, H2, F_2)  # 用户2等效信道 [b, Q, Nc, 2, 1]

    H_eff1_down = H_eff1[:, ::rate1, ::rate2, :, :]
    H_eff2_down = H_eff2[:, ::rate1, rate2//2::rate2, :, :]
    H_eff1_up, H_eff2_up = Subcar_Interp(H_eff1_down, H_eff2_down, rate1, rate2)

    # print(NMSE(H_eff1_up,H_eff1))
    # print(NMSE(H_eff2_up,H_eff2))

    # 拼接两个用户的等效信道，形成 [b, T, Nc, 2, 2]
    H_eff_stack = torch.cat([H_eff1_up, H_eff2_up], dim=-1)  # [b, Q, Nc, 2, 2]

    def compute_zf_digital_combiner(H_eff_stack):
        T = Q
        """
        计算基站端的数字合并器，使用零迫（ZF）方法

        输入:
            H_eff_stack: 张量，形状为 [b, T, Nc, 2, 2]

        输出:
            C_digital: 张量，形状为 [b, T, Nc, 2, 2]
        """
        # 计算数字合并器 W = (H_eff^H * H_eff)^{-1} * H_eff^H
        # 首先计算 H_eff^H
        H_eff_H = H_eff_stack.conj().transpose(-2, -1)  # [b, T, Nc, 2, 2]

        # 计算 H_eff^H * H_eff
        H_eff_H_H_eff = torch.matmul(H_eff_H, H_eff_stack)  # [b, T, Nc, 2, 2]

        # 计算 (H_eff^H * H_eff)^{-1}
        H_eff_H_H_eff_inv = torch.linalg.inv(H_eff_H_H_eff)  # [b, T, Nc, 2, 2]

        # 计算 W = (H_eff^H * H_eff)^{-1} * H_eff^H
        C_digital = torch.matmul(H_eff_H_H_eff_inv, H_eff_H)  # [b, T, Nc, 2, 2]

        return C_digital  # [b, T, Nc, 2, 2]

    # 计算基站端数字合并器
    C_digital = compute_zf_digital_combiner(H_eff_stack)  # [b, Q, Nc, 2, 2]

    W = C_digital @ C_analog



    # 最终输出
    return P_analog1,P_analog2,C_analog,P_digital1,P_digital2,C_digital, F_1, F_2, W

def Pre_PCA_total_analog(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2):
    """
    MIMO预编码函数 Pre_PCA

    参数:
        H1: 张量，形状为 [b, Q, Nc, N_r, N_t]，用户1到基站的信道矩阵
        H2: 张量，形状为 [b, Q, Nc, N_r, N_t]，用户2到基站的信道矩阵
        H1_UE: 张量，形状为 [b, 1, Nc, N_r, N_t]，用户端信道估计的信道 用户1到基站的信道矩阵
        H2_UE: 张量，形状为 [b, 1, Nc, N_r, N_t]，用户端信道估计的信道 用户2到基站的信道矩阵
        H1_base: 张量，形状为 [b, 1, Nc, N_r, N_t]，基站端CSI反馈重建的信道 用户1到基站的信道矩阵
        H2_base: 张量，形状为 [b, 1, Nc, N_r, N_t]，基站端CSI反馈重建的信道 用户2到基站的信道矩阵
    返回:
        {
            'P_analog1': 用户1模拟预编码器，形状 [b, T, 1, N_t, 2],
            'P_analog2': 用户2模拟预编码器，形状 [b, T, 1, N_t, 2],
            'C_analog': 基站端模拟合并器，形状 [b, T, 1, 2, N_r],
            'P_digital1': 用户1数字预编码器，形状 [b, T, Nc, 2, 1],
            'P_digital2': 用户2数字预编码器, 形状 [b, T, Nc, 2, 1],
            'C_digital': 基站端数字合并器，形状 [b, T, Nc, 2, 2]
        }
    """
    # 获取输入信道矩阵的尺寸信息
    b, Q, Nc, N_r, N_t = H1.shape
    N_RF = 2  # 射频链路数

    def compute_user_analog_precoder(H):
        T = 1
        """
        计算用户端的模拟预编码器

        输入:
            H: 张量，形状为 [b, T, Nc, N_r, N_t]

        输出:
            P_analog: 张量，形状为 [b, T, 1, N_t, 2]
        """
        # 将子载波和接收天线维度展平
        H_reshaped = H.view(b, T, Nc * N_r, N_t)  # [b, T, Nc*N_r, N_t]

        # 对展平后的信道矩阵进行奇异值分解（SVD）
        U, S, Vh = torch.linalg.svd(H_reshaped, full_matrices=False)  # Vh: [b, T, Nc*N_r, N_t]

        # 取右奇异矩阵的共轭转置，得到 V 矩阵
        V = Vh.conj().transpose(-2, -1)  # [b, T, N_t, Nc*N_r]

        # 取前1个右奇异向量作为模拟预编码器
        P_analog = V[:, :, :, :1]  # [b, T, N_t, 1]

        # 恒模约束：仅保留相位信息
        P_analog = P_analog / (P_analog.abs() + 1e-8)  # 避免除零

        # 调整维度以匹配输出形状 [b, T, 1, N_t, 1]
        P_analog = P_analog.unsqueeze(2)  # [b, T, 1, N_t, 1]

        return P_analog

    # 计算两个用户的模拟预编码器
    P_analog1 = compute_user_analog_precoder(H1_UE)  # 用户1的模拟预编码器 [b, 1, 1, N_t, 1]
    P_analog2 = compute_user_analog_precoder(H2_UE)  # 用户2的模拟预编码器 [b, 1, 1, N_t, 1]

    def compute_base_analog_combiner(H):
        T = 1
        """
        计算基站端的模拟合并器

        输入:
            H: 张量，形状为 [b, T, Nc, N_r, N_t]

        输出:
            C_analog_i: 张量，形状为 [b, T, 1, 1, N_r]
        """
        # 置换维度，确保子载波和发送天线维度相邻
        H_permuted = H.permute(0, 1, 2, 4, 3).contiguous()  # [b, T, Nc, N_t, N_r]

        # 将子载波和发送天线维度展平
        H_reshaped = H_permuted.view(b, T, Nc * N_t, N_r)  # [b, T, Nc*N_t, N_r]

        # 对展平后的信道矩阵进行奇异值分解（SVD）
        U, S, Vh = torch.linalg.svd(H_reshaped, full_matrices=False)  # Vh: [b, T, Nc*N_t, N_r]

        # 取右奇异矩阵的共轭转置，得到 V 矩阵
        V = Vh.conj().transpose(-2, -1)  # [b, T, N_r, Nc*N_t]

        # 取前一个右奇异向量作为模拟合并器
        C_analog_i = V[:, :, :, :1]  # [b, T, N_r, 1]

        # 恒模约束：仅保留相位信息
        C_analog_i = C_analog_i / (C_analog_i.abs() + 1e-8)  # 避免除零

        # 调整维度以匹配输出形状 [b, T, 1, 1, N_r]
        C_analog_i = C_analog_i.unsqueeze(2).permute(0, 1, 2, 4, 3)  # [b, T, 1, 1, N_r]

        return C_analog_i

    # 计算两个用户的模拟合并器
    C_analog1 = compute_base_analog_combiner(H1_base)  # 用户1的模拟合并器 [b, 1, 1, 1, N_r]
    C_analog2 = compute_base_analog_combiner(H2_base)  # 用户2的模拟合并器 [b, 1, 1, 1, N_r]

    # 将两个用户的模拟合并器拼接到基站端的2个射频链路
    C_analog = torch.cat([C_analog1, C_analog2], dim=3)  # [b, 1, 1, 2, N_r]

    # def compute_digital_precoder(H, P_analog):
    #     T = 1
    #     """
    #     计算用户端的数字预编码器

    #     输入:
    #         H: 张量，形状为 [b, T, Nc, N_r, N_t]
    #         P_analog: 张量，形状为 [b, T, 1, N_t, 2]

    #     输出:
    #         P_digital: 张量，形状为 [b, T, Nc, 2, 1]
    #     """
    #     # 计算 H * P_analog -> [b, T, Nc, N_r, 2]
    #     H_P = torch.matmul(H, P_analog)  # [b, T, Nc, N_r, 2]

    #     # 将 H_P 重塑为 [b*T*Nc, N_r, 2] 以进行批量 SVD
    #     H_P_reshaped = H_P.view(-1, H_P.shape[-2], H_P.shape[-1])  # [b*T*Nc, N_r, 2]

    #     # 对每个子载波的等效信道进行奇异值分解（SVD）
    #     U, S, Vh = torch.linalg.svd(H_P_reshaped, full_matrices=False)  # Vh: [b*T*Nc, 2, 2]

    #     # 取前一个右奇异向量作为数字预编码器
    #     P_digital = Vh[:, :1, :].conj().transpose(-2, -1)  # [b*T*Nc, 2, 1]

    #     # 恢复到原始形状 [b, T, Nc, 2, 1]
    #     P_digital = P_digital.view(b, T, Nc, 2, 1)  # [b, T, Nc, 2, 1]

    #     return P_digital  # [b, T, Nc, 2, 1]

    # # 计算两个用户的数字预编码器
    # P_digital1 = compute_digital_precoder(H1_UE, P_analog1)  # 用户1数字预编码器 [b, 1, Nc, 2, 1]
    # P_digital2 = compute_digital_precoder(H2_UE, P_analog2)  # 用户2数字预编码器 [b, 1, Nc, 2, 1]

    def compute_digital_combiner(C_analog, H, F):
        """
        计算数字合并器的等效信道,实际在抽取之后就是接收DMRS导频（已加噪声）

        输入:
            C_analog: 张量，形状为 [b, 1, 1, 2, N_r]
            H: 张量，形状为 [b, Q, Nc, N_r, N_t]
            P_analog: 张量，形状为 [b, 1, 1, N_t, 2]
            P_digital: 张量，形状为 [b, 1, Nc, 2, 1]

        输出:
            H_eff: 张量，形状为 [b, T, Nc, 2, 1]
        """
       

        
        norm_factor = torch.sqrt(torch.sum(torch.abs(F)**2, dim=(-2, -1), keepdim=True))
        F = F / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))  #[b, 1, 1, N_t, 1]

        norm_factor = torch.sqrt(torch.sum(torch.abs(C_analog)**2, dim=(-2, -1), keepdim=True))
        W = C_analog / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))

        # Calculate effective channel
        sigma = 10**(-SNR_dB/10)
        n = (torch.randn(b,Q,Nc,N_RF,1).cuda() + 1j*torch.randn(b,Q,Nc,N_RF,1).cuda())/sqrt(2)*sqrt(sigma)
        H_eff = W @ H @ F + n  # [b, Q, Nc, 2, 1]
        # H_eff = W @ H @ F   # [b, Q, Nc, 2, 1]


        return H_eff  # [b, Q, Nc, 2, 1]

    # 计算两个用户的等效信道
    F_1 = P_analog1
    F_2 = P_analog2

    norm_factor = torch.sqrt(torch.sum(torch.abs(F_1)**2, dim=(-2, -1), keepdim=True))
    F_1 = F_1 / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))
    norm_factor = torch.sqrt(torch.sum(torch.abs(F_2)**2, dim=(-2, -1), keepdim=True))
    F_2 = F_2 / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))
    norm_factor = torch.sqrt(torch.sum(torch.abs(C_analog)**2, dim=(-2, -1), keepdim=True))
    C_analog = C_analog / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))

    H_eff1 = compute_digital_combiner(C_analog, H1, F_1)  # 用户1等效信道 [b, Q, Nc, 2, 1]
    H_eff2 = compute_digital_combiner(C_analog, H2, F_2)  # 用户2等效信道 [b, Q, Nc, 2, 1]

    H_eff1_down = H_eff1[:, ::rate1, ::rate2, :, :]
    H_eff2_down = H_eff2[:, ::rate1, rate2//2::rate2, :, :]
    H_eff1_up, H_eff2_up = Subcar_Interp(H_eff1_down, H_eff2_down, rate1, rate2)

    # print(NMSE(H_eff1_up,H_eff1))
    # print(NMSE(H_eff2_up,H_eff2))

    # 拼接两个用户的等效信道，形成 [b, T, Nc, 2, 2]
    H_eff_stack = torch.cat([H_eff1_up, H_eff2_up], dim=-1)  # [b, Q, Nc, 2, 2]

    def compute_zf_digital_combiner(H_eff_stack):
        T = Q
        """
        计算基站端的数字合并器，使用零迫（ZF）方法

        输入:
            H_eff_stack: 张量，形状为 [b, T, Nc, 2, 2]

        输出:
            C_digital: 张量，形状为 [b, T, Nc, 2, 2]
        """
        # 计算数字合并器 W = (H_eff^H * H_eff)^{-1} * H_eff^H
        # 首先计算 H_eff^H
        H_eff_H = H_eff_stack.conj().transpose(-2, -1)  # [b, T, Nc, 2, 2]

        # 计算 H_eff^H * H_eff
        H_eff_H_H_eff = torch.matmul(H_eff_H, H_eff_stack)  # [b, T, Nc, 2, 2]

        # 计算 (H_eff^H * H_eff)^{-1}
        H_eff_H_H_eff_inv = torch.linalg.inv(H_eff_H_H_eff)  # [b, T, Nc, 2, 2]

        # 计算 W = (H_eff^H * H_eff)^{-1} * H_eff^H
        C_digital = torch.matmul(H_eff_H_H_eff_inv, H_eff_H)  # [b, T, Nc, 2, 2]

        return C_digital  # [b, T, Nc, 2, 2]

    # 计算基站端数字合并器
    C_digital = compute_zf_digital_combiner(H_eff_stack)  # [b, Q, Nc, 2, 2]

    W = C_digital @ C_analog



    # 最终输出
    return P_analog1,P_analog2,C_analog,C_digital, F_1, F_2, W[:,0:Q]


In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2
v_max_km = 120
L = 8
Q = 12
T = L+Q
b=16
N_t = Nt[0]*Nt[1]
N_r = Nr[0]*Nr[1]
alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

for i in range(T):
    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
    H_batch[:,i] = H
H_batch_1 = H_batch[0:b]
H_batch_2 = H_batch[b:2*b]

# rate1 (int): 时间域上DMRS采样因子。
# rate2 (int): 频域上DMRS采样因子。
rate1 = 4
rate2 = 2
SNR_dB = 10

H1 = H_batch_1[:,L:L+Q] 
H2 = H_batch_2[:,L:L+Q] 
H1_UE = H_batch_1[:,L:L+1]
H2_UE = H_batch_2[:,L:L+1]
H1_base = H_batch_1[:,L:L+1]
H2_base = H_batch_2[:,L:L+1]
P_analog1,P_analog2,C_analog,C_digital,F_1, F_2, W = Pre_PCA_total_analog(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2)

# 打印输出形状以验证
print("用户1端模拟预编码器 P_analog1 的形状:", P_analog1.shape)    # [b, T, 1, N_t, 2]
print("用户2端模拟预编码器 P_analog2 的形状:", P_analog2.shape)    # [b, T, 1, N_t, 2]
print("基站端模拟合并器 C_analog 的形状:", C_analog.shape)           # [b, T, 1, 2, N_r]
print("基站端数字合并器 C_digital 的形状:", C_digital.shape)         # [b, T, Nc, 2, 2]


# W = torch.randn(b, T, Nc, 2, N_r).cuda() + 1j*torch.randn(b, T, Nc, 2, N_r).cuda()
R1 = calculate_average_channel_capacity_T(H1, F_1, W, SNR_dB)
R2 = calculate_average_channel_capacity_T(H2, F_2, W, SNR_dB)
print("R1:", R1.mean().item(),";  R2:", R2.mean().item())


In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2
v_max_km = 120
L = 8
Q = 12
T = L+Q
b=16
N_t = Nt[0]*Nt[1]
N_r = Nr[0]*Nr[1]
alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

for i in range(T):
    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
    H_batch[:,i] = H
H_batch_1 = H_batch[0:b]
H_batch_2 = H_batch[b:2*b]

# rate1 (int): 时间域上DMRS采样因子。
# rate2 (int): 频域上DMRS采样因子。
rate1 = 4
rate2 = 2
SNR_dB = 10

H1 = H_batch_1[:,L:L+Q] 
H2 = H_batch_2[:,L:L+Q] 
H1_UE = H_batch_1[:,L:L+1]
H2_UE = H_batch_2[:,L:L+1]
H1_base = H_batch_1[:,L:L+1]
H2_base = H_batch_2[:,L:L+1]
P_analog1,P_analog2,C_analog,P_digital1,P_digital2,C_digital,F_1, F_2, W = Pre_PCA_total(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2)

# 打印输出形状以验证
print("用户1端模拟预编码器 P_analog1 的形状:", P_analog1.shape)    # [b, T, 1, N_t, 2]
print("用户2端模拟预编码器 P_analog2 的形状:", P_analog2.shape)    # [b, T, 1, N_t, 2]
print("基站端模拟合并器 C_analog 的形状:", C_analog.shape)           # [b, T, 1, 2, N_r]
print("用户1端数字预编码器 P_digital1 的形状:", P_digital1.shape)  # [b, T, Nc, 2, 1]
print("用户2端数字预编码器 P_digital2 的形状:", P_digital2.shape)  # [b, T, Nc, 2, 1]
print("基站端数字合并器 C_digital 的形状:", C_digital.shape)         # [b, T, Nc, 2, 2]


# W = torch.randn(b, T, Nc, 2, N_r).cuda() + 1j*torch.randn(b, T, Nc, 2, N_r).cuda()
R1 = calculate_average_channel_capacity_T(H1, F_1, W, SNR_dB)
R2 = calculate_average_channel_capacity_T(H2, F_2, W, SNR_dB)
print("R1:", R1.mean().item(),";  R2:", R2.mean().item())


### WY信道估计

#### 网络和训练函数

In [ ]:
from tqdm import tqdm
from tqdm import trange
from time import sleep
from my_log import logger, rmLogger

# 可训练的三个导频
class Pilot_trans_WY(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
        ):
        super().__init__();

        self.N_r = N_r
        self.L = L
        self.N_t = N_t
        self.N_t_RF = N_t_RF

        # self.p_UE_1 = nn.Parameter(torch.randn(1, Nc, L, N_t, 1) + 1j*torch.randn(1, Nc, L, N_t, 1))
        # self.p_UE_2 = nn.Parameter(torch.randn(1, Nc, L, N_t, 1) + 1j*torch.randn(1, Nc, L, N_t, 1))

        # self.p_UE_1 = nn.Parameter(torch.randn(1, 1, L, N_t, 1) + 1j*torch.randn(1, 1, L, N_t, 1))
        # self.p_UE_2 = nn.Parameter(torch.randn(1, 1, L, N_t, 1) + 1j*torch.randn(1, 1, L, N_t, 1))

        self.p_BS_RF = nn.Parameter(torch.randn(1, 1, L, N_r, N_r_RF) + 1j*torch.randn(1, 1, L, N_r, N_r_RF))
        self.p_BS_BB = nn.Parameter(torch.randn(1, Nc, L, N_r_RF, 1) + 1j*torch.randn(1, Nc, L, N_r_RF, 1))

        self.p_UE = nn.Parameter(torch.randn(1, 1, L, N_t_RF, N_t) + 1j*torch.randn(1, 1, L, N_t_RF, N_t))

    def forward(
            self, 
            H_batch,  # b,T, Nc, N_r, N_t
        ):


        b,T, Nc, N_r, N_t = H_batch.shape
        N_t_RF = self.N_t_RF
        L = self.L
        H_batch = H_batch[:,0:L].permute(0,2,1,4,3) # b, Nc, L, N_t, N_r


        p_UE = self.p_UE/torch.abs(self.p_UE)/sqrt(self.N_t)
        p_BS = self.p_BS_RF @ self.p_BS_BB





        received_pilot = p_UE @ (H_batch @ p_BS) # b, Nc, L, N_t_RF, 1


        norm_factor = torch.sqrt(torch.sum(torch.abs(received_pilot)**2, dim=(-4,-3,-2, -1), keepdim=True))
        received_pilot = received_pilot / norm_factor * torch.sqrt(torch.tensor(Nc*L*N_t_RF, dtype=received_pilot.dtype, device=received_pilot.device))



        return received_pilot   

class pilot2H_WY(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
        ):
        super().__init__();

        self.d_model = 1024
        self.N_t = N_t
        self.N_t_RF = N_t_RF
        self.N_r = N_r

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)

        self.embed= nn.Linear(2*N_t_RF*L, self.d_model) 
        self.positional_embedding=nn.Parameter(torch.zeros(1, Nc, self.d_model));

        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(6)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model

        self.head=nn.Sequential(  
            nn.Linear(self.d_model, 2*N_r*N_t),  
        );




    
    def forward(
            self, 
            received_pilot,  # b, Nc, L, N_t_RF, 1
        ):
        # b = H_DL_batch.shape[0]
        # H_DL_batch_1 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,0,:].conj() #这里加了conj是因为上下行信道之间有个共轭转置，所以共轭这部分需要conj回去消除掉
        # H_DL_batch_2 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,1,:].conj()

        # H_DL_batch_beam_1 = (H_DL_batch_1*np.exp(1j*pi/2 * 0) + H_DL_batch_2*np.exp(1j*pi/2 * 0)).reshape(b,1,48,4,256)

        b, Nc, L, N_t_RF, _ = received_pilot.shape
        N_t = self.N_t
        N_t_RF = self.N_t_RF
        N_r = self.N_r

        y_cat = received_pilot.reshape(b,Nc,L*N_t_RF) 
        y_cat=torch.cat([y_cat.real, y_cat.imag], dim=-1);  # b,Nc,2*L*N_t_RF

        x = self.embed(y_cat) + self.positional_embedding  # b,Nc,d_model

        x = self.backbone(x)   # b,Nc,d_model

        H = self.head(x)  # b,Nc,2*N_r*N_t
        H = H.reshape(b,Nc,N_r,N_t,2)
        H_hat = H[:,:,:,:,0] + 1j*H[:,:,:,:,1]


        return H_hat     #[b,Nc,N_r,N_t]


class CE_WY(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
        ):
        super().__init__();

        
        self.Pilot_trans = Pilot_trans_WY(Nc,N_r,N_r_RF,N_t,N_t_RF,L)
        self.pilot2H = pilot2H_WY(Nc,N_r,N_r_RF,N_t,N_t_RF,L)

       



    
    def forward(
            self, 
            H_batch,  # b,T, Nc, N_r, N_t 用户1
        ):
        received_pilot = self.Pilot_trans(H_batch)
        H_hat = self.pilot2H(received_pilot)
        return H_hat




In [ ]:
def SGCS_H(H1, H2):
    tx=H1.shape[-1];
    rx=H1.shape[-2]
    v1=H1.to(torch.complex64);
    v2=H2.to(torch.complex64);


    v1=v1.reshape(-1, 1, rx*tx);
    v2=v2.reshape(-1, 1, rx*tx);

    tmp12=(v1@v2.mH).abs();
    tmp11=(v1@v1.mH).abs();
    tmp22=(v2@v2.mH).abs();

    SGCS=tmp12.square()/tmp11/tmp22;

    return -SGCS.mean();

In [ ]:
def train_CE_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,SNR_dB,v_max_km,v_max_km_pre=None,ITERS=20_000,b=32,acc_num=1):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=10;

    model_name = 'CE_WY'+'_L' + str(L)+'_v' + str(v_max_km)

    dec=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
    
    opt_dec=torch.optim.AdamW(
        dec.parameters(), 
        lr=lr, 
    )
    T = L+1
    load_path=save_path+"CE_WY"+"/";
    if v_max_km_pre == None:
        print('从头训练')
    else:
        dec_dict = dec.state_dict()
        model_name_pre = 'CE_WY' + '_L' + str(L)+'_v' + str(v_max_km_pre)
        dec_load_path=load_path+model_name_pre+"_model.pth.tar";
        ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
        pretrained_dict=ckpt['state_dict'];
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict and ".quan" not in k and ".Dequan" not in k}
        dec_dict.update(pretrained_dict);
        dec.load_state_dict(dec_dict);
        print('全部预训练参数读取成功')

    def step(step_id):
        dec.train();
        opt_dec.zero_grad();
        loss_acc=0;

        


        

        for _ in range(acc_num):
            alpha,A_r,A_t,fd,tau = SV_Channel_set(b,Nc,N,Nt,Nr,1,v_max_km)
            H_batch = torch.zeros(b,T, Nc, N_r, N_t).cuda() + 0j

            for i in range(T):
                t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                H = SV_Channel_final(b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                H_batch[:,i] = H
            # H_batch_1 = H_batch[0:b]
            # H_batch_2 = H_batch[b:2*b]

            

            
            H_hat=dec(
                    H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                );
            # H_hat # b,Nc, N_r, N_t 用户1



            # loss_SE = -calculate_average_channel_capacity(H_DL_batch_beam[:,0:1], F, Pt,N0)/acc_num

            loss = SGCS_H(H_batch[:,L],H_hat)/acc_num
            loss_acc += loss.item()

            loss.backward();
        

        opt_dec.step();

        return loss_acc






    def evaluate(step_id):
        global loader_test;
        dec.eval();
        loss_acc=0;
        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8
                    H = SV_Channel_final(b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                # H_batch_1 = H_batch[0:b]
                # H_batch_2 = H_batch[b:2*b]


                
                H_hat=dec(
                        H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                    );
                loss = SGCS_H(H_batch[:,L],H_hat)/val_num
                loss_acc += loss.item()

        return loss_acc

    def save_ckpt(folder_name):
        folder_path=save_path+folder_name+"/";
        if not os.path.exists(folder_path):
            os.makedirs(folder_path);

        torch.save({'state_dict': dec.state_dict()}, folder_path+model_name+"_model.pth.tar");
        torch.save({'best_loss': best_loss, 
                    'opt_dec_dict': opt_dec.state_dict(), 
                    }, folder_path+model_name+"_opt.pth.tar");

    loss_SGCS_log=0;
    t1=time.time();




    with trange(iter_begin, iter_begin+ITERS) as t:
        for i in t:            
            loss_SGCS = step(i);
            loss_SGCS_log+=loss_SGCS/log_interval;

            if (i+1)%log_interval==0:
                loss_SGCS = evaluate(i);
                
                logger("");
                if loss_SGCS<best_loss:
                    best_loss=loss_SGCS;
                    save_ckpt("CE_WY");
                    logger("\nbest - SGCS = {}".format(best_loss));
                
                logger("log - iter[{}] - train - SGCS = {}".format(i, -loss_SGCS_log));
                logger("log - iter[{}] - test - SGCS = {}".format(i, -loss_SGCS));

                

                logger("time: {} s".format(time.time()-t1));
                t1=time.time();

                if (i+1)%save_interval==0:
                    logger("");
                    logger("save - iter[{}] - test - SGCS = {}".format(i, -loss_SGCS));
                    logger("save - iter[{}] - best - SGCS = {}".format(i, -best_loss));

                    

                t.set_postfix(train_SGCS=loss_SGCS_log,test_SGCS=loss_SGCS,best_SGCS=-best_loss)

                loss_SGCS_log=0;
    rmLogger();

In [ ]:
#这里测试频谱效率的时候假设完美CSI反馈
def test_CE_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,SNR_dB,v_max_km):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=10;

    model_name = 'CE_WY'+'_L' + str(L)+'_v' + str(v_max_km)

    dec=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
    
    
    load_path=save_path+"CE_WY"+"/";

    dec_dict = dec.state_dict()
    dec_load_path=load_path+model_name+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec.load_state_dict(dec_dict);

    Q = 12
    T = L+Q
    rate1 = 4
    rate2 = 2

    






    def evaluate(step_id):
        global loader_test;
        dec.eval();
        loss_acc=0;
        R_acc = 0
        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]


                
                H_hat=dec(
                        H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                    );
                loss = SGCS_H(H_batch[:,L],H_hat)/val_num
                loss_acc += loss.item()

                H_hat_1 = H_hat[0:b]
                H_hat_2 = H_hat[b:2*b]

                # H1 = H_batch_1[:,L:L+Q] 
                # H2 = H_batch_2[:,L:L+Q] 
                # H1_UE = H_batch_1[:,L:L+1]
                # H2_UE = H_batch_2[:,L:L+1]
                # H1_base = H_batch_1[:,L:L+1]
                # H2_base = H_batch_2[:,L:L+1]

                H1 = H_batch_1[:,L:L+Q] 
                H2 = H_batch_2[:,L:L+Q] 
                H1_UE = H_hat_1.reshape(b,1,Nc,N_r,N_t)
                H2_UE = H_hat_2.reshape(b,1,Nc,N_r,N_t)
                H1_base = H_hat_1.reshape(b,1,Nc,N_r,N_t)
                H2_base = H_hat_2.reshape(b,1,Nc,N_r,N_t)
                P_analog1,P_analog2,C_analog,P_digital1,P_digital2,C_digital,F_1, F_2, W = Pre_PCA_total(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2)

                # 打印输出形状以验证
                # print("用户1端模拟预编码器 P_analog1 的形状:", P_analog1.shape)    # [b, T, 1, N_t, 2]
                # print("用户2端模拟预编码器 P_analog2 的形状:", P_analog2.shape)    # [b, T, 1, N_t, 2]
                # print("基站端模拟合并器 C_analog 的形状:", C_analog.shape)           # [b, T, 1, 2, N_r]
                # print("用户1端数字预编码器 P_digital1 的形状:", P_digital1.shape)  # [b, T, Nc, 2, 1]
                # print("用户2端数字预编码器 P_digital2 的形状:", P_digital2.shape)  # [b, T, Nc, 2, 1]
                # print("基站端数字合并器 C_digital 的形状:", C_digital.shape)         # [b, T, Nc, 2, 2]


                # W = torch.randn(b, T, Nc, 2, N_r).cuda() + 1j*torch.randn(b, T, Nc, 2, N_r).cuda()
                R1 = calculate_average_channel_capacity_T(H1, F_1, W, SNR_dB)
                R2 = calculate_average_channel_capacity_T(H2, F_2, W, SNR_dB)
                R_acc = R_acc + (R1.mean().item() + R2.mean().item())/2/val_num

        return loss_acc,R_acc






    loss_SGCS,R_acc = evaluate(0);
    print("SGCS: ",-loss_SGCS,"    SE: ",R_acc)

#### 开始训练

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 1

v_max_km_pre = None

SNR_dB = 10
v_max_km = 120
train_CE_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,SNR_dB,v_max_km,v_max_km_pre,ITERS=60_000,b=64,acc_num=1)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

v_max_km_pre = None

SNR_dB = 10
v_max_km = 120
for L in [16,1,2,4,8]:
    train_CE_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,SNR_dB,v_max_km,v_max_km_pre,ITERS=60_000,b=64,acc_num=1)

#### 开始测试

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

v_max_km_pre = None

SNR_dB = 10
v_max_km = 120
for L in [1,2,4,8,16]:
    test_CE_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,SNR_dB,v_max_km)

### WY CSI反馈

#### 网络和训练函数

In [ ]:
from tqdm import tqdm
from tqdm import trange
from time import sleep
from my_log import logger, rmLogger






class H2bits(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_t,    #用户发送天线数
            feedback_bits,  #反馈比特数
        ):
        super().__init__();

        self.d_model = 1024
        self.B = 4  #量化精度

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)

        self.embed= nn.Linear(2*N_r*N_t, self.d_model) 
        self.positional_embedding=nn.Parameter(torch.zeros(1, Nc, self.d_model));

        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(4)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model

        self.quan=nn.Sequential(  
            ReshapeLayer([-1, Nc*self.d_model]),  
            nn.Linear(Nc*self.d_model, feedback_bits//self.B),  # b, feedback_bits//self.B
            nn.BatchNorm1d(feedback_bits//self.B), 
            nn.Sigmoid(), 
            QuantizationLayer(self.B),  # b, feedback_bits
        );

    



    
    def forward(
            self, 
            H,  # b, Nc, N_r, N_t
        ):
        # b = H_DL_batch.shape[0]
        # H_DL_batch_1 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,0,:].conj() #这里加了conj是因为上下行信道之间有个共轭转置，所以共轭这部分需要conj回去消除掉
        # H_DL_batch_2 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,1,:].conj()

        # H_DL_batch_beam_1 = (H_DL_batch_1*np.exp(1j*pi/2 * 0) + H_DL_batch_2*np.exp(1j*pi/2 * 0)).reshape(b,1,48,4,256)

        b, Nc, N_r, N_t = H.shape


        y_cat = H.reshape(b,Nc,N_r*N_t) 
        y_cat=torch.cat([y_cat.real, y_cat.imag], dim=-1);  # b,Nc,2*N_r*N_t

        x = self.embed(y_cat) + self.positional_embedding  # b,Nc,d_model

        x = self.backbone(x)   # b,Nc,d_model
        out = self.quan(x)   #[batch, Bits]
        return out     #[batch, Bits]



class bits2H(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_t,    #用户发送天线数
            feedback_bits,  #反馈比特数
        ):
        super().__init__();

        self.d_model = 1024
        self.B = 4  #量化精度
        self.feedback_bits = feedback_bits

        self.Nc = Nc
        self.N_r = N_r
        self.N_r_RF = N_r_RF
        self.N_t = N_t

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)
        self.Dequan=nn.Sequential(  
            DequantizationLayer(self.B),  
        );

        self.embed=nn.Sequential(  
            nn.Linear(feedback_bits//self.B, Nc*self.d_model),  
            ReshapeLayer([-1, Nc,self.d_model]),  
            NormLayer(self.d_model), 
        );

        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(4)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model

        # self.head=nn.Sequential(  
        #     nn.Linear(self.d_model, N_t*2),  
        # );
        self.head=nn.Sequential(  
            nn.Linear(self.d_model, 2*N_t*N_r),  
        );

    
    
    def forward(
            self, 
            x,  # b, feedback_bits   #反馈比特
        ):
        N_r = self.N_r
        N_t = self.N_t
        Nc = self.Nc

        b = x.shape[0]

        x = self.Dequan(x) # b,feedback_bits//self.B

        out = self.embed(x) # b,Nc,d_model
        out = self.backbone(out)# b,Nc,d_model
        out = self.head(out)   # b,Nc,2*N_r*N_t


        H_hat = out.reshape(b,Nc,N_r,N_t,2)
        H_hat = H_hat[:,:,:,:,0] + 1j*H_hat[:,:,:,:,1]


        return H_hat     #[b,Nc,N_r,N_t]

class CF_WY(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_t,    #用户发送天线数
            feedback_bits,  #反馈比特数
        ):
        super().__init__();

        self.H2bits = H2bits(Nc,N_r,N_t,feedback_bits)
        self.bits2H = bits2H(Nc,N_r,N_t,feedback_bits)
    
    def forward(
            self, 
            H,  # b, Nc, N_r, N_t 用户
        ):

        bits = self.H2bits(H)
        H_hat = self.bits2H(bits)
        return H_hat
    
    


In [ ]:
def train_CF_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre = None,v_max_km_pre=None,ITERS=20_000,b=32,acc_num=1):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=10;

    model_name_CE = 'CE_WY'+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_CF = 'CF_WY'+'_bits' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    model_name = model_name_CF

    dec_CE=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
    dec_CF=CF_WY(Nc=Nc,N_r=N_r,N_t=N_t,feedback_bits=feedback_bits).cuda()
    
    opt_dec=torch.optim.AdamW(
        dec_CF.parameters(), 
        lr=lr, 
    )
    T = L+1
    load_path_CE=save_path+"CE_WY"+"/";
    load_path_CF=save_path+"CF_WY"+"/";

    dec_dict = dec_CE.state_dict()
    dec_load_path=load_path_CE+model_name_CE+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CE.load_state_dict(dec_dict);
    print('CE参数读取成功')

    dec_CE.eval()


    if v_max_km_pre == None:
        print('从头训练')
    else:
        dec_dict = dec_CF.state_dict()
        model_name_pre = 'CF_WY'+'_bits' + str(feedback_bits_pre)+'_L' + str(L)+'_v' + str(v_max_km_pre)
        dec_load_path=load_path_CF+model_name_pre+"_model.pth.tar";
        ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
        pretrained_dict=ckpt['state_dict'];
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict and ".quan" not in k and ".Dequan" not in k}
        dec_dict.update(pretrained_dict);
        dec_CF.load_state_dict(dec_dict);
        print('全部预训练参数读取成功')

    def step(step_id):
        dec_CF.train();
        opt_dec.zero_grad();
        loss_acc=0;

        


        

        for _ in range(acc_num):
            alpha,A_r,A_t,fd,tau = SV_Channel_set(b,Nc,N,Nt,Nr,1,v_max_km)
            H_batch = torch.zeros(b,T, Nc, N_r, N_t).cuda() + 0j

            for i in range(T):
                t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                H = SV_Channel_final(b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                H_batch[:,i] = H
            # H_batch_1 = H_batch[0:b]
            # H_batch_2 = H_batch[b:2*b]

            

            with torch.no_grad():
                H_CE=dec_CE(
                    H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                );
            H_CF=dec_CF(
                    H = H_CE,  # b,T, Nc, N_r, N_t 用户1
                );
            # H_hat # b,Nc, N_r, N_t 用户1



            # loss_SE = -calculate_average_channel_capacity(H_DL_batch_beam[:,0:1], F, Pt,N0)/acc_num

            loss = SGCS_H(H_batch[:,L],H_CF)/acc_num
            loss_acc += loss.item()

            loss.backward();
        

        opt_dec.step();

        return loss_acc






    def evaluate(step_id):
        global loader_test;
        dec_CF.eval();
        loss_acc_CE=0;
        loss_acc_CF=0;
        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8
                    H = SV_Channel_final(b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                # H_batch_1 = H_batch[0:b]
                # H_batch_2 = H_batch[b:2*b]


                
                H_CE=dec_CE(
                    H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                );
                H_CF=dec_CF(
                    H = H_CE,  # b,T, Nc, N_r, N_t 用户1
                );

                loss_CE = SGCS_H(H_batch[:,L],H_CE)/val_num
                loss_acc_CE += loss_CE.item()

                loss_CF = SGCS_H(H_batch[:,L],H_CF)/val_num
                loss_acc_CF += loss_CF.item()

        return loss_acc_CE,loss_acc_CF

    def save_ckpt(folder_name):
        folder_path=save_path+folder_name+"/";
        if not os.path.exists(folder_path):
            os.makedirs(folder_path);

        torch.save({'state_dict': dec_CF.state_dict()}, folder_path+model_name+"_model.pth.tar");
        torch.save({'best_loss': best_loss, 
                    'opt_dec_dict': opt_dec.state_dict(), 
                    }, folder_path+model_name+"_opt.pth.tar");

    loss_SGCS_log=0;
    t1=time.time();




    with trange(iter_begin, iter_begin+ITERS) as t:
        for i in t:            
            loss_SGCS = step(i);
            loss_SGCS_log+=loss_SGCS/log_interval;

            if (i+1)%log_interval==0:
                loss_SGCS_CE,loss_SGCS_CF = evaluate(i);
                
                logger("");
                if loss_SGCS_CF<best_loss:
                    best_loss=loss_SGCS_CF;
                    save_ckpt("CF_WY");
                    logger("\nbest - SGCS = {}".format(best_loss));
                
                logger("log - iter[{}] - train - SGCS = {}".format(i, -loss_SGCS_log));
                logger("log - iter[{}] - test - SGCS = {}".format(i, -loss_SGCS_CF));

                

                logger("time: {} s".format(time.time()-t1));
                t1=time.time();

                if (i+1)%save_interval==0:
                    logger("");
                    logger("save - iter[{}] - test - CE SGCS = {}".format(i, -loss_SGCS_CE));
                    logger("save - iter[{}] - test - SGCS = {}".format(i, -loss_SGCS_CF));
                    logger("save - iter[{}] - best - SGCS = {}".format(i, -best_loss));

                    

                t.set_postfix(train_SGCS=-loss_SGCS_log,test_SGCS=-loss_SGCS_CF,best_SGCS=-best_loss)

                loss_SGCS_log=0;
    rmLogger();

In [ ]:
def test_CF_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=10;

    model_name_CE = 'CE_WY'+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_CF = 'CF_WY'+'_bits' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    model_name = model_name_CF

    dec_CE=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
    dec_CF=CF_WY(Nc=Nc,N_r=N_r,N_t=N_t,feedback_bits=feedback_bits).cuda()

    rate1 = 4
    rate2 = 2
    
    Q = 12
    T = L+Q
    load_path_CE=save_path+"CE_WY"+"/";
    load_path_CF=save_path+"CF_WY"+"/";

    dec_dict = dec_CE.state_dict()
    dec_load_path=load_path_CE+model_name_CE+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CE.load_state_dict(dec_dict);
    dec_CE.eval()



    dec_dict = dec_CF.state_dict()
    dec_load_path=load_path_CF+model_name_CF+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CF.load_state_dict(dec_dict);
    dec_CF.eval()

    





    def evaluate(step_id):
        global loader_test;
        dec_CF.eval();
        loss_acc_CE=0;
        loss_acc_CF=0;
        R_acc = 0
        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]


                
                H_CE=dec_CE(
                    H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                );
                H_CF=dec_CF(
                    H = H_CE,  # b,T, Nc, N_r, N_t 用户1
                );

                loss_CE = SGCS_H(H_batch[:,L],H_CE)/val_num
                loss_acc_CE += loss_CE.item()

                loss_CF = SGCS_H(H_batch[:,L],H_CF)/val_num
                loss_acc_CF += loss_CF.item()
                

                H_CE_1 = H_CE[0:b]
                H_CE_2 = H_CE[b:2*b]
                H_CF_1 = H_CF[0:b]
                H_CF_2 = H_CF[b:2*b]

                # H1 = H_batch_1[:,L:L+Q] 
                # H2 = H_batch_2[:,L:L+Q] 
                # H1_UE = H_batch_1[:,L:L+1]
                # H2_UE = H_batch_2[:,L:L+1]
                # H1_base = H_batch_1[:,L:L+1]
                # H2_base = H_batch_2[:,L:L+1]

                H1 = H_batch_1[:,L:L+Q] 
                H2 = H_batch_2[:,L:L+Q] 
                H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
                H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
                H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
                H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
                P_analog1,P_analog2,C_analog,P_digital1,P_digital2,C_digital,F_1, F_2, W = Pre_PCA_total(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2)

                # 打印输出形状以验证
                # print("用户1端模拟预编码器 P_analog1 的形状:", P_analog1.shape)    # [b, T, 1, N_t, 2]
                # print("用户2端模拟预编码器 P_analog2 的形状:", P_analog2.shape)    # [b, T, 1, N_t, 2]
                # print("基站端模拟合并器 C_analog 的形状:", C_analog.shape)           # [b, T, 1, 2, N_r]
                # print("用户1端数字预编码器 P_digital1 的形状:", P_digital1.shape)  # [b, T, Nc, 2, 1]
                # print("用户2端数字预编码器 P_digital2 的形状:", P_digital2.shape)  # [b, T, Nc, 2, 1]
                # print("基站端数字合并器 C_digital 的形状:", C_digital.shape)         # [b, T, Nc, 2, 2]


                # W = torch.randn(b, T, Nc, 2, N_r).cuda() + 1j*torch.randn(b, T, Nc, 2, N_r).cuda()
                R1 = calculate_average_channel_capacity_T(H1, F_1, W, SNR_dB)
                R2 = calculate_average_channel_capacity_T(H2, F_2, W, SNR_dB)
                R_acc = R_acc + (R1.mean().item() + R2.mean().item())/2/val_num

        return loss_acc_CE,loss_acc_CF,R_acc






    loss_SGCS_CE,loss_SGCS_CF,R_acc = evaluate(0);
    print("SGCS_CE: ",-loss_SGCS_CE,"    SGCS_CF: ",-loss_SGCS_CF,"    SE: ",R_acc)

#### 开始训练

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for L in [8,1,2,4,16]:
    train_CF_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre,v_max_km_pre,ITERS=60_000,b=64,acc_num=1)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for feedback_bits in [4,16,64,256,4096]:
    train_CF_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre,v_max_km_pre,ITERS=60_000,b=64,acc_num=1)

#### 开始测试

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for L in [1,2,4,8,16]:
    test_CF_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for feedback_bits in [4,16,64,256,1024]:
    test_CF_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km)

### WY 预编码

#### 网络和训练函数

In [ ]:
class H2precoding_WY(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
        ):
        super().__init__();

        self.d_model = 1024
        self.N_t = N_t
        self.N_t_RF = N_t_RF

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)

        self.embed= nn.Linear(2*N_t*N_r, self.d_model) 
        self.positional_embedding=nn.Parameter(torch.zeros(1, Nc, self.d_model));

        self.backbone=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(2)], 
            nn.LayerNorm(self.d_model),
        );  # b, K*T, d_model

        self.head_RF=nn.Sequential(  
            ReshapeLayer([-1, Nc*self.d_model]),  
            nn.Linear(Nc*self.d_model, 2*1*N_t),  
        );
        # self.head_BB=nn.Sequential(  
        #     nn.Linear(self.d_model, 2*N_t_RF),  
        # );
    



    
    def forward(
            self, 
            H_CE,  # b, Nc, N_r, N_t
        ):
        # b = H_DL_batch.shape[0]
        # H_DL_batch_1 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,0,:].conj() #这里加了conj是因为上下行信道之间有个共轭转置，所以共轭这部分需要conj回去消除掉
        # H_DL_batch_2 = H_DL_batch.reshape(b, 48, 4, 128,2,2)[:,:,:,:,1,:].conj()

        # H_DL_batch_beam_1 = (H_DL_batch_1*np.exp(1j*pi/2 * 0) + H_DL_batch_2*np.exp(1j*pi/2 * 0)).reshape(b,1,48,4,256)

        b, Nc, N_r, N_t = H_CE.shape

        y_cat = H_CE.reshape(b,Nc,N_t*N_r) 
        y_cat=torch.cat([y_cat.real, y_cat.imag], dim=-1);  # b,Nc,2*N_t*N_r

        x = self.embed(y_cat) + self.positional_embedding  # b,Nc,d_model

        x = self.backbone(x)   # b,Nc,d_model

        F_RF = self.head_RF(x)
        F_RF = F_RF.reshape(b,1,N_t,1,2) # b,1,N_t,N_t_RF,2
        F_RF = F_RF[:,:,:,:,0] + 1j*F_RF[:,:,:,:,1]
        F_RF = F_RF/torch.abs(F_RF)/sqrt(N_t) #[batch, 1,N_t,1]

        # F_BB = self.head_BB(x)
        # F_BB = F_BB.reshape(b,Nc,N_t_RF,1,2) # b,Nc,N_t_RF,1,2
        # F_BB = F_BB[:,:,:,:,0] + 1j*F_BB[:,:,:,:,1] # b,Nc,N_t_RF,1
        # F = F_RF @ F_BB

        return F_RF     #[batch, 1,N_t,1]





class H2combiner_WY(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数

        ):
        super().__init__();

        self.d_model = 1024


        self.Nc = Nc
        self.N_r = N_r
        self.N_r_RF = N_r_RF

        # self.trans  = TRANS_BLOCK(tx*2,tx*2,512,4)


        self.embed_1 = nn.Linear(2*N_t*N_r, self.d_model) 
        self.embed_2 = nn.Linear(2*N_t*N_r, self.d_model) 

        self.positional_embedding_1=nn.Parameter(torch.zeros(1, Nc, self.d_model));
        self.positional_embedding_2=nn.Parameter(torch.zeros(1, Nc, self.d_model));

        self.backbone_1=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(1)], 
            nn.LayerNorm(self.d_model),
            nn.Linear(self.d_model, 512),  
        );  

        self.backbone_2=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(1)], 
            nn.LayerNorm(self.d_model),
            nn.Linear(self.d_model, 512),  
        );  

        self.embed_3 = nn.Linear(1024, self.d_model) 

        self.backbone_3=nn.Sequential(  # b, K*T, d_model
            *[nn.Sequential(
                nn.TransformerEncoderLayer(
                    d_model=self.d_model, 
                    nhead=8, 
                    dim_feedforward=4*self.d_model, 
                    dropout=0.0, 
                    activation="gelu", 
                    batch_first=True,
                    norm_first = True, 
                ), 
            ) for _ in range(2)], 
            nn.LayerNorm(self.d_model),
        );  

        # self.head=nn.Sequential(  
        #     nn.Linear(self.d_model, N_t*2),  
        # );
        self.head_RF=nn.Sequential(  
            ReshapeLayer([-1, Nc*self.d_model]),  
            nn.Linear(Nc*self.d_model, 2*N_r*N_r_RF),  
        );
        # self.head_BB=nn.Sequential(  
        #     nn.Linear(self.d_model, 2*N_r_RF*N_r_RF),  
        # );
    
    

    
    
    def forward(
            self, 
            H_CF_1,  # b, Nc, N_r, N_t   
            H_CF_2,  # b, Nc, N_r, N_t   
        ):
        

        b, Nc, N_r, N_t = H_CF_1.shape
        N_r_RF = self.N_r_RF

        y_cat = H_CF_1.reshape(b,Nc,N_t*N_r) 
        y_cat=torch.cat([y_cat.real, y_cat.imag], dim=-1);  # b,Nc,2*N_t*N_r
        x_1 = self.embed_1(y_cat) + self.positional_embedding_1  # b,Nc,d_model
        x_1 = self.backbone_1(x_1)   # b,Nc,512

        y_cat = H_CF_2.reshape(b,Nc,N_t*N_r) 
        y_cat=torch.cat([y_cat.real, y_cat.imag], dim=-1);  # b,Nc,2*N_t*N_r
        x_2 = self.embed_2(y_cat) + self.positional_embedding_2  # b,Nc,d_model
        x_2 = self.backbone_2(x_2)   # b,Nc,512

        x = torch.cat([x_1, x_2], dim=-1);  # b,Nc,1024


        out = self.embed_3(x) # b,Nc,d_model
        out = self.backbone_3(out)# b,Nc,d_model
        
        # F = out.reshape(b,1,self.N_t,N_t_RF,2) # b,1,N_t,N_t_RF,2
        # F = F[:,:,:,:,0] + 1j*F[:,:,:,:,1]
        # F = F/torch.abs(F)/sqrt(self.N_t) #[batch, 1,N_t,N_t_RF]

        W_RF = self.head_RF(out)   # b,N_t*2
        W_RF = W_RF.reshape(b,1,self.N_r_RF,self.N_r,2)
        W_RF = W_RF[:,:,:,:,0] + 1j*W_RF[:,:,:,:,1]
        W_RF = W_RF/torch.abs(W_RF)/sqrt(self.N_r)  #b,1,self.N_r_RF,self.N_r

        # W_BB = self.head_BB(out)   # b,Nc,2*N_r_RF*N_r_RF
        # W_BB = W_BB.reshape(b,Nc,self.N_r_RF,self.N_r_RF,2)
        # W_BB = W_BB[:,:,:,:,0] + 1j*W_BB[:,:,:,:,1] # b,Nc,N_r_RF,N_r_RF

        # W = W_BB @ W_RF


        return W_RF     #[b,1,N_r_RF,N_r]

class Precoding_WY(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
        ):
        super().__init__();

        self.H2precoding_WY_1 = H2precoding_WY(Nc,N_r,N_r_RF,N_t,N_t_RF)
        self.H2precoding_WY_2 = H2precoding_WY(Nc,N_r,N_r_RF,N_t,N_t_RF)
        self.H2combiner_WY = H2combiner_WY(Nc,N_r,N_r_RF,N_t,N_t_RF)

    def forward(
            self, 
            H_CE_1,  # b, Nc, N_r, N_t 
            H_CE_2,  # b, Nc, N_r, N_t 
            H_CF_1,  # b, Nc, N_r, N_t 
            H_CF_2,  # b, Nc, N_r, N_t 
        ):
        F_1 = self.H2precoding_WY_1(H_CE_1)
        F_2 = self.H2precoding_WY_2(H_CE_2)
        W = self.H2combiner_WY(H_CF_1,H_CF_2)
        return W,F_1,F_2
    
    


In [ ]:
import torch
### 时域和频域插值外推
def interpolate_upsample(x, scale_factors, mode='bilinear', align_corners=False):
    """
    使用双线性插值对张量 x 在时间和频域维度上进行上采样。
    
    参数:
        x (torch.Tensor): 输入张量，形状为 [b, T, Nc, N_r, N_t]。
        scale_factors (tuple): 时间和频域的上采样因子 (scale_time, scale_freq)。
        mode (str): 插值模式，默认为 'bilinear'。
        align_corners (bool): 插值参数，默认为 False。
    
    返回:
        x_up (torch.Tensor): 上采样后的张量，形状为 [b, T*scale_time, Nc*scale_freq, N_r, N_t]。
    """
    scale_time, scale_freq = scale_factors
    
    # Reshape to [b * N_r * N_t, 1, T, Nc] for 2D interpolation
    b, T, Nc, N_r, N_t = x.shape
    x_reshaped = x.permute(0, 3, 4, 1, 2).contiguous().view(b * N_r * N_t, 1, T, Nc)
    
    # Perform 2D interpolation on time and frequency dimensions
    x_up = F.interpolate(x_reshaped, scale_factor=(scale_time, scale_freq), mode=mode, align_corners=align_corners)
    
    # Reshape back to [b, T*scale_time, Nc*scale_freq, N_r, N_t]
    b_new, _, T_up, Nc_up = x_up.shape
    x_up = x_up.view(b, N_r, N_t, T_up, Nc_up).permute(0, 3, 4, 1, 2).contiguous()
    
    return x_up
def Subcar_Interp(H1, H2, rate1, rate2):
    """
    对下采样后的信道 H1 和 H2 进行时间域和频域的插值上采样，恢复到完整的维度。
    
    参数:
        H1 (torch.Tensor): 下采样后的信道矩阵，形状为 [b, T_down, Nc_down, N_r, N_t]，复数类型。
        H2 (torch.Tensor): 下采样后的信道矩阵，形状为 [b, T_down, Nc_down, N_r, N_t]，复数类型。
        rate1 (int): 时间域上采样因子（多普勒域）。
        rate2 (int): 频域上采样因子（延时域）。
    
    返回:
        H1_up (torch.Tensor): 上采样后的 H1，形状为 [b, T*rate1, Nc*rate2, N_r, N_t]。
        H2_up_freq_shifted (torch.Tensor): 上采样并频率偏移后的 H2，形状为 [b, T*rate1, Nc*rate2, N_r, N_t]。
    """
    # 分离实部和虚部进行插值
    H1_real = H1.real
    H1_imag = H1.imag
    H2_real = H2.real
    H2_imag = H2.imag
    
    # Perform interpolation on real and imaginary parts
    H1_up_real = interpolate_upsample(H1_real, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H1_up_imag = interpolate_upsample(H1_imag, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H2_up_real = interpolate_upsample(H2_real, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    H2_up_imag = interpolate_upsample(H2_imag, scale_factors=(rate1, rate2), mode='bilinear', align_corners=False)
    
    # Reconstruct complex tensors
    H1_up = torch.complex(H1_up_real, H1_up_imag)
    H2_up = torch.complex(H2_up_real, H2_up_imag)
    
    # Frequency shift for H2_up
    H1_up = torch.roll(H1_up, shifts=-rate2//2, dims=2)  # Shift along frequency dimension
    
    return H1_up, H2_up
def Pre_WY_DMRS(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2, W, F_1, F_2):
    """
    MIMO预编码函数 Pre_PCA

    参数:
        H1: 张量，形状为 [b, Q, Nc, N_r, N_t]，用户1到基站的信道矩阵
        H2: 张量，形状为 [b, Q, Nc, N_r, N_t]，用户2到基站的信道矩阵
        H1_UE: 张量，形状为 [b, 1, Nc, N_r, N_t]，用户端信道估计的信道 用户1到基站的信道矩阵
        H2_UE: 张量，形状为 [b, 1, Nc, N_r, N_t]，用户端信道估计的信道 用户2到基站的信道矩阵
        H1_base: 张量，形状为 [b, 1, Nc, N_r, N_t]，基站端CSI反馈重建的信道 用户1到基站的信道矩阵
        H2_base: 张量，形状为 [b, 1, Nc, N_r, N_t]，基站端CSI反馈重建的信道 用户2到基站的信道矩阵
    返回:
        {
            'P_analog1': 用户1模拟预编码器，形状 [b, T, 1, N_t, 2],
            'P_analog2': 用户2模拟预编码器，形状 [b, T, 1, N_t, 2],
            'C_analog': 基站端模拟合并器，形状 [b, T, 1, 2, N_r],
            'P_digital1': 用户1数字预编码器，形状 [b, T, Nc, 2, 1],
            'P_digital2': 用户2数字预编码器, 形状 [b, T, Nc, 2, 1],
            'C_digital': 基站端数字合并器，形状 [b, T, Nc, 2, 2]
        }
    """
    # 获取输入信道矩阵的尺寸信息
    b, Q, Nc, N_r, N_t = H1.shape
    N_RF = 2  # 射频链路数

    

    def compute_digital_combiner(W, H, F):
        """
        计算数字合并器的等效信道

        输入:
            C_analog: 张量，形状为 [b, 1, 1, 2, N_r]
            H: 张量，形状为 [b, Q, Nc, N_r, N_t]
            P_analog: 张量，形状为 [b, 1, 1, N_t, 2]
            P_digital: 张量，形状为 [b, 1, Nc, 2, 1]

        输出:
            H_eff: 张量，形状为 [b, T, Nc, 2, 1]
        """
       

        
        norm_factor = torch.sqrt(torch.sum(torch.abs(F)**2, dim=(-2, -1), keepdim=True))
        F = F / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))  #[b, 1, 1, N_t, 1]

        norm_factor = torch.sqrt(torch.sum(torch.abs(W)**2, dim=(-2, -1), keepdim=True))
        W = W / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))

        # Calculate effective channel
        sigma = 10**(-SNR_dB/10)
        n = (torch.randn(b,Q,Nc,N_RF,1).cuda() + 1j*torch.randn(b,Q,Nc,N_RF,1).cuda())/sqrt(2)*sqrt(sigma)
        H_eff = W @ H @ F + n  # [b, Q, Nc, 2, 1]
        # H_eff = W @ H @ F   # [b, Q, Nc, 2, 1]


        return H_eff  # [b, Q, Nc, 2, 1]

    # 计算两个用户的等效信道
    

    norm_factor = torch.sqrt(torch.sum(torch.abs(F_1)**2, dim=(-2, -1), keepdim=True))
    F_1 = F_1 / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))
    norm_factor = torch.sqrt(torch.sum(torch.abs(F_2)**2, dim=(-2, -1), keepdim=True))
    F_2 = F_2 / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))
    norm_factor = torch.sqrt(torch.sum(torch.abs(W)**2, dim=(-2, -1), keepdim=True))
    W = W / norm_factor * torch.sqrt(torch.tensor(N_RF, dtype=H1.dtype, device=H1.device))

    H_eff1 = compute_digital_combiner(W, H1, F_1)  # 用户1等效信道 [b, Q, Nc, 2, 1]
    H_eff2 = compute_digital_combiner(W, H2, F_2)  # 用户2等效信道 [b, Q, Nc, 2, 1]

    H_eff1_down = H_eff1[:, ::rate1, ::rate2, :, :]
    H_eff2_down = H_eff2[:, ::rate1, rate2//2::rate2, :, :]
    H_eff1_up, H_eff2_up = Subcar_Interp(H_eff1_down, H_eff2_down, rate1, rate2)

    # print(NMSE(H_eff1_up,H_eff1))
    # print(NMSE(H_eff2_up,H_eff2))

    # 拼接两个用户的等效信道，形成 [b, T, Nc, 2, 2]
    H_eff_stack = torch.cat([H_eff1_up, H_eff2_up], dim=-1)  # [b, Q, Nc, 2, 2]

    def compute_zf_digital_combiner(H_eff_stack):
        T = Q
        """
        计算基站端的数字合并器，使用零迫（ZF）方法

        输入:
            H_eff_stack: 张量，形状为 [b, T, Nc, 2, 2]

        输出:
            C_digital: 张量，形状为 [b, T, Nc, 2, 2]
        """
        # 计算数字合并器 W = (H_eff^H * H_eff)^{-1} * H_eff^H
        # 首先计算 H_eff^H
        H_eff_H = H_eff_stack.conj().transpose(-2, -1)  # [b, T, Nc, 2, 2]

        # 计算 H_eff^H * H_eff
        H_eff_H_H_eff = torch.matmul(H_eff_H, H_eff_stack)  # [b, T, Nc, 2, 2]

        # 计算 (H_eff^H * H_eff)^{-1}
        H_eff_H_H_eff_inv = torch.linalg.inv(H_eff_H_H_eff+1)  # [b, T, Nc, 2, 2]

        # 计算 W = (H_eff^H * H_eff)^{-1} * H_eff^H
        C_digital = torch.matmul(H_eff_H_H_eff_inv, H_eff_H)  # [b, T, Nc, 2, 2]

        return C_digital  # [b, T, Nc, 2, 2]

    # 计算基站端数字合并器
    C_digital = compute_zf_digital_combiner(H_eff_stack)  # [b, Q, Nc, 2, 2]

    W = C_digital @ W



    # 最终输出
    return W[:,0:Q] #b,Q,Nc,2,N_r




In [ ]:
def train_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre = None,v_max_km_pre=None,ITERS=20_000,b=32,acc_num=1):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;
    rate1 = 4
    rate2 = 2


    model_name_CE = 'CE_WY'+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_CF = 'CF_WY'+'_bits' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)

    dec_CE=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
    dec_CF=CF_WY(Nc=Nc,N_r=N_r,N_t=N_t,feedback_bits=feedback_bits).cuda()
    
    Q = 12
    T = L+Q
    load_path_CE=save_path+"CE_WY"+"/";
    load_path_CF=save_path+"CF_WY"+"/";

    dec_dict = dec_CE.state_dict()
    dec_load_path=load_path_CE+model_name_CE+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CE.load_state_dict(dec_dict);
    dec_CE.eval()



    dec_dict = dec_CF.state_dict()
    dec_load_path=load_path_CF+model_name_CF+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CF.load_state_dict(dec_dict);
    dec_CF.eval()


    model_name_Precoding = 'Precoding_WY' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    model_name = model_name_Precoding

    dec=Precoding_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF).cuda()
    
    opt_dec=torch.optim.AdamW(
        dec.parameters(), 
        lr=lr, 
    )
    load_path=save_path+"Precoding_WY"+"/";
    if feedback_bits_pre == None:
        print('从头训练')
    else:
        dec_dict = dec.state_dict()
        model_name_pre = 'Precoding_WY' + str(feedback_bits_pre)+'_L' + str(L)+'_v' + str(v_max_km_pre)
        dec_load_path=load_path+model_name_pre+"_model.pth.tar";
        ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
        pretrained_dict=ckpt['state_dict'];
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict and ".quan" not in k and ".Dequan" not in k}
        dec_dict.update(pretrained_dict);
        dec.load_state_dict(dec_dict);
        print('全部预训练参数读取成功')

    def step(step_id):
        dec.train();
        opt_dec.zero_grad();
        loss_acc=0;

        


        

        for _ in range(acc_num):
            alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
            H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

            for i in range(T):
                t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                H_batch[:,i] = H
            H_batch_1 = H_batch[0:b]
            H_batch_2 = H_batch[b:2*b]

            

            with torch.no_grad():
                H_CE=dec_CE(
                    H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                );
                H_CF=dec_CF(
                        H = H_CE,  # b,T, Nc, N_r, N_t 用户1
                    );
                H_CE_1 = H_CE[0:b]
                H_CE_2 = H_CE[b:2*b]
                H_CF_1 = H_CF[0:b]
                H_CF_2 = H_CF[b:2*b]
            
            W,F_1,F_2=dec(
                    H_CE_1 = H_CE_1,  # b, Nc, N_r, N_t 用户1
                    H_CE_2 = H_CE_2,  # b, Nc, N_r, N_t 用户2
                    H_CF_1 = H_CF_1,  # b, Nc, N_r, N_t 用户1
                    H_CF_2 = H_CF_2,  # b, Nc, N_r, N_t 用户2
                );



            # loss_SE = -calculate_average_channel_capacity(H_DL_batch_beam[:,0:1], F, Pt,N0)/acc_num

            loss = -calculate_average_channel_capacity_OMA_woInt(H_batch_1[:,L],H_batch_2[:,L],F_1,F_2,W, SNR_dB)/acc_num
            loss_acc += loss.item()

            loss.backward();
        

        opt_dec.step();

        return loss_acc






    def evaluate(step_id):
        global loader_test;
        dec.eval();
        loss_acc=0;
        loss_acc_CE = 0
        loss_acc_CF = 0
        R_acc = 0

        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]

                H_CE=dec_CE(
                    H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                );
                H_CF=dec_CF(
                        H = H_CE,  # b,T, Nc, N_r, N_t 用户1
                    );

                loss_CE = SGCS_H(H_batch[:,L],H_CE)/val_num
                loss_acc_CE += loss_CE.item()

                loss_CF = SGCS_H(H_batch[:,L],H_CF)/val_num
                loss_acc_CF += loss_CF.item()

                H_CE_1 = H_CE[0:b]
                H_CE_2 = H_CE[b:2*b]
                H_CF_1 = H_CF[0:b]
                H_CF_2 = H_CF[b:2*b]
            


                
                W,F_1,F_2=dec(
                        H_CE_1 = H_CE_1,  # b, Nc, N_r, N_t 用户1
                        H_CE_2 = H_CE_2,  # b, Nc, N_r, N_t 用户2
                        H_CF_1 = H_CF_1,  # b, Nc, N_r, N_t 用户1
                        H_CF_2 = H_CF_2,  # b, Nc, N_r, N_t 用户2
                    );
                loss = -calculate_average_channel_capacity_OMA_woInt(H_batch_1[:,L],H_batch_2[:,L],F_1,F_2,W, SNR_dB)/val_num
                loss_acc += loss.item()

                H1 = H_batch_1[:,L:L+Q] 
                H2 = H_batch_2[:,L:L+Q] 
                H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
                H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
                H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
                H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
                W = W.reshape(b,1,1,N_r_RF,N_r)
                F_1 = F_1.reshape(b,1,1,N_t,1)
                F_2 = F_2.reshape(b,1,1,N_t,1)
                W = Pre_WY_DMRS(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2, W, F_1, F_2)

                R1 = calculate_average_channel_capacity_T(H1, F_1, W, SNR_dB)
                R2 = calculate_average_channel_capacity_T(H2, F_2, W, SNR_dB)
                R_acc = R_acc + (R1.mean().item() + R2.mean().item())/2/val_num

                

        return loss_acc,R_acc,loss_acc_CE,loss_acc_CF

    def save_ckpt(folder_name):
        folder_path=save_path+folder_name+"/";
        if not os.path.exists(folder_path):
            os.makedirs(folder_path);

        torch.save({'state_dict': dec.state_dict()}, folder_path+model_name+"_model.pth.tar");
        torch.save({'best_loss': best_loss, 
                    'opt_dec_dict': opt_dec.state_dict(), 
                    }, folder_path+model_name+"_opt.pth.tar");

    loss_SE_log=0;
    t1=time.time();




    with trange(iter_begin, iter_begin+ITERS) as t:
        for i in t:            
            loss_SE = step(i);
            loss_SE_log+=loss_SE/log_interval;

            if (i+1)%log_interval==0:
                loss_SE,R,SGCS_CE,SGCS_CF = evaluate(i);
                
                logger("");
                if loss_SE<best_loss:
                    best_loss=loss_SE;
                    save_ckpt("Precoding_WY");
                    logger("\nbest - SE = {}".format(-best_loss));
                
                logger("log - iter[{}] - train - SE = {}".format(i, -loss_SE_log));
                logger("log - iter[{}] - test - SE = {}".format(i, -loss_SE));
                logger("log - iter[{}] - test - ZF_SE = {}".format(i, R));
                logger("log - iter[{}] - test - SGCS_CE = {}".format(i, -SGCS_CE));
                logger("log - iter[{}] - test - SGCS_CF = {}".format(i, -SGCS_CF));

                

                logger("time: {} s".format(time.time()-t1));
                t1=time.time();

                if (i+1)%save_interval==0:
                    logger("");
                    logger("save - iter[{}] - test - SE = {}".format(i, -loss_SE));
                    logger("save - iter[{}] - best - SE = {}".format(i, -best_loss));

                    

                t.set_postfix(train_SE=-loss_SE_log,test_SE=-loss_SE,best_SE=-best_loss)

                loss_SE_log=0;
    rmLogger();

In [ ]:
def train_Precoding_WY_perfect(N,Nc,Nr,N_r_RF,Nt,N_t_RF,SNR_dB,v_max_km,ITERS=20_000,b=32,acc_num=1):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;
    rate1 = 4
    rate2 = 2


    
    Q = 12
    T = L+Q








    model_name_Precoding = 'Precoding_WY_perfect' + '_v' + str(v_max_km)
    model_name = model_name_Precoding

    dec=Precoding_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF).cuda()
    
    opt_dec=torch.optim.AdamW(
        dec.parameters(), 
        lr=lr, 
    )
    print('从头训练')


    def step(step_id):
        dec.train();
        opt_dec.zero_grad();
        loss_acc=0;

        


        

        for _ in range(acc_num):
            alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
            H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

            for i in range(T):
                t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                H_batch[:,i] = H
            H_batch_1 = H_batch[0:b]
            H_batch_2 = H_batch[b:2*b]

            

            with torch.no_grad():

                H_CE_1 = H_batch_1[:,L]
                H_CE_2 = H_batch_2[:,L]
                H_CF_1 = H_batch_1[:,L]
                H_CF_2 = H_batch_2[:,L]
            
            W,F_1,F_2=dec(
                    H_CE_1 = H_CE_1,  # b, Nc, N_r, N_t 用户1
                    H_CE_2 = H_CE_2,  # b, Nc, N_r, N_t 用户2
                    H_CF_1 = H_CF_1,  # b, Nc, N_r, N_t 用户1
                    H_CF_2 = H_CF_2,  # b, Nc, N_r, N_t 用户2
                );



            # loss_SE = -calculate_average_channel_capacity(H_DL_batch_beam[:,0:1], F, Pt,N0)/acc_num

            loss = -calculate_average_channel_capacity_OMA_woInt(H_batch_1[:,L],H_batch_2[:,L],F_1,F_2,W, SNR_dB)/acc_num
            loss_acc += loss.item()

            loss.backward();
        

        opt_dec.step();

        return loss_acc






    def evaluate(step_id):
        global loader_test;
        dec.eval();
        loss_acc=0;
        loss_acc_CE = 0
        loss_acc_CF = 0
        R_acc = 0

        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]


                H_CE_1 = H_batch_1[:,L]
                H_CE_2 = H_batch_2[:,L]
                H_CF_1 = H_batch_1[:,L]
                H_CF_2 = H_batch_2[:,L]
            


                
                W,F_1,F_2=dec(
                        H_CE_1 = H_CE_1,  # b, Nc, N_r, N_t 用户1
                        H_CE_2 = H_CE_2,  # b, Nc, N_r, N_t 用户2
                        H_CF_1 = H_CF_1,  # b, Nc, N_r, N_t 用户1
                        H_CF_2 = H_CF_2,  # b, Nc, N_r, N_t 用户2
                    );
                loss = -calculate_average_channel_capacity_OMA_woInt(H_batch_1[:,L],H_batch_2[:,L],F_1,F_2,W, SNR_dB)/val_num
                loss_acc += loss.item()

                H1 = H_batch_1[:,L:L+Q] 
                H2 = H_batch_2[:,L:L+Q] 
                H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
                H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
                H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
                H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
                W = W.reshape(b,1,1,N_r_RF,N_r)
                F_1 = F_1.reshape(b,1,1,N_t,1)
                F_2 = F_2.reshape(b,1,1,N_t,1)
                W = Pre_WY_DMRS(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2, W, F_1, F_2)

                R1 = calculate_average_channel_capacity_T(H1, F_1, W, SNR_dB)
                R2 = calculate_average_channel_capacity_T(H2, F_2, W, SNR_dB)
                R_acc = R_acc + (R1.mean().item() + R2.mean().item())/2/val_num

                

        return loss_acc,R_acc

    def save_ckpt(folder_name):
        folder_path=save_path+folder_name+"/";
        if not os.path.exists(folder_path):
            os.makedirs(folder_path);

        torch.save({'state_dict': dec.state_dict()}, folder_path+model_name+"_model.pth.tar");
        torch.save({'best_loss': best_loss, 
                    'opt_dec_dict': opt_dec.state_dict(), 
                    }, folder_path+model_name+"_opt.pth.tar");

    loss_SE_log=0;
    t1=time.time();




    with trange(iter_begin, iter_begin+ITERS) as t:
        for i in t:            
            loss_SE = step(i);
            loss_SE_log+=loss_SE/log_interval;

            if (i+1)%log_interval==0:
                loss_SE,R = evaluate(i);
                
                logger("");
                if loss_SE<best_loss:
                    best_loss=loss_SE;
                    save_ckpt("Precoding_WY");
                    logger("\nbest - SE = {}".format(-best_loss));
                
                logger("log - iter[{}] - train - SE = {}".format(i, -loss_SE_log));
                logger("log - iter[{}] - test - SE = {}".format(i, -loss_SE));
                logger("log - iter[{}] - test - ZF_SE = {}".format(i, R));

                

                logger("time: {} s".format(time.time()-t1));
                t1=time.time();

                if (i+1)%save_interval==0:
                    logger("");
                    logger("save - iter[{}] - test - SE = {}".format(i, -loss_SE));
                    logger("save - iter[{}] - best - SE = {}".format(i, -best_loss));

                    

                t.set_postfix(train_SE=-loss_SE_log,test_SE=-loss_SE,best_SE=-best_loss)

                loss_SE_log=0;
    rmLogger();

In [ ]:
def test_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,flag_DL='DL'):
    # N代表路径数
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;
    rate1 = 4
    rate2 = 2


    model_name_CE = 'CE_WY'+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_CF = 'CF_WY'+'_bits' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_Pre = 'Precoding_WY' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)

    dec_CE=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
    dec_CF=CF_WY(Nc=Nc,N_r=N_r,N_t=N_t,feedback_bits=feedback_bits).cuda()
    dec_Pre=Precoding_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF).cuda()
    
    Q = 12
    T = L+Q
    load_path_CE=save_path+"CE_WY"+"/";
    load_path_CF=save_path+"CF_WY"+"/";
    load_path_Pre=save_path+"Precoding_WY"+"/";

    dec_dict = dec_CE.state_dict()
    dec_load_path=load_path_CE+model_name_CE+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CE.load_state_dict(dec_dict);
    dec_CE.eval()



    dec_dict = dec_CF.state_dict()
    dec_load_path=load_path_CF+model_name_CF+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CF.load_state_dict(dec_dict);
    dec_CF.eval()


    

    

    dec_dict = dec_Pre.state_dict()
    dec_load_path=load_path_Pre+model_name_Pre+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_Pre.load_state_dict(dec_dict);
    dec_Pre.eval()

    





    def evaluate(step_id):
        global loader_test;
        loss_acc=0;
        loss_acc_CE = 0
        loss_acc_CF = 0
        R_acc = 0

        val_num = 10
        b = 64
        with torch.no_grad():
            for i in range(val_num):

                alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
                H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

                for i in range(T):
                    t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                    H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                    H_batch[:,i] = H
                H_batch_1 = H_batch[0:b]
                H_batch_2 = H_batch[b:2*b]

                H_CE=dec_CE(
                    H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
                );
                H_CF=dec_CF(
                        H = H_CE,  # b,T, Nc, N_r, N_t 用户1
                    );

                loss_CE = SGCS_H(H_batch[:,L],H_CE)/val_num
                loss_acc_CE += loss_CE.item()

                loss_CF = SGCS_H(H_batch[:,L],H_CF)/val_num
                loss_acc_CF += loss_CF.item()

                H_CE_1 = H_CE[0:b]
                H_CE_2 = H_CE[b:2*b]
                H_CF_1 = H_CF[0:b]
                H_CF_2 = H_CF[b:2*b]
            


                
                # W,F_1,F_2=dec_Pre(
                #         H_CE_1 = H_CE_1,  # b, Nc, N_r, N_t 用户1
                #         H_CE_2 = H_CE_2,  # b, Nc, N_r, N_t 用户2
                #         H_CF_1 = H_CF_1,  # b, Nc, N_r, N_t 用户1
                #         H_CF_2 = H_CF_2,  # b, Nc, N_r, N_t 用户2
                #     );
                # # loss = -calculate_average_channel_capacity_OMA_woInt(H_batch_1[:,L],H_batch_2[:,L],F_1,F_2,W, SNR_dB)/val_num
                # # loss_acc += loss.item()

                # H1 = H_batch_1[:,L:L+Q] 
                # H2 = H_batch_2[:,L:L+Q] 
                # H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
                # H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
                # H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
                # H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
                # W = W.reshape(b,1,1,N_r_RF,N_r)
                # F_1 = F_1.reshape(b,1,1,N_t,1)
                # F_2 = F_2.reshape(b,1,1,N_t,1)
                # W = Pre_WY_DMRS(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2, W, F_1, F_2)


                if flag_DL == 'DL':
                    W,F_1,F_2=dec_Pre(
                            H_CE_1 = H_CE_1,  # b, Nc, N_r, N_t 用户1
                            H_CE_2 = H_CE_2,  # b, Nc, N_r, N_t 用户2
                            H_CF_1 = H_CF_1,  # b, Nc, N_r, N_t 用户1
                            H_CF_2 = H_CF_2,  # b, Nc, N_r, N_t 用户2
                        );
                    loss = -calculate_average_channel_capacity_OMA_woInt(H_batch_1[:,L],H_batch_2[:,L],F_1,F_2,W, SNR_dB)/val_num
                    loss_acc += loss.item()


                    H1 = H_batch_1[:,L:L+Q] 
                    H2 = H_batch_2[:,L:L+Q] 
                    H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
                    H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
                    H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
                    H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
                    W = W.reshape(b,1,1,N_r_RF,N_r)
                    F_1 = F_1.reshape(b,1,1,N_t,1)
                    F_2 = F_2.reshape(b,1,1,N_t,1)
                    W = Pre_WY_DMRS(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2, W, F_1, F_2) #b,Q,Nc,2,N_r
                elif flag_DL == 'PCA':
                    H1 = H_batch_1[:,L:L+Q] 
                    H2 = H_batch_2[:,L:L+Q] 
                    H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
                    H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
                    H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
                    H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
                    P_analog1,P_analog2,C_analog,C_digital,F_1, F_2, W = Pre_PCA_total_analog(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2)

                    loss = -calculate_average_channel_capacity_OMA_woInt(H_batch_1[:,L],H_batch_2[:,L],F_1.reshape(b,1,N_t,1),F_2.reshape(b,1,N_t,1),C_analog.reshape(b,1,N_r_RF,N_r), SNR_dB)/val_num
                    loss_acc += loss.item()



                R1 = calculate_average_channel_capacity_T(H1, F_1, W, SNR_dB)
                R2 = calculate_average_channel_capacity_T(H2, F_2, W, SNR_dB)
                R_acc = R_acc + (R1.mean().item() + R2.mean().item())/2/val_num

                

        return loss_acc,R_acc,loss_acc_CE,loss_acc_CF

    
    loss_SE,R,SGCS_CE,SGCS_CF = evaluate(0);
    print("SGCS_CE: ",-SGCS_CE,"    SGCS_CF: ",-SGCS_CF,"    SE: ",loss_SE,"    ZF_SE: ",R)
                


#### 开始训练

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2


SNR_dB = 10
v_max_km = 120
# train_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre,v_max_km_pre,ITERS=20_000,b=64,acc_num=1)

train_Precoding_WY_perfect(N,Nc,Nr,N_r_RF,Nt,N_t_RF,SNR_dB,v_max_km,ITERS=40_000,b=64,acc_num=1)

In [ ]:
# t = 0
# Nc = 100
# N = 2
# Nt = [1,8]
# Nr = [1,64]
# N_r_RF = 2
# N_t_RF = 2

# L = 8

# feedback_bits_pre = None
# v_max_km_pre = None

# feedback_bits = 1024
# SNR_dB = 10
# v_max_km = 120
# for L in [16]:
#     train_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre,v_max_km_pre,ITERS=20_000,b=64,acc_num=1)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for feedback_bits in [256]:
# for feedback_bits in [256,4096]:
    train_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km,feedback_bits_pre,v_max_km_pre,ITERS=10_000,b=64,acc_num=1)

#### 开始测试

##### DL

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for L in [1,2,4,8,16]:
    test_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for feedback_bits in [4,16,64,256,1024]:
    test_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km)

##### PCA

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for L in [1,2,4,8,16]:
    test_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km,flag_DL=flag_DL)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = 10
v_max_km = 120
for feedback_bits in [4,16,64,256,1024]:
    test_Precoding_WY(N,Nc,Nr,N_r_RF,Nt,N_r_RF,L,feedback_bits,SNR_dB,v_max_km,flag_DL=flag_DL)

### 端到端预编码

#### 网络

In [ ]:
class E2E_base(nn.Module):
    def __init__(
            self,
            Nc,     #子载波数
            N_r,    #基站接收天线数
            N_r_RF, #基站射频数
            N_t,    #用户发送天线数
            N_t_RF, #用户发送射频数
            L,      #导频OFDM符号数
            feedback_bits, # 反馈比特数
        ):
        super().__init__();

        
        dec_CE=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
        dec_CF=CF_WY(Nc=Nc,N_r=N_r,N_t=N_t,feedback_bits=feedback_bits).cuda()
        dec_Pre=Precoding_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF).cuda()

       



    
    def forward(
            self, 
            H_batch,  # b,T, Nc, N_r, N_t 用户1
        ):
        received_pilot = self.Pilot_trans(H_batch)
        H_hat = self.pilot2H(received_pilot)
        return H_hat

### 含噪语义分割

#### 网络

In [ ]:
class ConvBnLeakyRelu2d(nn.Module):
    # convolution
    # batch normalization
    # leaky relu
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1, dilation=1, groups=1):
        super(ConvBnLeakyRelu2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, dilation=dilation, groups=groups)
        self.bn   = nn.BatchNorm2d(out_channels)
    def forward(self, x):
        return F.leaky_relu(self.bn(self.conv(x)), negative_slope=0.2)


class MiniInception(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MiniInception, self).__init__()
        self.conv1_left  = ConvBnLeakyRelu2d(in_channels,   out_channels//2)
        self.conv1_right = ConvBnLeakyRelu2d(in_channels,   out_channels//2, padding=2, dilation=2)
        self.conv2_left  = ConvBnLeakyRelu2d(out_channels,  out_channels//2)
        self.conv2_right = ConvBnLeakyRelu2d(out_channels,  out_channels//2, padding=2, dilation=2)
        self.conv3_left  = ConvBnLeakyRelu2d(out_channels,  out_channels//2)
        self.conv3_right = ConvBnLeakyRelu2d(out_channels,  out_channels//2, padding=2, dilation=2)
    def forward(self,x):
        x = torch.cat((self.conv1_left(x), self.conv1_right(x)), dim=1)
        x = torch.cat((self.conv2_left(x), self.conv2_right(x)), dim=1)
        x = torch.cat((self.conv3_left(x), self.conv3_right(x)), dim=1)
        return x


class MFNet_Enc_rgb_noise(nn.Module):

    def __init__(self, Q_ini):
        super(MFNet_Enc_rgb_noise, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        # 压缩后的每个样本维度为[2，30，40]我们正好可以设置为单流2个实虚部组成的2、30个子载波和40个OFDM符号
        compress_ch = [192,96,96,2*Q_ini]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.conv1_rgb   = ConvBnLeakyRelu2d(3, rgb_ch[0])
        self.conv2_1_rgb = ConvBnLeakyRelu2d(rgb_ch[0], rgb_ch[1])
        self.conv2_2_rgb = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[1])
        self.conv3_1_rgb = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[2])
        self.conv3_2_rgb = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[2])
        self.conv4_rgb   = MiniInception(rgb_ch[2], rgb_ch[3])
        self.conv5_rgb   = MiniInception(rgb_ch[3], rgb_ch[4])
        self.compress_1_rgb = ConvBnLeakyRelu2d(compress_ch[0], compress_ch[1])
        self.compress_2_rgb = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[2])
        self.compress_3_rgb = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[3])
    def forward(self, x):
        # split data into RGB and INF
        x_rgb = x[:,:3]

        # encode
        x_rgb    = self.conv1_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb, kernel_size=2, stride=2) # pool1
        x_rgb    = self.conv2_1_rgb(x_rgb)
        x_rgb_p2 = self.conv2_2_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p2, kernel_size=2, stride=2) # pool2
        x_rgb    = self.conv3_1_rgb(x_rgb)
        x_rgb_p3 = self.conv3_2_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p3, kernel_size=2, stride=2) # pool3
        x_rgb_p4 = self.conv4_rgb(x_rgb)
        x_rgb    = F.max_pool2d(x_rgb_p4, kernel_size=2, stride=2) # pool4 #[batch,2,30,40]
        x_rgb    = self.conv5_rgb(x_rgb)  #[batch,96,30,40]
        x_rgb    = F.max_pool2d(x_rgb, kernel_size=(3,4), stride=(3,4)) # pool5  

        x_rgb    = self.compress_1_rgb(x_rgb)  
        x_rgb    = self.compress_2_rgb(x_rgb)
        x_rgb    = self.compress_3_rgb(x_rgb) #[batch,2*Q_ini,10,10]
        return x_rgb


class MFNet_Enc_inf_noise(nn.Module):

    def __init__(self, Q_ini):
        super(MFNet_Enc_inf_noise, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        # 压缩后的每个样本维度为[2，30，40]我们正好可以设置为单流2个实虚部组成的2、30个子载波和40个OFDM符号
        compress_ch = [192,96,96,2*Q_ini]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.conv1_inf   = ConvBnLeakyRelu2d(1, rgb_ch[0])
        self.conv2_1_inf = ConvBnLeakyRelu2d(rgb_ch[0], rgb_ch[1])
        self.conv2_2_inf = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[1])
        self.conv3_1_inf = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[2])
        self.conv3_2_inf = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[2])
        self.conv4_inf   = MiniInception(rgb_ch[2], rgb_ch[3])
        self.conv5_inf   = MiniInception(rgb_ch[3], rgb_ch[4])
        self.compress_1_inf = ConvBnLeakyRelu2d(compress_ch[0], compress_ch[1])
        self.compress_2_inf = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[2])
        self.compress_3_inf = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[3])
    def forward(self, x):
        # split data into RGB and INF
        x_inf = x[:,3:]

        # encode
        x_inf    = self.conv1_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf, kernel_size=2, stride=2) # pool1
        x_inf    = self.conv2_1_inf(x_inf)
        x_inf_p2 = self.conv2_2_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p2, kernel_size=2, stride=2) # pool2
        x_inf    = self.conv3_1_inf(x_inf)
        x_inf_p3 = self.conv3_2_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p3, kernel_size=2, stride=2) # pool3
        x_inf_p4 = self.conv4_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf_p4, kernel_size=2, stride=2) # pool4 #[batch,96,30,40]
        x_inf    = self.conv5_inf(x_inf)
        x_inf    = F.max_pool2d(x_inf, kernel_size=(3,4), stride=(3,4)) # pool5  

        x_inf    = self.compress_1_inf(x_inf)  
        x_inf    = self.compress_2_inf(x_inf)  
        x_inf    = self.compress_3_inf(x_inf)    #[batch,2*Q_ini,10,10]
        return x_inf

class MFNet_Dec_noise(nn.Module):

    def __init__(self, n_class,Q_ini):
        super(MFNet_Dec_noise, self).__init__()
        rgb_ch = [16,48,48,96,192]
        # self.pho = 2
        # inf_ch = [16,48,48,96,96]
        # 压缩后的每个样本维度为 #[2*Q_ini,10,10]我们keyi
        compress_ch = [192,96,96,2*Q_ini]
        self.compress_ch = compress_ch
        out_dim = 2400

        self.decompress_3 = ConvBnLeakyRelu2d(compress_ch[3], compress_ch[2])
        self.decompress_2 = ConvBnLeakyRelu2d(compress_ch[2], compress_ch[1])
        self.decompress_1 = ConvBnLeakyRelu2d(compress_ch[1], compress_ch[0])


        self.decode5     = ConvBnLeakyRelu2d(rgb_ch[4], rgb_ch[3])
        self.decode4     = ConvBnLeakyRelu2d(rgb_ch[3], rgb_ch[2])
        self.decode3     = ConvBnLeakyRelu2d(rgb_ch[2], rgb_ch[1])
        self.decode2     = ConvBnLeakyRelu2d(rgb_ch[1], rgb_ch[0])
        self.decode1     = ConvBnLeakyRelu2d(rgb_ch[0], n_class)
    def forward(self, x):
        #[batch,8,30,40]
        # x = self.decompress_4(x)
        # x = x.reshape(-1,8,self.pho,15,20).permute(0,2,1,3,4)
        # x = x.reshape(-1,self.pho*8,15,20)
        # x = self.decompress_4(x.permute(0,1,3,2)).permute(0,1,3,2)
        x = self.decompress_3(x)
        x = self.decompress_2(x)
        x = self.decompress_1(x) #[batch,96,30,40]

        # decode
        x = F.interpolate(x, scale_factor=(3,4), mode='nearest')# unpool5
        x = self.decode5(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool4
        x = self.decode4(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool3
        x = self.decode3(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool2
        x = self.decode2(x)
        x = F.upsample(x, scale_factor=2, mode='nearest') # unpool1
        x = self.decode1(x)

        return x

#我这里Q_ini是假设不DMRS的导频开销数，实际的Q需要再乘上插值的系数，为了方便考虑就不完全模拟插值的过程了直接在Q_ini这个维度传输吧
class MFNet_noise(nn.Module):

    def __init__(self, n_class, Q_ini):
        super(MFNet_noise, self).__init__()
        self.enc_rgb = MFNet_Enc_rgb_noise(Q_ini)
        self.enc_inf = MFNet_Enc_inf_noise(Q_ini)
        self.dec = MFNet_Dec_noise(n_class,Q_ini)
        self.Q_ini = Q_ini
        
        

    def forward(self, x, snr_min, snr_max):
        b = x.shape[0]
        Q_ini = self.Q_ini
        x_rgb = self.enc_rgb(x)  #[batch,2*Q_ini,10,10]
        x_inf = self.enc_inf(x)  #[batch,2*Q_ini,10,10]


        x_rgb = x_rgb[:,0:Q_ini,:,:] + 1j*x_rgb[:,Q_ini:2*Q_ini,:,:] #[batch,Q_ini,10,10]
        norm_factor = torch.sqrt(torch.sum(torch.abs(x_rgb)**2, dim=(-3,-2, -1), keepdim=True))
        x_rgb = x_rgb / norm_factor * torch.sqrt(torch.tensor(100*Q_ini, dtype=x.dtype, device=x.device))

        x_inf = x_inf[:,0:Q_ini,:,:] + 1j*x_inf[:,Q_ini:2*Q_ini,:,:] #[batch,Q_ini,10,10]
        norm_factor = torch.sqrt(torch.sum(torch.abs(x_inf)**2, dim=(-3,-2, -1), keepdim=True))
        x_inf = x_inf / norm_factor * torch.sqrt(torch.tensor(100*Q_ini, dtype=x.dtype, device=x.device))

        ########## 接下来需要加个噪声，每个样本各自加不同的噪声，两个用户也加不同的，给x_rgb和x_inf各自加
        snr1 = snr_min + (snr_max - snr_min) * torch.rand(b).cuda()  # [b]
        snr2 = snr_min + (snr_max - snr_min) * torch.rand(b).cuda()  # [b]
        snr1_linear = 10**(snr1 / 10.0) # [b]
        snr2_linear = 10**(snr2 / 10.0) # [b]
        # 已知信号功率为1，根据SNR计算噪声功率
        noise_power_for_x1 = 1.0 / snr1_linear
        noise_power_for_x2 = 1.0 / snr2_linear
        # 为每个样本生成复噪声
        # 噪声为复高斯噪声：real和imag独立且都为N(0, noise_power/2)
        noise_x1_real = torch.randn_like(x_rgb.real) * torch.sqrt(noise_power_for_x1[:, None, None, None] / 2.0)
        noise_x1_imag = torch.randn_like(x_rgb.imag) * torch.sqrt(noise_power_for_x1[:, None, None, None] / 2.0)
        noise_x1 = noise_x1_real + 1j * noise_x1_imag
        noise_x2_real = torch.randn_like(x_inf.real) * torch.sqrt(noise_power_for_x2[:, None, None, None] / 2.0)
        noise_x2_imag = torch.randn_like(x_inf.imag) * torch.sqrt(noise_power_for_x2[:, None, None, None] / 2.0)
        noise_x2 = noise_x2_real + 1j * noise_x2_imag
        x_rgb = x_rgb + noise_x1
        x_inf = x_inf + noise_x2

        x = x_rgb + x_inf        #[batch,Q_ini,10,10]
        norm_factor = torch.sqrt(torch.sum(torch.abs(x)**2, dim=(-3,-2, -1), keepdim=True))
        x = x / norm_factor * torch.sqrt(torch.tensor(100*Q_ini, dtype=x.dtype, device=x.device))
        x=torch.cat([x.real, x.imag], dim=1);  # batch,2*Q_ini,10,10
        
        x = self.dec(x)
        return x


In [ ]:
# coding:utf-8
import os
import argparse
import time

import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

from util.MF_dataset import MF_dataset
from util.util import calculate_accuracy, calculate_result
from util.augmentation import RandomFlip, RandomCrop, RandomCropOut, RandomBrightness, RandomNoise
from model import MFNet, SegNet

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
# config
n_class   = 9
data_dir  = './ir_seg_dataset/'
model_dir = 'weights/'
augmentation_methods = [
    RandomFlip(prob=0.5),
    RandomCrop(crop_rate=0.1, prob=1.0), 
    # RandomCropOut(crop_rate=0.2, prob=1.0),
    # RandomBrightness(bright_range=0.15, prob=0.9),
    # RandomNoise(noise_range=5, prob=0.9),
]
lr_start  = 0.01
lr_decay  = 0.95


def train(args, epo, model, train_loader, optimizer, snr_min, snr_max):

    lr_this_epo = lr_start * lr_decay**(epo-1)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr_this_epo

    loss_avg = 0.
    acc_avg  = 0.
    start_t = t = time.time()
    model.train()

    for it, (images, labels, names) in enumerate(train_loader):
        images = Variable(images).cuda(args.gpu) 
        labels = Variable(labels).cuda(args.gpu)
        # if args.gpu >= 0:
        #     images = images.cuda(args.gpu)
        #     labels = labels.cuda(args.gpu)

        optimizer.zero_grad()
        logits = model(images, snr_min, snr_max)
        # print(labels.min(), labels.max())

        loss = F.cross_entropy(logits, labels)
        loss.backward()
        optimizer.step()

        acc = calculate_accuracy(logits, labels)
        loss_avg += float(loss.item())
        acc_avg  += float(acc)

        # cur_t = time.time()
        # if cur_t-t > 5:
        #     print('|- epo %s/%s. train iter %s/%s. %.2f img/sec loss: %.4f, acc: %.4f' \
        #         % (epo, args.epoch_max, it+1, train_loader.n_iter, (it+1)*args.batch_size/(cur_t-start_t), float(loss), float(acc)))
        #     t += 5

    content = '| epo:%s/%s lr:%.4f train_loss_avg:%.4f train_acc_avg:%.4f ' \
            % (epo, args.epoch_max, lr_this_epo, loss_avg/train_loader.n_iter, acc_avg/train_loader.n_iter)
    print(content)
    # with open(log_file, 'a') as appender:
    #     appender.write(content)


def validation(args, epo, model, val_loader, snr_min, snr_max):

    loss_avg = 0.
    acc_avg  = 0.
    start_t = time.time()
    model.eval()

    cf = np.zeros((n_class, n_class))

    with torch.no_grad():
        for it, (images, labels, names) in enumerate(val_loader):
            images = Variable(images)
            labels = Variable(labels)
            if args.gpu >= 0:
                images = images.cuda(args.gpu)
                labels = labels.cuda(args.gpu)

            logits = model(images, snr_min, snr_max)
            loss = F.cross_entropy(logits, labels)
            acc = calculate_accuracy(logits, labels)
            loss_avg += float(loss)
            acc_avg  += float(acc)

            # cur_t = time.time()
            # print('|- epo %s/%s. val iter %s/%s. %.2f img/sec loss: %.4f, acc: %.4f' \
            #         % (epo, args.epoch_max, it+1, val_loader.n_iter, (it+1)*args.batch_size/(cur_t-start_t), float(loss), float(acc)))

            predictions = logits.argmax(1)
            for gtcid in range(n_class): 
                for pcid in range(n_class):
                    gt_mask      = labels == gtcid 
                    pred_mask    = predictions == pcid
                    intersection = gt_mask * pred_mask
                    cf[gtcid, pcid] += int(intersection.sum())
        overall_acc, acc, IoU = calculate_result(cf)

    content = '| val_loss_avg:%.4f val_acc_avg:%.4f' \
            % (loss_avg/val_loader.n_iter, acc_avg/val_loader.n_iter)
    print(content)
    content = '| class accuracy avg:%.4f class IoU avg:%.4f\n' \
                % (acc.mean(), IoU.mean())
    print(content)
    
    # with open(log_file, 'a') as appender:
    #     appender.write(content)


import os
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

from ipdb import set_trace as st
class MF_dataset(Dataset):

    def __init__(self, data_dir, split, have_label, input_h=480, input_w=640 ,transform=[]):
        super(MF_dataset, self).__init__()

        # assert split in ['train', 'val', 'test'], 'split must be "train"|"val"|"test"'

        with open(os.path.join(data_dir, split+'.txt'), 'r') as f:
            self.names = [name.strip() for name in f.readlines()]

        self.data_dir  = data_dir
        self.split     = split
        self.input_h   = input_h
        self.input_w   = input_w
        self.transform = transform
        self.is_train  = have_label
        self.n_data    = len(self.names)


    def read_image(self, name, folder):
        file_path = os.path.join(self.data_dir, '%s/%s.png' % (folder, name))
        image     = np.asarray(Image.open(file_path)) # (w,h,c)
        image = np.require(image, dtype='f4', requirements=['O', 'W'])
        image.flags.writeable = True
        return image

    def get_train_item(self, index):
        name  = self.names[index]
        image = self.read_image(name, 'images')
        label = self.read_image(name, 'labels')

        for func in self.transform:
            image, label = func(image, label)

        image = np.asarray(Image.fromarray(np.uint8(image)).resize((self.input_w, self.input_h)), dtype=np.float32).transpose((2,0,1))/255
        # label = np.asarray(Image.fromarray(label).resize((self.input_w, self.input_h)), dtype=np.int64)
        label = np.asarray(Image.fromarray(label).resize((self.input_w, self.input_h), Image.NEAREST), dtype=np.int64)

        return torch.tensor(image), torch.tensor(label), name

    def get_test_item(self, index):
        name  = self.names[index]
        image = self.read_image(name, 'images')
        image = np.asarray(Image.fromarray(image).resize((self.input_w, self.input_h)), dtype=np.float32).transpose((2,0,1))/255

        return torch.tensor(image), name


    def __getitem__(self, index):

        if self.is_train is True:
            return self.get_train_item(index)
        else: 
            return self.get_test_item (index)

    def __len__(self):
        return self.n_data

In [ ]:
def train_Sem_noise(Q_ini,snr_max,snr_min):
    import sys
    sys.argv = ['run.py']
    parser = argparse.ArgumentParser(description='Train MFNet with pytorch')
    parser.add_argument('--model_name',  '-M',  type=str, default='MFNet_noise')
    parser.add_argument('--batch_size',  '-B',  type=int, default=8)

    parser.add_argument('--epoch_max' ,  '-E',  type=int, default=100)
    # parser.add_argument('--epoch_max' ,  '-E',  type=int, default=2)

    parser.add_argument('--epoch_from',  '-EF', type=int, default=1)
    parser.add_argument('--gpu',         '-G',  type=int, default=0)
    parser.add_argument('--num_workers', '-j',  type=int, default=0)
    args = parser.parse_args()
    model_dir = 'weights/'
    model_dir = os.path.join(model_dir, args.model_name)
    os.makedirs(model_dir, exist_ok=True)
    checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1_Qini'+str(Q_ini)+'.pth')
    checkpoint_optim_file = os.path.join(model_dir, 'tmp_pretrain1_Qini'+str(Q_ini)+'.optim')
    final_model_file      = os.path.join(model_dir, 'final_pretrain1_Qini'+str(Q_ini)+'.pth')
    log_file              = os.path.join(model_dir, 'log_pretrain1_Qini'+str(Q_ini)+'.txt')

    print('| training %s on GPU #%d with pytorch' % (args.model_name, args.gpu))
    print('| from epoch %d / %s' % (args.epoch_from, args.epoch_max))
    print('| model will be saved in: %s' % model_dir)


    model = eval(args.model_name)(n_class=n_class,Q_ini=Q_ini)
    if args.gpu >= 0: model.cuda(args.gpu)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr_start, momentum=0.9, weight_decay=0.0005) 
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr_start)

    if args.epoch_from > 1:
        print('| loading checkpoint file %s... ' % checkpoint_model_file, end='')
        model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
        optimizer.load_state_dict(torch.load(checkpoint_optim_file))
        print('done!')

    train_dataset = MF_dataset(data_dir, 'train', have_label=True, transform=augmentation_methods)
    val_dataset  = MF_dataset(data_dir, 'val', have_label=True)

    train_loader  = DataLoader(
        dataset     = train_dataset,
        batch_size  = args.batch_size,
        shuffle     = True,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = True
    )
    val_loader  = DataLoader(
        dataset     = val_dataset,
        batch_size  = args.batch_size,
        shuffle     = False,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = False
    )
    train_loader.n_iter = len(train_loader)
    val_loader.n_iter   = len(val_loader)

    for epo in (range(args.epoch_from, args.epoch_max+1)):
        print('\n| epo #%s begin...' % epo)

        train(args,epo, model, train_loader, optimizer, snr_min, snr_max)
        validation(args,epo, model, val_loader, snr_min, snr_max)

        # save check point model
        # print('| saving check point model file... ', end='')
        torch.save(model.state_dict(), checkpoint_model_file)
        torch.save(optimizer.state_dict(), checkpoint_optim_file)
        # print('done!')
    if os.path.exists(final_model_file):
        os.remove(final_model_file) # 删除已存在的文件
    os.rename(checkpoint_model_file, final_model_file)
    

#### 训练

In [ ]:
Q_ini = 12
snr_max = 0
snr_min = -20
for Q_ini in [1,2,4,6,9,12]:  #由于插入了导频，实际上开销应当是Q_ini*4/3
    train_Sem_noise(Q_ini,snr_max,snr_min)


In [ ]:
Q_ini = 12
snr_max = 0
snr_min = -20
for Q_ini in [3,5]:  #由于插入了导频，实际上开销应当是Q_ini*4/3
    train_Sem_noise(Q_ini,snr_max,snr_min)

#### 测试

##### demo

In [ ]:
snr_max = 0
snr_min = 0
Q_ini = 12


from util.util import visualize
import sys

sys.argv = ['run.py']


parser = argparse.ArgumentParser(description='Run MFNet demo with pytorch')
parser.add_argument('--model_name', '-M',  type=str, default='MFNet_noise')
parser.add_argument('--gpu',        '-G',  type=int, default=0)
args = parser.parse_args()
model_dir = 'weights/'
model_dir = os.path.join(model_dir, args.model_name)

checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1_Qini'+str(Q_ini)+'.pth')
final_model_file      = os.path.join(model_dir, 'final_pretrain1_Qini'+str(Q_ini)+'.pth')
print('| running %s demo on GPU #%d with pytorch' % (args.model_name, args.gpu))




model = eval(args.model_name)(n_class=n_class,Q_ini=Q_ini)
if args.gpu >= 0: model.cuda(args.gpu)
if os.path.exists(final_model_file):
    model.load_state_dict(torch.load(final_model_file, map_location='cuda:0'))
elif os.path.exists(checkpoint_model_file):
    model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
else:
    raise Exception('| model file do not exists in %s' % model_dir)
print('| model loaded!')

files = os.listdir('image')
images = []
fpath  = []
for file in files:
    if file[-3:] != 'png': continue
    fpath.append('image/'+file)
    images.append( np.asarray(Image.open('image/'+file)) )
images = np.asarray(images, dtype=np.float32).transpose((0,3,1,2))/255.
images = Variable(torch.tensor(images))
if args.gpu >= 0: images = images.cuda(args.gpu)

model.eval()
with torch.no_grad():
    logits = model(images, snr_min, snr_max)
    predictions = logits.argmax(1)
    visualize(fpath, predictions)

##### 性能指标

In [ ]:


def test_noise_sem(Q_ini,snr_max,snr_min):

    from util.util import calculate_accuracy, calculate_result
    import sys

    sys.argv = ['run.py']


    cf = np.zeros((n_class, n_class))


    parser = argparse.ArgumentParser(description='Test MFNet with pytorch')
    parser.add_argument('--model_name', '-M',  type=str, default='MFNet_noise')
    parser.add_argument('--batch_size',  '-B',  type=int, default=16)
    parser.add_argument('--gpu',        '-G',  type=int, default=0)
    parser.add_argument('--num_workers', '-j',  type=int, default=0)



    args = parser.parse_args()
    model_dir = 'weights/'
    model_dir = os.path.join(model_dir, args.model_name)

    checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1_Qini'+str(Q_ini)+'.pth')
    final_model_file      = os.path.join(model_dir, 'final_pretrain1_Qini'+str(Q_ini)+'.pth')
    # print('| testing %s on GPU #%d with pytorch' % (args.model_name, args.gpu))

    model = eval(args.model_name)(n_class=n_class,Q_ini=Q_ini)
    if args.gpu >= 0: model.cuda(args.gpu)
    if os.path.exists(final_model_file):
        model.load_state_dict(torch.load(final_model_file, map_location='cuda:0'))
    elif os.path.exists(checkpoint_model_file):
        model.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
    else:
        raise Exception('| model file do not exists in %s' % model_dir)
    # print('| model loaded!')


    test_dataset  = MF_dataset(data_dir, 'test', have_label=True)
    test_loader  = DataLoader(
        dataset     = test_dataset,
        batch_size  = args.batch_size,
        shuffle     = False,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = False
    )
    test_loader.n_iter = len(test_loader)

    loss_avg = 0.
    acc_avg  = 0.
    model.eval()
    with torch.no_grad():
        for it, (images, labels, names) in enumerate(test_loader):
            images = Variable(images)
            labels = Variable(labels)
            if args.gpu >= 0:
                images = images.cuda(args.gpu)
                labels = labels.cuda(args.gpu)

            logits = model(images, snr_min, snr_max)
            loss = F.cross_entropy(logits, labels)
            acc = calculate_accuracy(logits, labels)
            loss_avg += float(loss)
            acc_avg  += float(acc)

            # print('|- test iter %s/%s. loss: %.4f, acc: %.4f' \
            #         % (it+1, test_loader.n_iter, float(loss), float(acc)))

            predictions = logits.argmax(1)
            for gtcid in range(n_class): 
                for pcid in range(n_class):
                    gt_mask      = labels == gtcid 
                    pred_mask    = predictions == pcid
                    intersection = gt_mask * pred_mask
                    cf[gtcid, pcid] += int(intersection.sum())

    overall_acc, acc, IoU = calculate_result(cf)

    # print('| overall accuracy:', overall_acc)
    # print('| accuracy of each class:', acc)
    # print('| class accuracy avg:', acc.mean())
    # print('| IoU:', IoU)
    print('| class IoU avg:', IoU.mean())



In [ ]:
snr_max = 0
snr_min = 0
Q_ini = 12
for Q_ini in [1,2,3,4,5]: #由于插入了导频，实际上开销应当是Q_ini*4/3
    test_noise_sem(Q_ini,snr_max,snr_min)

### 分离模块连起来

In [ ]:
def test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_DL='DL',flag_csi='imperfect'):
    # N代表路径数
    n_class   = 9
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;
    rate1 = 4
    rate2 = 2


    model_name_CE = 'CE_WY'+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_CF = 'CF_WY'+'_bits' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_Pre = 'Precoding_WY' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    if flag_csi == 'perfect':
        model_name_Pre = 'Precoding_WY_perfect' + '_v' + str(v_max_km)

    dec_CE=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
    dec_CF=CF_WY(Nc=Nc,N_r=N_r,N_t=N_t,feedback_bits=feedback_bits).cuda()
    dec_Pre=Precoding_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF).cuda()
    Q=Q_ini
    T = L+Q_ini
    load_path_CE=save_path+"CE_WY"+"/";
    load_path_CF=save_path+"CF_WY"+"/";
    load_path_Pre=save_path+"Precoding_WY"+"/";

    dec_dict = dec_CE.state_dict()
    dec_load_path=load_path_CE+model_name_CE+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CE.load_state_dict(dec_dict);
    dec_CE.eval()



    dec_dict = dec_CF.state_dict()
    dec_load_path=load_path_CF+model_name_CF+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CF.load_state_dict(dec_dict);
    dec_CF.eval()


    

    

    dec_dict = dec_Pre.state_dict()
    dec_load_path=load_path_Pre+model_name_Pre+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_Pre.load_state_dict(dec_dict);
    dec_Pre.eval()

    
    from util.util import calculate_accuracy, calculate_result
    import sys
    sys.argv = ['run.py']
    cf = np.zeros((n_class, n_class))
    parser = argparse.ArgumentParser(description='Test MFNet with pytorch')
    parser.add_argument('--model_name', '-M',  type=str, default='MFNet_noise')
    parser.add_argument('--batch_size',  '-B',  type=int, default=16)
    parser.add_argument('--gpu',        '-G',  type=int, default=0)
    parser.add_argument('--num_workers', '-j',  type=int, default=0)
    args = parser.parse_args()
    model_dir = 'weights/'
    model_dir = os.path.join(model_dir, args.model_name)
    checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1_Qini'+str(Q_ini)+'.pth')
    final_model_file      = os.path.join(model_dir, 'final_pretrain1_Qini'+str(Q_ini)+'.pth')
    # print('| testing %s on GPU #%d with pytorch' % (args.model_name, args.gpu))
    dec_sem = eval(args.model_name)(n_class=n_class,Q_ini=Q_ini)
    if args.gpu >= 0: dec_sem.cuda(args.gpu)
    if os.path.exists(final_model_file):
        dec_sem.load_state_dict(torch.load(final_model_file, map_location='cuda:0'))
    elif os.path.exists(checkpoint_model_file):
        dec_sem.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
    else:
        raise Exception('| model file do not exists in %s' % model_dir)

    
            
    test_dataset  = MF_dataset(data_dir, 'test', have_label=True)
    test_loader  = DataLoader(
        dataset     = test_dataset,
        batch_size  = args.batch_size,
        shuffle     = False,
        num_workers = args.num_workers,
        pin_memory  = True,
        drop_last   = False
    )
    test_loader.n_iter = len(test_loader)

    loss_avg = 0.
    acc_avg  = 0.
    dec_sem.eval()
    with torch.no_grad():
        for it, (images, labels, names) in enumerate(test_loader):
            images = Variable(images)
            labels = Variable(labels)
            if args.gpu >= 0:
                images = images.cuda(args.gpu)
                labels = labels.cuda(args.gpu)

            b = images.shape[0]
            
            alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
            H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

            for i in range(T):
                t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
                H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
                H_batch[:,i] = H
            H_batch_1 = H_batch[0:b]
            H_batch_2 = H_batch[b:2*b]

            H_CE=dec_CE(
                H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
            );
            H_CF=dec_CF(
                    H = H_CE,  # b,T, Nc, N_r, N_t 用户1
                );

            H_CE_1 = H_CE[0:b]
            H_CE_2 = H_CE[b:2*b]
            H_CF_1 = H_CF[0:b]
            H_CF_2 = H_CF[b:2*b]

            if flag_csi == 'perfect':
                H_CE_1 = H_batch[0:b,L]
                H_CE_2 = H_batch[b:2*b,L]
                H_CF_1 = H_batch[0:b,L]
                H_CF_2 = H_batch[b:2*b,L]
            elif flag_csi == 'imperfect':
                pass

            # loss_CE = SGCS_H(H_batch[:,L],H_CE)

            # loss_CF = SGCS_H(H_batch[:,L],H_CF)
            # print(loss_CE,loss_CF)
    

            if flag_DL == 'DL':
                W,F_1,F_2=dec_Pre(
                        H_CE_1 = H_CE_1,  # b, Nc, N_r, N_t 用户1
                        H_CE_2 = H_CE_2,  # b, Nc, N_r, N_t 用户2
                        H_CF_1 = H_CF_1,  # b, Nc, N_r, N_t 用户1
                        H_CF_2 = H_CF_2,  # b, Nc, N_r, N_t 用户2
                    );


                H1 = H_batch_1[:,L:L+Q] 
                H2 = H_batch_2[:,L:L+Q] 
                H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
                H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
                H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
                H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
                W = W.reshape(b,1,1,N_r_RF,N_r)
                F_1 = F_1.reshape(b,1,1,N_t,1)
                F_2 = F_2.reshape(b,1,1,N_t,1)
                W = Pre_WY_DMRS(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2, W, F_1, F_2) #b,Q,Nc,2,N_r
            elif flag_DL == 'PCA':
                H1 = H_batch_1[:,L:L+Q] 
                H2 = H_batch_2[:,L:L+Q] 
                H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
                H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
                H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
                H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
                P_analog1,P_analog2,C_analog,C_digital,F_1, F_2, W = Pre_PCA_total_analog(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2)


            norm_factor_W = torch.sqrt(torch.sum(torch.abs(W)**2, dim=(-4,-3,-2, -1), keepdim=True))/torch.sqrt(torch.tensor(Q*Nc*N_r_RF, dtype=H.dtype, device=H.device))#1,1,1,1,1

            
            # norm_factor_W = 1

            
            # Calculate effective channel
            H_eff_1 = W @ H1 @ F_1  #H [batch,Q, Nc,N_r_RF,1]
            H_eff_2 = W @ H2 @ F_2  #H [batch,Q, Nc,N_r_RF,1]

            # H_eff_1[:,:,:,0,:] = 1
            # H_eff_1[:,:,:,1,:] = 0

            # H_eff_2[:,:,:,0,:] = 0
            # H_eff_2[:,:,:,1,:] = 1

            ################################################################

            x = images

            x_rgb = dec_sem.enc_rgb(x)  #[batch,2*Q_ini,10,10]
            x_inf = dec_sem.enc_inf(x)  #[batch,2*Q_ini,10,10]


            x_rgb = x_rgb[:,0:Q_ini,:,:] + 1j*x_rgb[:,Q_ini:2*Q_ini,:,:] #[batch,Q_ini,10,10]
            norm_factor = torch.sqrt(torch.sum(torch.abs(x_rgb)**2, dim=(-3,-2, -1), keepdim=True))
            x_rgb = x_rgb / norm_factor * torch.sqrt(torch.tensor(Q*Nc*1, dtype=x.dtype, device=x.device))

            x1 = x_rgb.reshape(b,Q,Nc,1,1) #b,Nc,Q,1,1
            x1 = H_eff_1 @ x1           #b,Q,Nc,N_r_RF,1
            # x_rgb = x_rgb[:,:,:,0:1,:].reshape(b,Q,10,10)

            

            x_inf = x_inf[:,0:Q_ini,:,:] + 1j*x_inf[:,Q_ini:2*Q_ini,:,:] #[batch,Q_ini,10,10]
            norm_factor = torch.sqrt(torch.sum(torch.abs(x_inf)**2, dim=(-3,-2, -1), keepdim=True))
            x_inf = x_inf / norm_factor * torch.sqrt(torch.tensor(Q*Nc*1, dtype=x.dtype, device=x.device))

            x2 = x_inf.reshape(b,Q,Nc,1,1) #b,Nc,Q,1,1
            x2 = H_eff_2 @ x2           #b,Q,Nc,N_r_RF,1
            # x_inf = x_inf[:,:,:,1:2,:].reshape(b,Q,10,10)

            sigma = 10**(-SNR_dB/10)
            n = (torch.randn(b,Q,Nc,2,1).cuda() + 1j*torch.randn(b,Q,Nc,2,1).cuda())/sqrt(2)*sqrt(sigma)*norm_factor_W

            x = x1 + x2 + n # b,Q,Nc,N_r_RF,1
            x = x[:,:,:,0:1,:] + x[:,:,:,1:2,:]
            x = x.reshape(b,Q,10,10)

            norm_factor = torch.sqrt(torch.sum(torch.abs(x)**2, dim=(-3,-2, -1), keepdim=True))
            x = x / norm_factor * torch.sqrt(torch.tensor(Q*Nc*1, dtype=x.dtype, device=x.device))
            x=torch.cat([x.real, x.imag], dim=1);  # batch,2*Q_ini,10,10
            logits = dec_sem.dec(x)

            
            loss = F.cross_entropy(logits, labels)
            acc = calculate_accuracy(logits, labels)
            loss_avg += float(loss)
            acc_avg  += float(acc)

            # print('|- test iter %s/%s. loss: %.4f, acc: %.4f' \
            #         % (it+1, test_loader.n_iter, float(loss), float(acc)))

            predictions = logits.argmax(1)
            for gtcid in range(n_class): 
                for pcid in range(n_class):
                    gt_mask      = labels == gtcid 
                    pred_mask    = predictions == pcid
                    intersection = gt_mask * pred_mask
                    cf[gtcid, pcid] += int(intersection.sum())

    overall_acc, acc, IoU = calculate_result(cf)

    # print('| overall accuracy:', overall_acc)
    # print('| accuracy of each class:', acc)
    # print('| class accuracy avg:', acc.mean())
    # print('| IoU:', IoU)
    print('| class IoU avg:', IoU.mean())

                
                


In [ ]:
def demo_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_DL='DL',flag_csi='imperfect'):
    # N代表路径数
    
    
    N_t = Nt[0]*Nt[1]
    N_r = Nr[0]*Nr[1]
    device=0;
    lr=1e-4;
    log_interval=100; # 每100个输出保存一下许训练记录
    save_interval=10*log_interval; # 每1000个测试一下保存记录
    iter_begin=0;   #从第0个开始训练 如果不是0就读取之前训练过的check points
    save_path="./weights/";
    best_loss=0;
    rate1 = 4
    rate2 = 2


    model_name_CE = 'CE_WY'+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_CF = 'CF_WY'+'_bits' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    model_name_Pre = 'Precoding_WY' + str(feedback_bits)+'_L' + str(L)+'_v' + str(v_max_km)
    if flag_csi == 'perfect':
        model_name_Pre = 'Precoding_WY_perfect' + '_v' + str(v_max_km)

    dec_CE=CE_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF,L=L).cuda()
    dec_CF=CF_WY(Nc=Nc,N_r=N_r,N_t=N_t,feedback_bits=feedback_bits).cuda()
    dec_Pre=Precoding_WY(Nc=Nc,N_r=N_r,N_r_RF=N_r_RF,N_t=N_t,N_t_RF=N_t_RF).cuda()
    Q=Q_ini
    T = L+Q_ini
    load_path_CE=save_path+"CE_WY"+"/";
    load_path_CF=save_path+"CF_WY"+"/";
    load_path_Pre=save_path+"Precoding_WY"+"/";

    dec_dict = dec_CE.state_dict()
    dec_load_path=load_path_CE+model_name_CE+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CE.load_state_dict(dec_dict);
    dec_CE.eval()



    dec_dict = dec_CF.state_dict()
    dec_load_path=load_path_CF+model_name_CF+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_CF.load_state_dict(dec_dict);
    dec_CF.eval()


    

    

    dec_dict = dec_Pre.state_dict()
    dec_load_path=load_path_Pre+model_name_Pre+"_model.pth.tar";
    ckpt=torch.load(dec_load_path, map_location="cuda:"+str(device));
    pretrained_dict=ckpt['state_dict'];
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in dec_dict}
    dec_dict.update(pretrained_dict);
    dec_Pre.load_state_dict(dec_dict);
    dec_Pre.eval()

    
    from util.util import calculate_accuracy, calculate_result
    import sys
    sys.argv = ['run.py']
    cf = np.zeros((n_class, n_class))
    parser = argparse.ArgumentParser(description='Test MFNet with pytorch')
    parser.add_argument('--model_name', '-M',  type=str, default='MFNet_noise')
    parser.add_argument('--batch_size',  '-B',  type=int, default=16)
    parser.add_argument('--gpu',        '-G',  type=int, default=0)
    parser.add_argument('--num_workers', '-j',  type=int, default=0)
    args = parser.parse_args()
    model_dir = 'weights/'
    model_dir = os.path.join(model_dir, args.model_name)
    checkpoint_model_file = os.path.join(model_dir, 'tmp_pretrain1_Qini'+str(Q_ini)+'.pth')
    final_model_file      = os.path.join(model_dir, 'final_pretrain1_Qini'+str(Q_ini)+'.pth')
    # print('| testing %s on GPU #%d with pytorch' % (args.model_name, args.gpu))
    dec_sem = eval(args.model_name)(n_class=n_class,Q_ini=Q_ini)
    if args.gpu >= 0: dec_sem.cuda(args.gpu)
    if os.path.exists(final_model_file):
        dec_sem.load_state_dict(torch.load(final_model_file, map_location='cuda:0'))
    elif os.path.exists(checkpoint_model_file):
        dec_sem.load_state_dict(torch.load(checkpoint_model_file, map_location='cuda:0'))
    else:
        raise Exception('| model file do not exists in %s' % model_dir)

    
            
    files = os.listdir('image')
    images = []
    fpath  = []
    fpath_out = []
    fpath_original = []
    for file in files:
        if file[-3:] != 'png': continue
        fpath.append('image/'+file)
        fpath_out.append('seg_seprate_'+str(flag_csi)+'_'+str(flag_DL)+'/'+file)
        fpath_original.append('image_original/'+file)
        images.append( np.asarray(Image.open('image/'+file)) )
    images = np.asarray(images, dtype=np.float32).transpose((0,3,1,2))/255.
    images = Variable(torch.tensor(images))
    if args.gpu >= 0: images = images.cuda(args.gpu)

    loss_avg = 0.
    acc_avg  = 0.
    dec_sem.eval()
    with torch.no_grad():

        b = images.shape[0]
        
        alpha,A_r,A_t,fd,tau = SV_Channel_set(2*b,Nc,N,Nt,Nr,1,v_max_km)
        H_batch = torch.zeros(2*b,T, Nc, N_r, N_t).cuda() + 0j

        for i in range(T):
            t = i*1/14000/8 #i就表示第i个OFDM符号，每个subframe时间是1ms，每个slot包含14个OFDM符号，当子载波间隔15kHz对应一个frame包含1个slot，30kHz对应2个，60kHz->4,120kHz->8,这里采用120kHz间隔所以除以8
            H = SV_Channel_final(2*b,Nc,N,Nt,Nr,1,t,alpha,A_r,A_t,fd,tau)
            H_batch[:,i] = H
        H_batch_1 = H_batch[0:b]
        H_batch_2 = H_batch[b:2*b]

        H_CE=dec_CE(
            H_batch = H_batch,  # b,T, Nc, N_r, N_t 用户1
        );
        H_CF=dec_CF(
                H = H_CE,  # b,T, Nc, N_r, N_t 用户1
            );

        H_CE_1 = H_CE[0:b]
        H_CE_2 = H_CE[b:2*b]
        H_CF_1 = H_CF[0:b]
        H_CF_2 = H_CF[b:2*b]

        if flag_csi == 'perfect':
            H_CE_1 = H_batch[0:b,L]
            H_CE_2 = H_batch[b:2*b,L]
            H_CF_1 = H_batch[0:b,L]
            H_CF_2 = H_batch[b:2*b,L]
        elif flag_csi == 'imperfect':
            pass

        # loss_CE = SGCS_H(H_batch[:,L],H_CE)

        # loss_CF = SGCS_H(H_batch[:,L],H_CF)
        # print(loss_CE,loss_CF)


        if flag_DL == 'DL':
            W,F_1,F_2=dec_Pre(
                    H_CE_1 = H_CE_1,  # b, Nc, N_r, N_t 用户1
                    H_CE_2 = H_CE_2,  # b, Nc, N_r, N_t 用户2
                    H_CF_1 = H_CF_1,  # b, Nc, N_r, N_t 用户1
                    H_CF_2 = H_CF_2,  # b, Nc, N_r, N_t 用户2
                );


            H1 = H_batch_1[:,L:L+Q] 
            H2 = H_batch_2[:,L:L+Q] 
            H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
            H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
            H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
            H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
            W = W.reshape(b,1,1,N_r_RF,N_r)
            F_1 = F_1.reshape(b,1,1,N_t,1)
            F_2 = F_2.reshape(b,1,1,N_t,1)
            W = Pre_WY_DMRS(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2, W, F_1, F_2) #b,Q,Nc,2,N_r
        elif flag_DL == 'PCA':
            H1 = H_batch_1[:,L:L+Q] 
            H2 = H_batch_2[:,L:L+Q] 
            H1_UE = H_CE_1.reshape(b,1,Nc,N_r,N_t)
            H2_UE = H_CE_2.reshape(b,1,Nc,N_r,N_t)
            H1_base = H_CF_1.reshape(b,1,Nc,N_r,N_t)
            H2_base = H_CF_2.reshape(b,1,Nc,N_r,N_t)
            P_analog1,P_analog2,C_analog,C_digital,F_1, F_2, W = Pre_PCA_total_analog(H1, H2,  H1_UE, H2_UE, H1_base, H2_base, SNR_dB, rate1, rate2)


        norm_factor_W = torch.sqrt(torch.sum(torch.abs(W)**2, dim=(-4,-3,-2, -1), keepdim=True))/torch.sqrt(torch.tensor(Q*Nc*N_r_RF, dtype=H.dtype, device=H.device))#1,1,1,1,1

        
        # norm_factor_W = 1

        
        # Calculate effective channel
        H_eff_1 = W @ H1 @ F_1  #H [batch,Q, Nc,N_r_RF,1]
        H_eff_2 = W @ H2 @ F_2  #H [batch,Q, Nc,N_r_RF,1]

        # H_eff_1[:,:,:,0,:] = 1
        # H_eff_1[:,:,:,1,:] = 0

        # H_eff_2[:,:,:,0,:] = 0
        # H_eff_2[:,:,:,1,:] = 1

        ################################################################

        x = images

        x_rgb = dec_sem.enc_rgb(x)  #[batch,2*Q_ini,10,10]
        x_inf = dec_sem.enc_inf(x)  #[batch,2*Q_ini,10,10]


        x_rgb = x_rgb[:,0:Q_ini,:,:] + 1j*x_rgb[:,Q_ini:2*Q_ini,:,:] #[batch,Q_ini,10,10]
        norm_factor = torch.sqrt(torch.sum(torch.abs(x_rgb)**2, dim=(-3,-2, -1), keepdim=True))
        x_rgb = x_rgb / norm_factor * torch.sqrt(torch.tensor(Q*Nc*1, dtype=x.dtype, device=x.device))

        x1 = x_rgb.reshape(b,Q,Nc,1,1) #b,Nc,Q,1,1
        x1 = H_eff_1 @ x1           #b,Q,Nc,N_r_RF,1
        # x_rgb = x_rgb[:,:,:,0:1,:].reshape(b,Q,10,10)

        

        x_inf = x_inf[:,0:Q_ini,:,:] + 1j*x_inf[:,Q_ini:2*Q_ini,:,:] #[batch,Q_ini,10,10]
        norm_factor = torch.sqrt(torch.sum(torch.abs(x_inf)**2, dim=(-3,-2, -1), keepdim=True))
        x_inf = x_inf / norm_factor * torch.sqrt(torch.tensor(Q*Nc*1, dtype=x.dtype, device=x.device))

        x2 = x_inf.reshape(b,Q,Nc,1,1) #b,Nc,Q,1,1
        x2 = H_eff_2 @ x2           #b,Q,Nc,N_r_RF,1
        # x_inf = x_inf[:,:,:,1:2,:].reshape(b,Q,10,10)

        sigma = 10**(-SNR_dB/10)
        n = (torch.randn(b,Q,Nc,2,1).cuda() + 1j*torch.randn(b,Q,Nc,2,1).cuda())/sqrt(2)*sqrt(sigma)*norm_factor_W

        x = x1 + x2 + n # b,Q,Nc,N_r_RF,1
        x = x[:,:,:,0:1,:] + x[:,:,:,1:2,:]
        x = x.reshape(b,Q,10,10)

        norm_factor = torch.sqrt(torch.sum(torch.abs(x)**2, dim=(-3,-2, -1), keepdim=True))
        x = x / norm_factor * torch.sqrt(torch.tensor(Q*Nc*1, dtype=x.dtype, device=x.device))
        x=torch.cat([x.real, x.imag], dim=1);  # batch,2*Q_ini,10,10
        logits = dec_sem.dec(x)

        files = os.listdir('image')
        images = []
        fpath  = []
        fpath_out = []
        fpath_original = []
        for file in files:
            if file[-3:] != 'png': continue
            fpath.append('image/'+file)
            fpath_out.append('seprate_'+flag_DL+'_'+flag_csi+'/'+file)
            fpath_original.append('image_original/'+file)
            images.append( np.asarray(Image.open('image/'+file)) )
        images = np.asarray(images, dtype=np.float32).transpose((0,3,1,2))/255.
        images = Variable(torch.tensor(images))
        if args.gpu >= 0: images = images.cuda(args.gpu)
        
        predictions = logits.argmax(1)
        # save_images(images, fpath_original)  #保存原始图像
        visualize(fpath_out, predictions)
                
                


#### 开始测试

##### demo画图

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

# flag_csi = 'perfect'
# flag_DL = 'DL'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3

demo_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_DL='DL',flag_csi='perfect')
demo_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_DL='DL',flag_csi='imperfect')
demo_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_DL='PCA',flag_csi='perfect')
demo_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_DL='PCA',flag_csi='imperfect')

##### 不完美CSI+DL

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for Q_ini in [1,2,3,4,5]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for L in [1,2,4,8]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 1
for L in [1,2,4,8]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for feedback_bits in [4,16,64,256,1024]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 1
for feedback_bits in [4,16,64,256,1024]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for SNR_dB in [-40,-35,-30,-25,-20,-15,-10,-5,0]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini)

##### 不完美CSI+PCA

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'imperfect'
flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for Q_ini in [1,2,3,4,5]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'imperfect'
flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for L in [1,2,4,8]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'imperfect'
flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 1
for L in [1,2,4,8]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'imperfect'
flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for feedback_bits in [4,16,64,256,1024]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'imperfect'
flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 1
for feedback_bits in [4,16,64,256,1024]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'imperfect'
flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for SNR_dB in [-40,-35,-30,-25,-20,-15,-10,-5,0]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

##### 完美CSI+DL

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'perfect'
flag_DL = 'DL'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for Q_ini in [1,2,3,4,5]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'perfect'
flag_DL = 'DL'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for SNR_dB in [-40,-35,-30,-25,-20,-15,-10,-5,0]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

##### 完美CSI+PCA

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'perfect'
flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for Q_ini in [1,2,3,4,5]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)

In [ ]:
t = 0
Nc = 100
N = 2
Nt = [1,8]
Nr = [1,64]
N_r_RF = 2
N_t_RF = 2

L = 8

feedback_bits_pre = None
v_max_km_pre = None

flag_csi = 'perfect'
flag_DL = 'PCA'

feedback_bits = 1024
SNR_dB = -25
v_max_km = 120
Q_ini = 3
for SNR_dB in [-40,-35,-30,-25,-20,-15,-10,-5,0]:
    test_seprate(N,Nc,Nr,N_r_RF,Nt,N_t_RF,L,feedback_bits,SNR_dB,v_max_km,Q_ini,flag_csi=flag_csi,flag_DL=flag_DL)